In [1]:
#version 1

# Import necessary packages
import dash
from dash import dcc, html, Input, Output
import plotly.express as px
import pandas as pd
import numpy as np

# Step 2: Prepare the Data
mnch_outcomes = {
    'Preterm Birth': {'odds_ratio': 1.05, 'current_rate': 0.1},
    'Low Birth Weight': {'odds_ratio': 1.03, 'current_rate': 0.08},
    'Stillbirth': {'odds_ratio': 1.04, 'current_rate': 0.005}
}

rcp_scenarios = {
    'RCP 2.6': 1.5,  # degrees Celsius increase by 2100
    'RCP 4.5': 2.4,
    'RCP 6.0': 3.0,
    'RCP 8.5': 4.3
}

population_growth_rates = {
    'Low Growth': 0.01,  # 1% annual growth
    'Medium Growth': 0.02,  # 2% annual growth
    'High Growth': 0.03  # 3% annual growth
}
#add feritlity growth rates
fertility_growth_rates = {
    'Low Growth': 0.01, # 1% annual growth
    'Medium Growth': 0.02,
    'High Growth': 0.03
}
years = list(range(2020, 2051))

def calculate_population_growth(initial_population, growth_rate, years):
    return [initial_population * ((1 + growth_rate) ** year) for year in range(len(years))]

# Step 3: Create the Dash Layout
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("MNCH Outcomes Due to Heat - Climate Change Projection"),
    html.Label("Select MNCH Outcome"),
    dcc.Dropdown(
        id='mnch-outcome-dropdown',
        options=[{'label': outcome, 'value': outcome} for outcome in mnch_outcomes.keys()],
        value='Preterm Birth'
    ),
    html.Label("Select RCP Scenario"),
    dcc.Dropdown(
        id='rcp-scenario-dropdown',
        options=[{'label': scenario, 'value': scenario} for scenario in rcp_scenarios.keys()],
        value='RCP 2.6'
    ),
    html.Label("Select Population Growth Curve"),
    dcc.Dropdown(
        id='population-growth-dropdown',
        options=[{'label': growth, 'value': growth} for growth in population_growth_rates.keys()],
        value='Medium Growth'
    ),
    html.Label("Select Time Period"),
    dcc.RangeSlider(
        id='time-period-slider',
        min=2020,
        max=2050,
        step=1,
        marks={year: str(year) for year in years},
        value=[2020, 2030]
    ),
    html.Label("Current Annual Pregnancies"),
    dcc.Input(
        id='current-population-input',
        type='number',
        value=1000000
    ),
    html.Div(id='odds-and-rate-display'),
    dcc.Graph(id='outcome-projection-graph'),
    dcc.Graph(id='change-in-number-graph'),
    html.Button("Export Data", id="export-button"),
    dcc.Download(id="download-dataframe-csv")
])

# Step 4: Add Interactivity
@app.callback(
    Output('odds-and-rate-display', 'children'),
    Output('outcome-projection-graph', 'figure'),
    Output('change-in-number-graph', 'figure'),
    Input('mnch-outcome-dropdown', 'value'),
    Input('rcp-scenario-dropdown', 'value'),
    Input('population-growth-dropdown', 'value'),
    Input('time-period-slider', 'value'),
    Input('current-population-input', 'value')
)
def update_dashboard(outcome, rcp_scenario, population_growth_curve, time_period, current_population):
    odds_ratio = mnch_outcomes[outcome]['odds_ratio']
    current_rate = mnch_outcomes[outcome]['current_rate']
    temp_increase = rcp_scenarios[rcp_scenario]
    growth_rate = population_growth_rates[population_growth_curve]
    
    start_year, end_year = time_period
    selected_years = list(range(start_year, end_year + 1))
    
    population_growth_selected = calculate_population_growth(current_population, growth_rate, selected_years)

    projected_rates = current_rate * (odds_ratio ** (temp_increase - 1))
    projected_outcomes = [projected_rates * (pop / current_population) for pop in population_growth_selected]
    
    projected_numbers = [pop * projected_rate for pop, projected_rate in zip(population_growth_selected, projected_outcomes)]
    change_in_numbers = [number - (current_population * current_rate) for number in projected_numbers]
    
    display_text = f"Odds Ratio per 1°C increase: {odds_ratio}, Current Rate: {current_rate}"
    
    fig_outcome_projection = px.line(x=selected_years, y=projected_outcomes, labels={'x': 'Year', 'y': 'Projected Rates'},
                                     title=f"Projected {outcome} Rates from {start_year} to {end_year}")
    
    fig_change_in_number = px.line(x=selected_years, y=change_in_numbers, labels={'x': 'Year', 'y': 'Change in Number of Outcomes'},
                                   title=f"Change in Number of {outcome} Outcomes from {start_year} to {end_year}")
    
    return display_text, fig_outcome_projection, fig_change_in_number

# Export Data Callback
@app.callback(
    Output("download-dataframe-csv", "data"),
    Input("export-button", "n_clicks"),
    prevent_initial_call=True,
)
def export_data(n_clicks):
    if n_clicks is None:
        return dash.no_update

    # Retrieve values from the update_dashboard function (need to be moved to global scope)
    outcome = 'Preterm Birth'  # or get dynamically if possible
    rcp_scenario = 'RCP 2.6'  # or get dynamically if possible
    population_growth_curve = 'Medium Growth'  # or get dynamically if possible
    time_period = [2020, 2030]  # or get dynamically if possible
    current_population = 1000000  # or get dynamically if possible
    
    odds_ratio = mnch_outcomes[outcome]['odds_ratio']
    current_rate = mnch_outcomes[outcome]['current_rate']
    temp_increase = rcp_scenarios[rcp_scenario]
    growth_rate = population_growth_rates[population_growth_curve]
    
    start_year, end_year = time_period
    selected_years = list(range(start_year, end_year + 1))
    
    population_growth_selected = calculate_population_growth(current_population, growth_rate, selected_years)

    projected_rates = current_rate * (odds_ratio ** (temp_increase - 1))
    projected_outcomes = [projected_rates * (pop / current_population) for pop in population_growth_selected]
    
    projected_numbers = [pop * projected_rate for pop, projected_rate in zip(population_growth_selected, projected_outcomes)]
    change_in_numbers = [number - (current_population * current_rate) for number in projected_numbers]

    # Create a DataFrame with the current projections
    data = {
        'Year': selected_years,
        'Projected Rates': projected_outcomes,
        'Projected Numbers': projected_numbers,
        'Change in Number of Outcomes': change_in_numbers
    }
    df = pd.DataFrame(data)

    return dcc.send_data_frame(df.to_csv, "projected_outcomes.csv")

# Step 5: Run the App
if __name__ == '__main__':
    app.run_server(debug=True)


Address already in use
Port 8050 is in use by another program. Either identify and stop that program, or start the server with a different port.


AttributeError: 'tuple' object has no attribute 'tb_frame'

In [3]:
#version 2
# Import necessary packages
import dash
from dash import dcc, html, Input, Output
import plotly.express as px
import pandas as pd
import numpy as np

# Step 2: Prepare the Data
mnch_outcomes = {
    'Preterm Birth': {'odds_ratio': 1.05, 'current_rate': 0.1},
    'Low Birth Weight': {'odds_ratio': 1.03, 'current_rate': 0.08},
    'Stillbirth': {'odds_ratio': 1.04, 'current_rate': 0.005}
}

rcp_scenarios = {
    'RCP 2.6': 1.5,  # degrees Celsius increase by 2100
    'RCP 4.5': 2.4,
    'RCP 6.0': 3.0,
    'RCP 8.5': 4.3
}

population_growth_rates = {
    'Low Growth': 0.01,  # 1% annual growth
    'Medium Growth': 0.02,  # 2% annual growth
    'High Growth': 0.03  # 3% annual growth
}

pregnancy_growth_rates = {
    'Low Growth': 0.01,  # 1% annual growth
    'Medium Growth': 0.02,  # 2% annual growth
    'High Growth': 0.03  # 3% annual growth
}

years = list(range(2024, 2051))

def calculate_population_growth(initial_population, growth_rate, num_years):
    return [initial_population * ((1 + growth_rate) ** year) for year in range(num_years)]

def calculate_pregnancies(population, pregnancy_rate):
    return [pop * pregnancy_rate for pop in population]

# Step 3: Create the Dash Layout
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("MNCH Outcomes Due to Heat - Climate Change Projection"),
    html.Label("Select MNCH Outcome"),
    dcc.Dropdown(
        id='mnch-outcome-dropdown',
        options=[{'label': outcome, 'value': outcome} for outcome in mnch_outcomes.keys()],
        value='Preterm Birth'
    ),
    html.Label("Select RCP Scenario"),
    dcc.Dropdown(
        id='rcp-scenario-dropdown',
        options=[{'label': scenario, 'value': scenario} for scenario in rcp_scenarios.keys()],
        value='RCP 2.6'
    ),
    html.Label("Select Population Growth Curve"),
    dcc.Dropdown(
        id='population-growth-dropdown',
        options=[{'label': growth, 'value': growth} for growth in population_growth_rates.keys()],
        value='Medium Growth'
    ),
    html.Label("Select Pregnancy Growth Rate"),
    dcc.Dropdown(
        id='pregnancy-growth-dropdown',
        options=[{'label': growth, 'value': growth} for growth in pregnancy_growth_rates.keys()],
        value='Medium Growth'
    ),
    html.Label("Select Time Period"),
    dcc.RangeSlider(
        id='time-period-slider',
        min=2024,
        max=2050,
        step=1,
        marks={year: str(year) for year in years},
        value=[2024, 2030]
    ),
    html.Label("Current Population"),
    dcc.Input(
        id='current-population-input',
        type='number',
        value=7000000000
    ),
    html.Label("Initial Pregnancy Rate"),
    dcc.Input(
        id='initial-pregnancy-rate-input',
        type='number',
        value=0.02,  # 2% initial pregnancy rate
        step=0.001
    ),
    html.Div(id='odds-and-rate-display'),
    dcc.Graph(id='outcome-projection-graph'),
    dcc.Graph(id='change-in-number-graph'),
    html.Div(id='additional-outcome-display', style={'margin-top': '20px', 'font-size': '20px'}),
    html.Button("Export Data", id="export-button"),
    dcc.Download(id="download-dataframe-csv")
])

# Step 4: Add Interactivity
@app.callback(
    Output('odds-and-rate-display', 'children'),
    Output('outcome-projection-graph', 'figure'),
    Output('change-in-number-graph', 'figure'),
    Output('additional-outcome-display', 'children'),
    Input('mnch-outcome-dropdown', 'value'),
    Input('rcp-scenario-dropdown', 'value'),
    Input('population-growth-dropdown', 'value'),
    Input('pregnancy-growth-dropdown', 'value'),
    Input('time-period-slider', 'value'),
    Input('current-population-input', 'value'),
    Input('initial-pregnancy-rate-input', 'value')
)
def update_dashboard(outcome, rcp_scenario, population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate):
    odds_ratio = mnch_outcomes[outcome]['odds_ratio']
    current_rate = mnch_outcomes[outcome]['current_rate']
    temp_increase = rcp_scenarios[rcp_scenario]
    population_growth_rate = population_growth_rates[population_growth_curve]
    pregnancy_growth_rate = pregnancy_growth_rates[pregnancy_growth_curve]
    
    start_year, end_year = time_period
    num_years = end_year - start_year + 1
    selected_years = list(range(start_year, end_year + 1))
    
    # Calculate population growth
    population_growth_selected = calculate_population_growth(current_population, population_growth_rate, num_years)
    
    # Calculate pregnancies without the effect of heat
    pregnancies = calculate_pregnancies(population_growth_selected, initial_pregnancy_rate)
    
    # Calculate baseline outcomes without the effect of heat
    baseline_outcomes = [preg * current_rate for preg in pregnancies]
    
    # Calculate projected outcomes with the effect of heat
    projected_rates = [current_rate * (odds_ratio ** ((temp_increase * year) / (end_year - start_year))) for year in range(num_years)]
    projected_outcomes = [rate * preg for rate, preg in zip(projected_rates, pregnancies)]
    
    # Calculate additional outcomes due to the effect of heat
    additional_outcomes = [proj - base for proj, base in zip(projected_outcomes, baseline_outcomes)]
    
    total_additional_outcomes = sum(additional_outcomes)
    
    display_text = f"Odds Ratio per 1°C increase: {odds_ratio}, Current Rate: {current_rate}, Initial Pregnancy Rate: {initial_pregnancy_rate}"
    
    fig_outcome_projection = px.line(x=selected_years, y=projected_outcomes, labels={'x': 'Year', 'y': 'Projected Outcomes'},
                                     title=f"Projected {outcome} Outcomes from {start_year} to {end_year}")
    
    fig_change_in_number = px.line(x=selected_years, y=additional_outcomes, labels={'x': 'Year', 'y': 'Additional Outcomes'},
                                   title=f"Additional {outcome} Outcomes due to Heat from {start_year} to {end_year}")
    
    additional_outcome_text = f"Total Additional {outcome} Outcomes from {start_year} to {end_year}: {total_additional_outcomes:.2f}"
    
    return display_text, fig_outcome_projection, fig_change_in_number, additional_outcome_text

# Export Data Callback
@app.callback(
    Output("download-dataframe-csv", "data"),
    Input("export-button", "n_clicks"),
    prevent_initial_call=True,
)
def export_data(n_clicks):
    if n_clicks is None:
        return dash.no_update

    # Retrieve values from the update_dashboard function (need to be moved to global scope)
    outcome = 'Preterm Birth'  # or get dynamically if possible
    rcp_scenario = 'RCP 2.6'  # or get dynamically if possible
    population_growth_curve = 'Medium Growth'  # or get dynamically if possible
    pregnancy_growth_curve = 'Medium Growth'  # or get dynamically if possible
    time_period = [2024, 2030]  # or get dynamically if possible
    current_population = 7000000000  # or get dynamically if possible
    initial_pregnancy_rate = 0.02  # or get dynamically if possible
    
    odds_ratio = mnch_outcomes[outcome]['odds_ratio']
    current_rate = mnch_outcomes[outcome]['current_rate']
    temp_increase = rcp_scenarios[rcp_scenario]
    population_growth_rate = population_growth_rates[population_growth_curve]
    pregnancy_growth_rate = pregnancy_growth_rates[pregnancy_growth_curve]
    
    start_year, end_year = time_period
    num_years = end_year - start_year + 1
    selected_years = list(range(start_year, end_year + 1))
    
    # Calculate population growth
    population_growth_selected = calculate_population_growth(current_population, population_growth_rate, num_years)
    
    # Calculate pregnancies without the effect of heat
    pregnancies = calculate_pregnancies(population_growth_selected, initial_pregnancy_rate)
    
    # Calculate baseline outcomes without the effect of heat
    baseline_outcomes = [preg * current_rate for preg in pregnancies]
    
    # Calculate projected outcomes with the effect of heat
    projected_rates = [current_rate * (odds_ratio ** ((temp_increase * year) / (end_year - start_year))) for year in range(num_years)]
    projected_outcomes = [rate * preg for rate, preg in zip(projected_rates, pregnancies)]
    
    # Calculate additional outcomes due to the effect of heat
    additional_outcomes = [proj - base for proj, base in zip(projected_outcomes, baseline_outcomes)]

    # Create a DataFrame with the current projections
    data = {
        'Year': selected_years,
        'Projected Outcomes': projected_outcomes,
        'Baseline Outcomes': baseline_outcomes,
        'Additional Outcomes': additional_outcomes
    }
    df = pd.DataFrame(data)

    return dcc.send_data_frame(df.to_csv, "projected_outcomes.csv")

# Step 5: Run the App
if __name__ == '__main__':
    app.run_server(debug=True)



Address already in use
Port 8050 is in use by another program. Either identify and stop that program, or start the server with a different port.


AttributeError: 'tuple' object has no attribute 'tb_frame'

In [ ]:
#version 3
# Import necessary packages
import dash
from dash import dcc, html, Input, Output
import plotly.graph_objs as go
import pandas as pd
import numpy as np

# Step 2: Prepare the Data
mnch_outcomes = {
    'Preterm Birth': {'odds_ratio': 1.05, 'current_rate': 0.1},
    'Low Birth Weight': {'odds_ratio': 1.03, 'current_rate': 0.08},
    'Stillbirth': {'odds_ratio': 1.04, 'current_rate': 0.005}
}

rcp_scenarios = {
    'RCP 2.6': 1.5,  # degrees Celsius increase by 2100
    'RCP 4.5': 2.4,
    'RCP 6.0': 3.0,
    'RCP 8.5': 4.3
}

population_growth_rates = {
    'Low Growth': 0.01,  # 1% annual growth
    'Medium Growth': 0.02,  # 2% annual growth
    'High Growth': 0.03  # 3% annual growth
}

pregnancy_growth_rates = {
    'Low Growth': 0.01,  # 1% annual growth
    'Medium Growth': 0.02,  # 2% annual growth
    'High Growth': 0.03  # 3% annual growth
}

years = list(range(2024, 2051))

def calculate_population_growth(initial_population, growth_rate, num_years):
    return [initial_population * ((1 + growth_rate) ** year) for year in range(num_years)]

def calculate_pregnancies(population, pregnancy_rate):
    return [pop * pregnancy_rate for pop in population]

def calculate_temperature_increase_per_year(temp_increase_2100, start_year, end_year):
    total_years = 2100 - 2024  # Calculate total years from start year to 2100
    annual_increase = temp_increase_2100 / total_years
    return [annual_increase * (year - 2024) for year in range(start_year, end_year + 1)]

# Step 3: Create the Dash Layout
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("MNCH Outcomes Due to Heat - Climate Change Projection"),
    html.Label("Select MNCH Outcome"),
    dcc.Dropdown(
        id='mnch-outcome-dropdown',
        options=[{'label': outcome, 'value': outcome} for outcome in mnch_outcomes.keys()],
        value='Preterm Birth'
    ),
    html.Label("Select RCP Scenario"),
    dcc.Dropdown(
        id='rcp-scenario-dropdown',
        options=[{'label': scenario, 'value': scenario} for scenario in rcp_scenarios.keys()],
        value='RCP 2.6'
    ),
    html.Label("Select Population Growth Curve"),
    dcc.Dropdown(
        id='population-growth-dropdown',
        options=[{'label': growth, 'value': growth} for growth in population_growth_rates.keys()],
        value='Medium Growth'
    ),
    html.Label("Select Pregnancy Growth Rate"),
    dcc.Dropdown(
        id='pregnancy-growth-dropdown',
        options=[{'label': growth, 'value': growth} for growth in pregnancy_growth_rates.keys()],
        value='Medium Growth'
    ),
    html.Label("Select Time Period"),
    dcc.RangeSlider(
        id='time-period-slider',
        min=2024,
        max=2050,
        step=1,
        marks={year: str(year) for year in years},
        value=[2024, 2030]
    ),
    html.Label("Current Population"),
    dcc.Input(
        id='current-population-input',
        type='number',
        value=7000000000
    ),
    html.Label("Initial Pregnancy Rate"),
    dcc.Input(
        id='initial-pregnancy-rate-input',
        type='number',
        value=0.02,  # 2% initial pregnancy rate
        step=0.001
    ),
    html.Div(id='odds-and-rate-display'),
    dcc.Graph(id='outcome-projection-graph'),
    html.Div(id='additional-outcome-display', style={'margin-top': '20px', 'font-size': '20px'}),
    html.Button("Export Data", id="export-button"),
    dcc.Download(id="download-dataframe-csv")
])

# Step 4: Add Interactivity
@app.callback(
    Output('odds-and-rate-display', 'children'),
    Output('outcome-projection-graph', 'figure'),
    Output('additional-outcome-display', 'children'),
    Input('mnch-outcome-dropdown', 'value'),
    Input('rcp-scenario-dropdown', 'value'),
    Input('population-growth-dropdown', 'value'),
    Input('pregnancy-growth-dropdown', 'value'),
    Input('time-period-slider', 'value'),
    Input('current-population-input', 'value'),
    Input('initial-pregnancy-rate-input', 'value')
)
def update_dashboard(outcome, rcp_scenario, population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate):
    odds_ratio = mnch_outcomes[outcome]['odds_ratio']
    current_rate = mnch_outcomes[outcome]['current_rate']
    temp_increase_2100 = rcp_scenarios[rcp_scenario]
    population_growth_rate = population_growth_rates[population_growth_curve]
    pregnancy_growth_rate = pregnancy_growth_rates[pregnancy_growth_curve]
    
    start_year, end_year = time_period
    num_years = end_year - start_year + 1
    selected_years = list(range(start_year, end_year + 1))
    
    # Calculate population growth
    population_growth_selected = calculate_population_growth(current_population, population_growth_rate, num_years)
    
    # Calculate pregnancies without the effect of heat
    pregnancies = calculate_pregnancies(population_growth_selected, initial_pregnancy_rate)
    
    # Calculate baseline outcomes without the effect of heat
    baseline_outcomes = [preg * current_rate for preg in pregnancies]
    
    # Calculate the temperature increase per year
    temperature_increase_per_year = calculate_temperature_increase_per_year(temp_increase_2100, start_year, end_year)
    
    # Calculate projected outcomes with the effect of heat
    projected_rates = [current_rate * (odds_ratio ** temp_increase) for temp_increase in temperature_increase_per_year]
    projected_outcomes = [rate * preg for rate, preg in zip(projected_rates, pregnancies)]
    
    # Calculate additional outcomes due to the effect of heat
    additional_outcomes = [proj - base for proj, base in zip(projected_outcomes, baseline_outcomes)]
    
    total_additional_outcomes = sum(additional_outcomes)
    
    display_text = f"Odds Ratio per 1°C increase: {odds_ratio}, Current Rate: {current_rate}, Initial Pregnancy Rate: {initial_pregnancy_rate}"
    
    fig = go.Figure()
    
    # Add projected outcomes
    fig.add_trace(go.Scatter(x=selected_years, y=projected_outcomes, mode='lines', name='Projected Outcomes'))
    
    # Add additional outcomes
    fig.add_trace(go.Scatter(x=selected_years, y=additional_outcomes, mode='lines', name='Additional Outcomes', yaxis='y2'))
    
    # Add temperature increase
    fig.add_trace(go.Scatter(x=selected_years, y=temperature_increase_per_year, mode='lines', name='Temperature Increase', yaxis='y3'))
    
    # Update layout for multiple y-axes
    fig.update_layout(
        title=f"Projected {outcome} Outcomes and Additional Outcomes Due to Heat from {start_year} to {end_year}",
        xaxis_title="Year",
        yaxis=dict(
            title="Projected Outcomes",
            titlefont=dict(color="#1f77b4"),
            tickfont=dict(color="#1f77b4")
        ),
        yaxis2=dict(
            title="Additional Outcomes",
            titlefont=dict(color="#ff7f0e"),
            tickfont=dict(color="#ff7f0e"),
            overlaying='y',
            side='right'
        ),
        yaxis3=dict(
            title="Temperature Increase (°C)",
            titlefont=dict(color="#2ca02c"),
            tickfont=dict(color="#2ca02c"),
            overlaying='y',
            side='right',
            anchor='free',
            position=1
        )
    )
    
    additional_outcome_text = f"Total Additional {outcome} Outcomes from {start_year} to {end_year}: {total_additional_outcomes:.2f}"
    
    return display_text, fig, additional_outcome_text

# Export Data Callback
@app.callback(
    Output("download-dataframe-csv", "data"),
    Input("export-button", "n_clicks"),
    prevent_initial_call=True,
)
def export_data(n_clicks):
    if n_clicks is None:
        return dash.no_update

    # Retrieve values from the update_dashboard function (need to be moved to global scope)
    outcome = 'Preterm Birth'  # or get dynamically if possible
    rcp_scenario = 'RCP 2.6'  # or get dynamically if possible
    population_growth_curve = 'Medium Growth'  # or get dynamically if possible
    pregnancy_growth_curve = 'Medium Growth'  # or get dynamically if possible
    time_period = [2024, 2030]  # or get dynamically if possible
    current_population = 7000000000  # or get dynamically if possible
    initial_pregnancy_rate = 0.02  # or get dynamically if possible
    
    odds_ratio = mnch_outcomes[outcome]['odds_ratio']
    current_rate = mnch_outcomes[outcome]['current_rate']
    temp_increase_2100 = rcp_scenarios[rcp_scenario]
    population_growth_rate = population_growth_rates[population_growth_curve]
    pregnancy_growth_rate = pregnancy_growth_rates[pregnancy_growth_curve]
    
    start_year, end_year = time_period
    num_years = end_year - start_year + 1
    selected_years = list(range(start_year, end_year + 1))
    
    # Calculate population growth
    population_growth_selected = calculate_population_growth(current_population, population_growth_rate, num_years)
    
    # Calculate pregnancies without the effect of heat
    pregnancies = calculate_pregnancies(population_growth_selected, initial_pregnancy_rate)
    
    # Calculate baseline outcomes without the effect of heat
    baseline_outcomes = [preg * current_rate for preg in pregnancies]
    
    # Calculate the temperature increase per year
    temperature_increase_per_year = calculate_temperature_increase_per_year(temp_increase_2100, start_year, end_year)
    
    # Calculate projected outcomes with the effect of heat
    projected_rates = [current_rate * (odds_ratio ** temp_increase) for temp_increase in temperature_increase_per_year]
    projected_outcomes = [rate * preg for rate, preg in zip(projected_rates, pregnancies)]
    
    # Calculate additional outcomes due to the effect of heat
    additional_outcomes = [proj - base for proj, base in zip(projected_outcomes, baseline_outcomes)]

    # Create a DataFrame with the current projections
    data = {
        'Year': selected_years,
        'Projected Outcomes': projected_outcomes,
        'Baseline Outcomes': baseline_outcomes,
        'Additional Outcomes': additional_outcomes,
        'Temperature Increase': temperature_increase_per_year
    }
    df = pd.DataFrame(data)

    return dcc.send_data_frame(df.to_csv, "projected_outcomes.csv")

# Step 5: Run the App
if __name__ == '__main__':
    app.run_server(debug=True)


In [1]:
#version 4
# Import necessary packages
import dash
from dash import dcc, html, Input, Output
import plotly.graph_objs as go
import pandas as pd
import numpy as np

# Step 2: Prepare the Data
mnch_outcomes = {
    'Preterm Birth': {'odds_ratio': 1.05, 'current_rate': 0.1},
    'Low Birth Weight': {'odds_ratio': 1.03, 'current_rate': 0.08},
    'Stillbirth': {'odds_ratio': 1.04, 'current_rate': 0.005}
}

rcp_scenarios = {
    'RCP 2.6': 1.5,  # degrees Celsius increase by 2100
    'RCP 4.5': 2.4,
    'RCP 6.0': 3.0,
    'RCP 8.5': 4.3
}

population_growth_rates = {
    'Low Growth': 0.01,  # 1% annual growth
    'Medium Growth': 0.02,  # 2% annual growth
    'High Growth': 0.03  # 3% annual growth
}

pregnancy_growth_rates = {
    'Low Growth': 0.01,  # 1% annual growth
    'Medium Growth': 0.02,  # 2% annual growth
    'High Growth': 0.03  # 3% annual growth
}

years = list(range(2024, 2100))

def calculate_population_growth(initial_population, growth_rate, num_years):
    return [initial_population * ((1 + growth_rate) ** year) for year in range(num_years)]

def calculate_pregnancies(population, pregnancy_rate):
    return [pop * pregnancy_rate for pop in population]

def calculate_temperature_increase_per_year(temp_increase_2100, start_year, end_year):
    total_years = 2100 - 2024  # Calculate total years from 2024 to 2100
    annual_increase = temp_increase_2100 / total_years
    return [annual_increase * (year - 2024) for year in range(start_year, end_year + 1)]

# Step 3: Create the Dash Layout
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("MNCH Outcomes Due to Heat - Climate Change Projection"),
    html.Label("Select MNCH Outcome"),
    dcc.Dropdown(
        id='mnch-outcome-dropdown',
        options=[{'label': outcome, 'value': outcome} for outcome in mnch_outcomes.keys()],
        value='Preterm Birth'
    ),
    html.Label("Select RCP Scenario"),
    dcc.Dropdown(
        id='rcp-scenario-dropdown',
        options=[{'label': scenario, 'value': scenario} for scenario in rcp_scenarios.keys()],
        value='RCP 2.6'
    ),
    html.Label("Select Population Growth Curve"),
    dcc.Dropdown(
        id='population-growth-dropdown',
        options=[{'label': growth, 'value': growth} for growth in population_growth_rates.keys()],
        value='Medium Growth'
    ),
    html.Label("Select Pregnancy Growth Rate"),
    dcc.Dropdown(
        id='pregnancy-growth-dropdown',
        options=[{'label': growth, 'value': growth} for growth in pregnancy_growth_rates.keys()],
        value='Medium Growth'
    ),
    html.Label("Select Time Period"),
    dcc.RangeSlider(
        id='time-period-slider',
        min=2024,
        max=2051,
        step=1,
        marks={year: str(year) for year in years},
        value=[2024, 2030]
    ),
    html.Label("Current Population"),
    dcc.Input(
        id='current-population-input',
        type='number',
        value=7000000000
    ),
    html.Label("Initial Pregnancy Rate"),
    dcc.Input(
        id='initial-pregnancy-rate-input',
        type='number',
        value=0.02,  # 2% initial pregnancy rate
        step=0.001
    ),
    html.Div(id='odds-and-rate-display'),
    dcc.Graph(id='outcome-projection-graph'),
    html.Div(id='additional-outcome-display', style={'margin-top': '20px', 'font-size': '20px'}),
    html.Button("Export Data", id="export-button"),
    dcc.Download(id="download-dataframe-csv")
])

# Step 4: Add Interactivity
@app.callback(
    Output('odds-and-rate-display', 'children'),
    Output('outcome-projection-graph', 'figure'),
    Output('additional-outcome-display', 'children'),
    Input('mnch-outcome-dropdown', 'value'),
    Input('rcp-scenario-dropdown', 'value'),
    Input('population-growth-dropdown', 'value'),
    Input('pregnancy-growth-dropdown', 'value'),
    Input('time-period-slider', 'value'),
    Input('current-population-input', 'value'),
    Input('initial-pregnancy-rate-input', 'value')
)
def update_dashboard(outcome, rcp_scenario, population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate):
    odds_ratio = mnch_outcomes[outcome]['odds_ratio']
    current_rate = mnch_outcomes[outcome]['current_rate']
    temp_increase_2100 = rcp_scenarios[rcp_scenario]
    population_growth_rate = population_growth_rates[population_growth_curve]
    pregnancy_growth_rate = pregnancy_growth_rates[pregnancy_growth_curve]
    
    start_year, end_year = time_period
    num_years = end_year - start_year + 1
    selected_years = list(range(start_year, end_year + 1))
    
    # Calculate population growth
    population_growth_selected = calculate_population_growth(current_population, population_growth_rate, num_years)
    
    # Calculate pregnancies without the effect of heat
    pregnancies = calculate_pregnancies(population_growth_selected, initial_pregnancy_rate)
    
    # Calculate baseline outcomes without the effect of heat
    baseline_outcomes = [preg * current_rate for preg in pregnancies]
    
    # Calculate the temperature increase per year
    temperature_increase_per_year = calculate_temperature_increase_per_year(temp_increase_2100, start_year, end_year)
    
    # Calculate projected outcomes with the effect of heat
    projected_rates = [current_rate * (odds_ratio ** temp_increase) for temp_increase in temperature_increase_per_year]
    projected_outcomes = [rate * preg for rate, preg in zip(projected_rates, pregnancies)]
    
    # Calculate additional outcomes due to the effect of heat
    additional_outcomes = [proj - base for proj, base in zip(projected_outcomes, baseline_outcomes)]
    
    total_additional_outcomes = sum(additional_outcomes)
    
    display_text = f"Odds Ratio per 1°C increase: {odds_ratio}, Current Rate: {current_rate}, Initial Pregnancy Rate: {initial_pregnancy_rate}"
    
    fig = go.Figure()
    
    # Add additional outcomes
    fig.add_trace(go.Scatter(x=selected_years, y=additional_outcomes, mode='lines', name='Additional Outcomes'))
    
    # Add temperature increase
    fig.add_trace(go.Scatter(x=selected_years, y=temperature_increase_per_year, mode='lines', name='Temperature Increase', yaxis='y2'))
    
    # Update layout for multiple y-axes
    fig.update_layout(
        title=f"Additional {outcome} Outcomes Due to Heat and Temperature Increase from {start_year} to {end_year}",
        xaxis_title="Year",
        yaxis=dict(
            title="Additional Outcomes",
            titlefont=dict(color="#1f77b4"),
            tickfont=dict(color="#1f77b4")
        ),
        yaxis2=dict(
            title="Temperature Increase (°C)",
            titlefont=dict(color="#ff7f0e"),
            tickfont=dict(color="#ff7f0e"),
            overlaying='y',
            side='right'
        )
    )
    
    additional_outcome_text = f"Total Additional {outcome} Outcomes due to heat from {start_year} to {end_year}: {total_additional_outcomes:.2f}"
    
    return display_text, fig, additional_outcome_text

# Export Data Callback
@app.callback(
    Output("download-dataframe-csv", "data"),
    Input("export-button", "n_clicks"),
    prevent_initial_call=True,
)
def export_data(n_clicks):
    if n_clicks is None:
        return dash.no_update

    # Retrieve values from the update_dashboard function (need to be moved to global scope)
    outcome = 'Preterm Birth'  # or get dynamically if possible
    rcp_scenario = 'RCP 2.6'  # or get dynamically if possible
    population_growth_curve = 'Medium Growth'  # or get dynamically if possible
    pregnancy_growth_curve = 'Medium Growth'  # or get dynamically if possible
    time_period = [2024, 2030]  # or get dynamically if possible
    current_population = 7000000000  # or get dynamically if possible
    initial_pregnancy_rate = 0.02  # or get dynamically if possible
    
    odds_ratio = mnch_outcomes[outcome]['odds_ratio']
    current_rate = mnch_outcomes[outcome]['current_rate']
    temp_increase_2100 = rcp_scenarios[rcp_scenario]
    population_growth_rate = population_growth_rates[population_growth_curve]
    pregnancy_growth_rate = pregnancy_growth_rates[pregnancy_growth_curve]
    
    start_year, end_year = time_period
    num_years = end_year - start_year + 1
    selected_years = list(range(start_year, end_year + 1))
    
    # Calculate population growth
    population_growth_selected = calculate_population_growth(current_population, population_growth_rate, num_years)
    
    # Calculate pregnancies without the effect of heat
    pregnancies = calculate_pregnancies(population_growth_selected, initial_pregnancy_rate)
    
    # Calculate baseline outcomes without the effect of heat
    baseline_outcomes = [preg * current_rate for preg in pregnancies]
    
    # Calculate the temperature increase per year
    temperature_increase_per_year = calculate_temperature_increase_per_year(temp_increase_2100, start_year, end_year)
    
    # Calculate projected outcomes with the effect of heat
    projected_rates = [current_rate * (odds_ratio ** temp_increase) for temp_increase in temperature_increase_per_year]
    projected_outcomes = [rate * preg for rate, preg in zip(projected_rates, pregnancies)]
    
    # Calculate additional outcomes due to the effect of heat
    additional_outcomes = [proj - base for proj, base in zip(projected_outcomes, baseline_outcomes)]

    # Create a DataFrame with the current projections
    data = {
        'Year': selected_years,
        'Additional Outcomes': additional_outcomes,
        'Temperature Increase': temperature_increase_per_year
    }
    df = pd.DataFrame(data)

    return dcc.send_data_frame(df.to_csv, "projected_outcomes.csv")

# Step 5: Run the App
if __name__ == '__main__':
    app.run_server(debug=True)


In [15]:
#version 5
# Import necessary packages
import dash
from dash import dcc, html, Input, Output
import plotly.graph_objs as go
import pandas as pd
import numpy as np

# Step 2: Prepare the Data
mnch_outcomes = {
    'Preterm Birth': {'odds_ratio': 1.05, 'current_rate': 0.1},
    'Low Birth Weight': {'odds_ratio': 1.03, 'current_rate': 0.08},
    'Stillbirth': {'odds_ratio': 1.04, 'current_rate': 0.005}
}

rcp_scenarios = {
    'RCP 2.6': 1.5,  # degrees Celsius increase by 2100
    'RCP 4.5': 2.4,
    'RCP 6.0': 3.0,
    'RCP 8.5': 4.3
}

population_growth_rates = {
    'Low Growth': 0.01,  # 1% annual growth
    'Medium Growth': 0.02,  # 2% annual growth
    'High Growth': 0.03  # 3% annual growth
}

pregnancy_growth_rates = {
    'High Reduction': -0.15,  # 15% reduction
    'Medium Reduction': -0.1,  # 10% reduction
    'Low Reduction': -0.05,  # 5% reduction
    'Low Growth': 0.01,  # 1% annual growth
    'Medium Growth': 0.02,  # 2% annual growth
    'High Growth': 0.03  # 3% annual growth
}

years = list(range(2024, 2051))

def calculate_population_growth(initial_population, growth_rate, num_years):
    return [initial_population * ((1 + growth_rate) ** year) for year in range(num_years)]

def calculate_pregnancies(population, pregnancy_rate):
    return [pop * pregnancy_rate for pop in population]

def calculate_temperature_increase_per_year(temp_increase_2100, start_year, end_year):
    total_years = 2100 - 2024  # Calculate total years from 2024 to 2100
    annual_increase = temp_increase_2100 / total_years
    return [annual_increase * (year - 2024) for year in range(start_year, end_year + 1)]

# Step 3: Create the Dash Layout
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("MNCH Outcomes Due to Heat - Climate Change Projection"),
    html.Label("Select MNCH Outcome"),
    dcc.Dropdown(
        id='mnch-outcome-dropdown',
        options=[{'label': outcome, 'value': outcome} for outcome in mnch_outcomes.keys()],
        value='Preterm Birth'
    ),
    html.Label("Select RCP Scenario"),
    dcc.Dropdown(
        id='rcp-scenario-dropdown',
        options=[{'label': scenario, 'value': scenario} for scenario in rcp_scenarios.keys()],
        value='RCP 2.6'
    ),
    html.Label("Select Population Growth Curve"),
    dcc.Dropdown(
        id='population-growth-dropdown',
        options=[{'label': growth, 'value': growth} for growth in population_growth_rates.keys()],
        value='Medium Growth'
    ),
    html.Label("Select Pregnancy Growth Rate"),
    dcc.Dropdown(
        id='pregnancy-growth-dropdown',
        options=[{'label': growth, 'value': growth} for growth in pregnancy_growth_rates.keys()],
        value='Medium Growth'
    ),
    html.Label("Select Time Period"),
    dcc.RangeSlider(
        id='time-period-slider',
        min=2024,
        max=2050,
        step=1,
        marks={year: str(year) for year in years},
        value=[2024, 2030]
    ),
    html.Label("Current Population"),
    dcc.Input(
        id='current-population-input',
        type='number',
        value=7000000000
    ),
    html.Label("Initial Pregnancy Rate"),
    dcc.Input(
        id='initial-pregnancy-rate-input',
        type='number',
        value=0.02,  # 2% initial pregnancy rate
        step=0.001
    ),
    html.Div(id='odds-and-rate-display'),
    dcc.Graph(id='outcome-projection-graph'),
    html.Div(id='additional-outcome-display', style={'margin-top': '20px', 'font-size': '20px'}),
    html.Button("Export Data", id="export-button"),
    dcc.Download(id="download-dataframe-csv")
])

# Step 4: Add Interactivity
@app.callback(
    Output('odds-and-rate-display', 'children'),
    Output('outcome-projection-graph', 'figure'),
    Output('additional-outcome-display', 'children'),
    Input('mnch-outcome-dropdown', 'value'),
    Input('rcp-scenario-dropdown', 'value'),
    Input('population-growth-dropdown', 'value'),
    Input('pregnancy-growth-dropdown', 'value'),
    Input('time-period-slider', 'value'),
    Input('current-population-input', 'value'),
    Input('initial-pregnancy-rate-input', 'value')
)
def update_dashboard(outcome, rcp_scenario, population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate):
    odds_ratio = mnch_outcomes[outcome]['odds_ratio']
    current_rate = mnch_outcomes[outcome]['current_rate']
    temp_increase_2100 = rcp_scenarios[rcp_scenario]
    population_growth_rate = population_growth_rates[population_growth_curve]
    pregnancy_growth_rate = pregnancy_growth_rates[pregnancy_growth_curve]
    
    start_year, end_year = time_period
    num_years = end_year - start_year + 1
    selected_years = list(range(start_year, end_year + 1))
    
    # Calculate population growth
    population_growth_selected = calculate_population_growth(current_population, population_growth_rate, num_years)
    
    # Calculate pregnancies without the effect of heat
    pregnancies = calculate_pregnancies(population_growth_selected, initial_pregnancy_rate * (1 + pregnancy_growth_rate))
    
    # Calculate baseline outcomes without the effect of heat
    baseline_outcomes = [preg * current_rate for preg in pregnancies]
    
    # Calculate the temperature increase per year
    temperature_increase_per_year = calculate_temperature_increase_per_year(temp_increase_2100, start_year, end_year)
    
    # Calculate projected outcomes with the effect of heat
    projected_rates = [current_rate * (odds_ratio ** temp_increase) for temp_increase in temperature_increase_per_year]
    projected_outcomes = [rate * preg for rate, preg in zip(projected_rates, pregnancies)]
    
    # Calculate additional outcomes due to the effect of heat
    additional_outcomes = [proj - base for proj, base in zip(projected_outcomes, baseline_outcomes)]
    
    total_additional_outcomes = sum(additional_outcomes)
    
    display_text = f"Odds Ratio per 1°C increase: {odds_ratio}, Current Rate: {current_rate}, Initial Pregnancy Rate: {initial_pregnancy_rate}"
    
    fig = go.Figure()
    
    # Add additional outcomes
    fig.add_trace(go.Scatter(x=selected_years, y=additional_outcomes, mode='lines', name='Additional Outcomes'))
    
    # Add temperature increase
    fig.add_trace(go.Scatter(x=selected_years, y=temperature_increase_per_year, mode='lines', name='Temperature Increase', yaxis='y2'))
    
    # Update layout for multiple y-axes
    fig.update_layout(
        title=f"Additional {outcome} Outcomes Due to Heat and Temperature Increase from {start_year} to {end_year}",
        xaxis_title="Year",
        yaxis=dict(
            title="Additional Outcomes",
            titlefont=dict(color="#1f77b4"),
            tickfont=dict(color="#1f77b4")
        ),
        yaxis2=dict(
            title="Temperature Increase (°C)",
            titlefont=dict(color="#ff7f0e"),
            tickfont=dict(color="#ff7f0e"),
            overlaying='y',
            side='right'
        )
    )
    
    additional_outcome_text = f"Total Additional {outcome} Outcomes from {start_year} to {end_year}: {total_additional_outcomes:.2f}"
    
    return display_text, fig, additional_outcome_text

# Export Data Callback
@app.callback(
    Output("download-dataframe-csv", "data"),
    Input("export-button", "n_clicks"),
    prevent_initial_call=True,
)
def export_data(n_clicks):
    if n_clicks is None:
        return dash.no_update

    # Retrieve values from the update_dashboard function (need to be moved to global scope)
    outcome = 'Preterm Birth'  # or get dynamically if possible
    rcp_scenario = 'RCP 2.6'  # or get dynamically if possible
    population_growth_curve = 'Medium Growth'  # or get dynamically if possible
    pregnancy_growth_curve = 'Medium Growth'  # or get dynamically if possible
    time_period = [2024, 2030]  # or get dynamically if possible
    current_population = 7000000000  # or get dynamically if possible
    initial_pregnancy_rate = 0.02  # or get dynamically if possible
    
    odds_ratio = mnch_outcomes[outcome]['odds_ratio']
    current_rate = mnch_outcomes[outcome]['current_rate']
    temp_increase_2100 = rcp_scenarios[rcp_scenario]
    population_growth_rate = population_growth_rates[population_growth_curve]
    pregnancy_growth_rate = pregnancy_growth_rates[pregnancy_growth_curve]
    
    start_year, end_year = time_period
    num_years = end_year - start_year + 1
    selected_years = list(range(start_year, end_year + 1))
    
    # Calculate population growth
    population_growth_selected = calculate_population_growth(current_population, population_growth_rate, num_years)
    
    # Calculate pregnancies without the effect of heat
    pregnancies = calculate_pregnancies(population_growth_selected, initial_pregnancy_rate * (1 + pregnancy_growth_rate))
    
    # Calculate baseline outcomes without the effect of heat
    baseline_outcomes = [preg * current_rate for preg in pregnancies]
    
    # Calculate the temperature increase per year
    temperature_increase_per_year = calculate_temperature_increase_per_year(temp_increase_2100, start_year, end_year)
    
    # Calculate projected outcomes with the effect of heat
    projected_rates = [current_rate * (odds_ratio ** temp_increase) for temp_increase in temperature_increase_per_year]
    projected_outcomes = [rate * preg for rate, preg in zip(projected_rates, pregnancies)]
    
    # Calculate additional outcomes due to the effect of heat
    additional_outcomes = [proj - base for proj, base in zip(projected_outcomes, baseline_outcomes)]

    # Create a DataFrame with the current projections
    data = {
        'Year': selected_years,
        'Additional Outcomes': additional_outcomes,
        'Temperature Increase': temperature_increase_per_year
    }
    df = pd.DataFrame(data)

    return dcc.send_data_frame(df.to_csv, "projected_outcomes.csv")

# Step 5: Run the App
if __name__ == '__main__':
    app.run_server(debug=True)


In [16]:
#version 6
# Import necessary packages
import dash
from dash import dcc, html, Input, Output
import plotly.graph_objs as go
import pandas as pd
import numpy as np

# Step 2: Prepare the Data
mnch_outcomes = {
    'Preterm Birth': {'odds_ratio': 1.05, 'current_rate': 0.1},
    'Low Birth Weight': {'odds_ratio': 1.03, 'current_rate': 0.08},
    'Stillbirth': {'odds_ratio': 1.04, 'current_rate': 0.005}
}

rcp_scenarios = {
    'RCP 2.6': 1.5,  # degrees Celsius increase by 2100
    'RCP 4.5': 2.4,
    'RCP 6.0': 3.0,
    'RCP 8.5': 4.3
}

population_growth_rates = {
    'Low Growth': 0.01,  # 1% annual growth
    'Medium Growth': 0.02,  # 2% annual growth
    'High Growth': 0.03  # 3% annual growth
}

pregnancy_growth_rates = {
    'High Reduction': -0.15,  # 15% reduction
    'Medium Reduction': -0.1,  # 10% reduction
    'Low Reduction': -0.05,  # 5% reduction
    'Low Growth': 0.01,  # 1% annual growth
    'Medium Growth': 0.02,  # 2% annual growth
    'High Growth': 0.03  # 3% annual growth
}

years = list(range(2024, 2051))

def calculate_population_growth(initial_population, growth_rate, num_years):
    return [initial_population * ((1 + growth_rate) ** year) for year in range(num_years)]

def calculate_pregnancies(population, pregnancy_rate):
    return [pop * pregnancy_rate for pop in population]

def calculate_temperature_increase_per_year(temp_increase_2100, start_year, end_year):
    total_years = 2100 - 2024  # Calculate total years from 2024 to 2100
    annual_increase = temp_increase_2100 / total_years
    return [annual_increase * (year - 2024) for year in range(start_year, end_year + 1)]

# Step 3: Create the Dash Layout
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("MNCH Outcomes Due to Heat - Climate Change Projection"),
    html.Label("Select MNCH Outcome"),
    dcc.Dropdown(
        id='mnch-outcome-dropdown',
        options=[{'label': outcome, 'value': outcome} for outcome in mnch_outcomes.keys()],
        value='Preterm Birth'
    ),
    html.Label("Select RCP Scenario"),
    dcc.Dropdown(
        id='rcp-scenario-dropdown',
        options=[{'label': scenario, 'value': scenario} for scenario in rcp_scenarios.keys()],
        value='RCP 2.6'
    ),
    html.Label("Select Population Growth Curve"),
    dcc.Dropdown(
        id='population-growth-dropdown',
        options=[{'label': growth, 'value': growth} for growth in population_growth_rates.keys()],
        value='Medium Growth'
    ),
    html.Label("Select Pregnancy Growth Rate"),
    dcc.Dropdown(
        id='pregnancy-growth-dropdown',
        options=[{'label': growth, 'value': growth} for growth in pregnancy_growth_rates.keys()],
        value='Medium Growth'
    ),
    html.Label("Select Time Period"),
    dcc.RangeSlider(
        id='time-period-slider',
        min=2024,
        max=2050,
        step=1,
        marks={year: str(year) for year in years},
        value=[2024, 2030]
    ),
    html.Label("Current Population"),
    dcc.Input(
        id='current-population-input',
        type='number',
        value=7000000000
    ),
    html.Label("Initial Pregnancy Rate"),
    dcc.Input(
        id='initial-pregnancy-rate-input',
        type='number',
        value=0.02,  # 2% initial pregnancy rate
        step=0.001
    ),
    html.Label("Adaptation Effectiveness (%)"),
    dcc.Slider(
        id='adaptation-effectiveness-slider',
        min=0,
        max=100,
        step=1,
        value=50,
        marks={i: f'{i}%' for i in range(0, 101, 10)}
    ),
    html.Div(id='odds-and-rate-display'),
    dcc.Graph(id='outcome-projection-graph'),
    html.Div(id='additional-outcome-display', style={'margin-top': '20px', 'font-size': '20px'}),
    html.Button("Export Data", id="export-button"),
    dcc.Download(id="download-dataframe-csv")
])

# Step 4: Add Interactivity
@app.callback(
    Output('odds-and-rate-display', 'children'),
    Output('outcome-projection-graph', 'figure'),
    Output('additional-outcome-display', 'children'),
    Input('mnch-outcome-dropdown', 'value'),
    Input('rcp-scenario-dropdown', 'value'),
    Input('population-growth-dropdown', 'value'),
    Input('pregnancy-growth-dropdown', 'value'),
    Input('time-period-slider', 'value'),
    Input('current-population-input', 'value'),
    Input('initial-pregnancy-rate-input', 'value'),
    Input('adaptation-effectiveness-slider', 'value')
)
def update_dashboard(outcome, rcp_scenario, population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, adaptation_effectiveness):
    odds_ratio = mnch_outcomes[outcome]['odds_ratio']
    current_rate = mnch_outcomes[outcome]['current_rate']
    temp_increase_2100 = rcp_scenarios[rcp_scenario]
    population_growth_rate = population_growth_rates[population_growth_curve]
    pregnancy_growth_rate = pregnancy_growth_rates[pregnancy_growth_curve]
    
    start_year, end_year = time_period
    num_years = end_year - start_year + 1
    selected_years = list(range(start_year, end_year + 1))
    
    # Calculate population growth
    population_growth_selected = calculate_population_growth(current_population, population_growth_rate, num_years)
    
    # Calculate pregnancies without the effect of heat
    pregnancies = calculate_pregnancies(population_growth_selected, initial_pregnancy_rate * (1 + pregnancy_growth_rate))
    
    # Calculate baseline outcomes without the effect of heat
    baseline_outcomes = [preg * current_rate for preg in pregnancies]
    
    # Calculate the temperature increase per year
    temperature_increase_per_year = calculate_temperature_increase_per_year(temp_increase_2100, start_year, end_year)
    
    # Calculate projected outcomes with the effect of heat
    projected_rates = [current_rate * (odds_ratio ** temp_increase) for temp_increase in temperature_increase_per_year]
    projected_outcomes = [rate * preg for rate, preg in zip(projected_rates, pregnancies)]
    
    # Calculate additional outcomes due to the effect of heat
    additional_outcomes = [proj - base for proj, base in zip(projected_outcomes, baseline_outcomes)]
    
    # Apply adaptation effectiveness
    effective_additional_outcomes = [outcome * (1 - adaptation_effectiveness / 100) for outcome in additional_outcomes]
    
    total_additional_outcomes = sum(effective_additional_outcomes)
    
    display_text = f"Odds Ratio per 1°C increase: {odds_ratio}, Current Rate: {current_rate}, Initial Pregnancy Rate: {initial_pregnancy_rate}, Adaptation Effectiveness: {adaptation_effectiveness}%"
    
    fig = go.Figure()
    
    # Add additional outcomes
    fig.add_trace(go.Scatter(x=selected_years, y=effective_additional_outcomes, mode='lines', name='Effective Additional Outcomes'))
    
    # Add temperature increase
    fig.add_trace(go.Scatter(x=selected_years, y=temperature_increase_per_year, mode='lines', name='Temperature Increase', yaxis='y2'))
    
    # Update layout for multiple y-axes
    fig.update_layout(
        title=f"Effective Additional {outcome} Outcomes Due to Heat and Temperature Increase from {start_year} to {end_year}",
        xaxis_title="Year",
        yaxis=dict(
            title="Effective Additional Outcomes",
            titlefont=dict(color="#1f77b4"),
            tickfont=dict(color="#1f77b4")
        ),
        yaxis2=dict(
            title="Temperature Increase (°C)",
            titlefont=dict(color="#ff7f0e"),
            tickfont=dict(color="#ff7f0e"),
            overlaying='y',
            side='right'
        )
    )
    
    additional_outcome_text = f"Total Effective Additional {outcome} Outcomes from {start_year} to {end_year}: {total_additional_outcomes:.2f}"
    
    return display_text, fig, additional_outcome_text

# Export Data Callback
@app.callback(
    Output("download-dataframe-csv", "data"),
    Input("export-button", "n_clicks"),
    prevent_initial_call=True,
)
def export_data(n_clicks):
    if n_clicks is None:
        return dash.no_update

    # Retrieve values from the update_dashboard function (need to be moved to global scope)
    outcome = 'Preterm Birth'  # or get dynamically if possible
    rcp_scenario = 'RCP 2.6'  # or get dynamically if possible
    population_growth_curve = 'Medium Growth'  # or get dynamically if possible
    pregnancy_growth_curve = 'Medium Growth'  # or get dynamically if possible
    time_period = [2024, 2030]  # or get dynamically if possible
    current_population = 7000000000  # or get dynamically if possible
    initial_pregnancy_rate = 0.02  # or get dynamically if possible
    adaptation_effectiveness = 50  # or get dynamically if possible
    
    odds_ratio = mnch_outcomes[outcome]['odds_ratio']
    current_rate = mnch_outcomes[outcome]['current_rate']
    temp_increase_2100 = rcp_scenarios[rcp_scenario]
    population_growth_rate = population_growth_rates[population_growth_curve]
    pregnancy_growth_rate = pregnancy_growth_rates[pregnancy_growth_curve]
    
    start_year, end_year = time_period
    num_years = end_year - start_year + 1
    selected_years = list(range(start_year, end_year + 1))
    
    # Calculate population growth
    population_growth_selected = calculate_population_growth(current_population, population_growth_rate, num_years)
    
    # Calculate pregnancies without the effect of heat
    pregnancies = calculate_pregnancies(population_growth_selected, initial_pregnancy_rate * (1 + pregnancy_growth_rate))
    
    # Calculate baseline outcomes without the effect of heat
    baseline_outcomes = [preg * current_rate for preg in pregnancies]
    
    # Calculate the temperature increase per year
    temperature_increase_per_year = calculate_temperature_increase_per_year(temp_increase_2100, start_year, end_year)
    
    # Calculate projected outcomes with the effect of heat
    projected_rates = [current_rate * (odds_ratio ** temp_increase) for temp_increase in temperature_increase_per_year]
    projected_outcomes = [rate * preg for rate, preg in zip(projected_rates, pregnancies)]
    
    # Calculate additional outcomes due to the effect of heat
    additional_outcomes = [proj - base for proj, base in zip(projected_outcomes, baseline_outcomes)]
    
    # Apply adaptation effectiveness
    effective_additional_outcomes = [outcome * (1 - adaptation_effectiveness / 100) for outcome in additional_outcomes]

    # Create a DataFrame with the current projections
    data = {
        'Year': selected_years,
        'Effective Additional Outcomes': effective_additional_outcomes,
        'Temperature Increase': temperature_increase_per_year
    }
    df = pd.DataFrame(data)

    return dcc.send_data_frame(df.to_csv, "projected_outcomes.csv")

# Step 5: Run the App
if __name__ == '__main__':
    app.run_server(debug=True)


In [17]:
#version 7 including costs
# Import necessary packages
import dash
from dash import dcc, html, Input, Output
import plotly.graph_objs as go
import pandas as pd
import numpy as np

# Step 2: Prepare the Data
mnch_outcomes = {
    'Preterm Birth': {'odds_ratio': 1.05, 'current_rate': 0.1, 'cost_per_case': 1000},
    'Low Birth Weight': {'odds_ratio': 1.03, 'current_rate': 0.08, 'cost_per_case': 800},
    'Stillbirth': {'odds_ratio': 1.04, 'current_rate': 0.005, 'cost_per_case': 5000}
}

rcp_scenarios = {
    'RCP 2.6': 1.5,  # degrees Celsius increase by 2100
    'RCP 4.5': 2.4,
    'RCP 6.0': 3.0,
    'RCP 8.5': 4.3
}

population_growth_rates = {
    'Low Growth': 0.01,  # 1% annual growth
    'Medium Growth': 0.02,  # 2% annual growth
    'High Growth': 0.03  # 3% annual growth
}

pregnancy_growth_rates = {
    'High Reduction': -0.15,  # 15% reduction
    'Medium Reduction': -0.1,  # 10% reduction
    'Low Reduction': -0.05,  # 5% reduction
    'Low Growth': 0.01,  # 1% annual growth
    'Medium Growth': 0.02,  # 2% annual growth
    'High Growth': 0.03  # 3% annual growth
}

years = list(range(2024, 2051))

def calculate_population_growth(initial_population, growth_rate, num_years):
    return [initial_population * ((1 + growth_rate) ** year) for year in range(num_years)]

def calculate_pregnancies(population, pregnancy_rate):
    return [pop * pregnancy_rate for pop in population]

def calculate_temperature_increase_per_year(temp_increase_2100, start_year, end_year):
    total_years = 2100 - 2024  # Calculate total years from 2024 to 2100
    annual_increase = temp_increase_2100 / total_years
    return [annual_increase * (year - 2024) for year in range(start_year, end_year + 1)]

# Step 3: Create the Dash Layout
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("MNCH Outcomes Due to Heat - Climate Change Projection"),
    html.Label("Select MNCH Outcome"),
    dcc.Dropdown(
        id='mnch-outcome-dropdown',
        options=[{'label': outcome, 'value': outcome} for outcome in mnch_outcomes.keys()],
        value='Preterm Birth'
    ),
    html.Label("Select RCP Scenario"),
    dcc.Dropdown(
        id='rcp-scenario-dropdown',
        options=[{'label': scenario, 'value': scenario} for scenario in rcp_scenarios.keys()],
        value='RCP 2.6'
    ),
    html.Label("Select Population Growth Curve"),
    dcc.Dropdown(
        id='population-growth-dropdown',
        options=[{'label': growth, 'value': growth} for growth in population_growth_rates.keys()],
        value='Medium Growth'
    ),
    html.Label("Select Pregnancy Growth Rate"),
    dcc.Dropdown(
        id='pregnancy-growth-dropdown',
        options=[{'label': growth, 'value': growth} for growth in pregnancy_growth_rates.keys()],
        value='Medium Growth'
    ),
    html.Label("Select Time Period"),
    dcc.RangeSlider(
        id='time-period-slider',
        min=2024,
        max=2050,
        step=1,
        marks={year: str(year) for year in years},
        value=[2024, 2030]
    ),
    html.Label("Current Population"),
    dcc.Input(
        id='current-population-input',
        type='number',
        value=7000000000
    ),
    html.Label("Initial Pregnancy Rate"),
    dcc.Input(
        id='initial-pregnancy-rate-input',
        type='number',
        value=0.02,  # 2% initial pregnancy rate
        step=0.001
    ),
    html.Label("Adaptation Effectiveness (%)"),
    dcc.Slider(
        id='adaptation-effectiveness-slider',
        min=0,
        max=100,
        step=1,
        value=50,
        marks={i: f'{i}%' for i in range(0, 101, 10)}
    ),
    html.Div(id='odds-and-rate-display'),
    dcc.Graph(id='outcome-projection-graph'),
    dcc.Graph(id='cost-projection-graph'),
    html.Div(id='additional-outcome-display', style={'margin-top': '20px', 'font-size': '20px'}),
    html.Button("Export Data", id="export-button"),
    dcc.Download(id="download-dataframe-csv")
])

# Step 4: Add Interactivity
@app.callback(
    Output('odds-and-rate-display', 'children'),
    Output('outcome-projection-graph', 'figure'),
    Output('cost-projection-graph', 'figure'),
    Output('additional-outcome-display', 'children'),
    Input('mnch-outcome-dropdown', 'value'),
    Input('rcp-scenario-dropdown', 'value'),
    Input('population-growth-dropdown', 'value'),
    Input('pregnancy-growth-dropdown', 'value'),
    Input('time-period-slider', 'value'),
    Input('current-population-input', 'value'),
    Input('initial-pregnancy-rate-input', 'value'),
    Input('adaptation-effectiveness-slider', 'value')
)
def update_dashboard(outcome, rcp_scenario, population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, adaptation_effectiveness):
    odds_ratio = mnch_outcomes[outcome]['odds_ratio']
    current_rate = mnch_outcomes[outcome]['current_rate']
    cost_per_case = mnch_outcomes[outcome]['cost_per_case']
    temp_increase_2100 = rcp_scenarios[rcp_scenario]
    population_growth_rate = population_growth_rates[population_growth_curve]
    pregnancy_growth_rate = pregnancy_growth_rates[pregnancy_growth_curve]
    
    start_year, end_year = time_period
    num_years = end_year - start_year + 1
    selected_years = list(range(start_year, end_year + 1))
    
    # Calculate population growth
    population_growth_selected = calculate_population_growth(current_population, population_growth_rate, num_years)
    
    # Calculate pregnancies without the effect of heat
    pregnancies = calculate_pregnancies(population_growth_selected, initial_pregnancy_rate * (1 + pregnancy_growth_rate))
    
    # Calculate baseline outcomes without the effect of heat
    baseline_outcomes = [preg * current_rate for preg in pregnancies]
    
    # Calculate the temperature increase per year
    temperature_increase_per_year = calculate_temperature_increase_per_year(temp_increase_2100, start_year, end_year)
    
    # Calculate projected outcomes with the effect of heat
    projected_rates = [current_rate * (odds_ratio ** temp_increase) for temp_increase in temperature_increase_per_year]
    projected_outcomes = [rate * preg for rate, preg in zip(projected_rates, pregnancies)]
    
    # Calculate additional outcomes due to the effect of heat
    additional_outcomes = [proj - base for proj, base in zip(projected_outcomes, baseline_outcomes)]
    
    # Apply adaptation effectiveness
    effective_additional_outcomes = [outcome * (1 - adaptation_effectiveness / 100) for outcome in additional_outcomes]
    
    # Calculate the total cost
    costs = [outcome * cost_per_case for outcome in effective_additional_outcomes]
    total_additional_outcomes = sum(effective_additional_outcomes)
    total_cost = sum(costs)
    
    display_text = f"Odds Ratio per 1°C increase: {odds_ratio}, Current Rate: {current_rate}, Initial Pregnancy Rate: {initial_pregnancy_rate}, Adaptation Effectiveness: {adaptation_effectiveness}%, Cost per Case: ${cost_per_case}"
    
    fig_outcomes = go.Figure()
    
    # Add additional outcomes
    fig_outcomes.add_trace(go.Scatter(x=selected_years, y=effective_additional_outcomes, mode='lines', name='Effective Additional Outcomes'))
    
    # Add temperature increase
    fig_outcomes.add_trace(go.Scatter(x=selected_years, y=temperature_increase_per_year, mode='lines', name='Temperature Increase', yaxis='y2'))
    
    # Update layout for multiple y-axes
    fig_outcomes.update_layout(
        title=f"Effective Additional {outcome} Outcomes Due to Heat and Temperature Increase from {start_year} to {end_year}",
        xaxis_title="Year",
        yaxis=dict(
            title="Effective Additional Outcomes",
            titlefont=dict(color="#1f77b4"),
            tickfont=dict(color="#1f77b4")
        ),
        yaxis2=dict(
            title="Temperature Increase (°C)",
            titlefont=dict(color="#ff7f0e"),
            tickfont=dict(color="#ff7f0e"),
            overlaying='y',
            side='right'
        )
    )
    
    fig_costs = go.Figure()
    
    # Add costs
    fig_costs.add_trace(go.Scatter(x=selected_years, y=costs, mode='lines', name='Costs'))
    
    fig_costs.update_layout(
        title=f"Costs Associated with Effective Additional {outcome} Outcomes Due to Heat from {start_year} to {end_year}",
        xaxis_title="Year",
        yaxis=dict(
            title="Costs (USD)",
            titlefont=dict(color="#1f77b4"),
            tickfont=dict(color="#1f77b4")
        )
    )
    
    additional_outcome_text = f"Total Effective Additional {outcome} Outcomes from {start_year} to {end_year}: {total_additional_outcomes:.2f}\nTotal Cost: ${total_cost:.2f}"
    
    return display_text, fig_outcomes, fig_costs, additional_outcome_text

# Export Data Callback
@app.callback(
    Output("download-dataframe-csv", "data"),
    Input("export-button", "n_clicks"),
    prevent_initial_call=True,
)
def export_data(n_clicks):
    if n_clicks is None:
        return dash.no_update

    # Retrieve values from the update_dashboard function (need to be moved to global scope)
    outcome = 'Preterm Birth'  # or get dynamically if possible
    rcp_scenario = 'RCP 2.6'  # or get dynamically if possible
    population_growth_curve = 'Medium Growth'  # or get dynamically if possible
    pregnancy_growth_curve = 'Medium Growth'  # or get dynamically if possible
    time_period = [2024, 2030]  # or get dynamically if possible
    current_population = 7000000000  # or get dynamically if possible
    initial_pregnancy_rate = 0.02  # or get dynamically if possible
    adaptation_effectiveness = 50  # or get dynamically if possible
    
    odds_ratio = mnch_outcomes[outcome]['odds_ratio']
    current_rate = mnch_outcomes[outcome]['current_rate']
    cost_per_case = mnch_outcomes[outcome]['cost_per_case']
    temp_increase_2100 = rcp_scenarios[rcp_scenario]
    population_growth_rate = population_growth_rates[population_growth_curve]
    pregnancy_growth_rate = pregnancy_growth_rates[pregnancy_growth_curve]
    
    start_year, end_year = time_period
    num_years = end_year - start_year + 1
    selected_years = list(range(start_year, end_year + 1))
    
    # Calculate population growth
    population_growth_selected = calculate_population_growth(current_population, population_growth_rate, num_years)
    
    # Calculate pregnancies without the effect of heat
    pregnancies = calculate_pregnancies(population_growth_selected, initial_pregnancy_rate * (1 + pregnancy_growth_rate))
    
    # Calculate baseline outcomes without the effect of heat
    baseline_outcomes = [preg * current_rate for preg in pregnancies]
    
    # Calculate the temperature increase per year
    temperature_increase_per_year = calculate_temperature_increase_per_year(temp_increase_2100, start_year, end_year)
    
    # Calculate projected outcomes with the effect of heat
    projected_rates = [current_rate * (odds_ratio ** temp_increase) for temp_increase in temperature_increase_per_year]
    projected_outcomes = [rate * preg for rate, preg in zip(projected_rates, pregnancies)]
    
    # Calculate additional outcomes due to the effect of heat
    additional_outcomes = [proj - base for proj, base in zip(projected_outcomes, baseline_outcomes)]
    
    # Apply adaptation effectiveness
    effective_additional_outcomes = [outcome * (1 - adaptation_effectiveness / 100) for outcome in additional_outcomes]

    # Calculate the total cost
    costs = [outcome * cost_per_case for outcome in effective_additional_outcomes]

    # Create a DataFrame with the current projections
    data = {
        'Year': selected_years,
        'Effective Additional Outcomes': effective_additional_outcomes,
        'Temperature Increase': temperature_increase_per_year,
        'Costs': costs
    }
    df = pd.DataFrame(data)

    return dcc.send_data_frame(df.to_csv, "projected_outcomes.csv")

# Step 5: Run the App
if __name__ == '__main__':
    app.run_server(debug=True)


In [18]:
#version 7
# Import necessary packages
import dash
from dash import dcc, html, Input, Output
import plotly.graph_objs as go
import pandas as pd
import numpy as np

# Step 2: Prepare the Data
mnch_outcomes = {
    'Preterm Birth': {'odds_ratio': 1.05, 'current_rate': 0.1, 'cost_per_case': 1000},
    'Low Birth Weight': {'odds_ratio': 1.03, 'current_rate': 0.08, 'cost_per_case': 800},
    'Stillbirth': {'odds_ratio': 1.04, 'current_rate': 0.005, 'cost_per_case': 5000}
}

rcp_scenarios = {
    'RCP 2.6': 1.5,  # degrees Celsius increase by 2100
    'RCP 4.5': 2.4,
    'RCP 6.0': 3.0,
    'RCP 8.5': 4.3
}

population_growth_rates = {
    'Low Growth': 0.01,  # 1% annual growth
    'Medium Growth': 0.02,  # 2% annual growth
    'High Growth': 0.03  # 3% annual growth
}

pregnancy_growth_rates = {
    'High Reduction': -0.15,  # 15% reduction
    'Medium Reduction': -0.1,  # 10% reduction
    'Low Reduction': -0.05,  # 5% reduction
    'Low Growth': 0.01,  # 1% annual growth
    'Medium Growth': 0.02,  # 2% annual growth
    'High Growth': 0.03  # 3% annual growth
}

years = list(range(2024, 2051))

def calculate_population_growth(initial_population, growth_rate, num_years):
    return [initial_population * ((1 + growth_rate) ** year) for year in range(num_years)]

def calculate_pregnancies(population, pregnancy_rate):
    return [pop * pregnancy_rate for pop in population]

def calculate_temperature_increase_per_year(temp_increase_2100, start_year, end_year):
    total_years = 2100 - 2024  # Calculate total years from 2024 to 2100
    annual_increase = temp_increase_2100 / total_years
    return [annual_increase * (year - 2024) for year in range(start_year, end_year + 1)]

def calculate_additional_outcomes_and_costs(outcome, rcp_scenario, population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, adaptation_effectiveness):
    odds_ratio = mnch_outcomes[outcome]['odds_ratio']
    current_rate = mnch_outcomes[outcome]['current_rate']
    cost_per_case = mnch_outcomes[outcome]['cost_per_case']
    temp_increase_2100 = rcp_scenarios[rcp_scenario]
    population_growth_rate = population_growth_rates[population_growth_curve]
    pregnancy_growth_rate = pregnancy_growth_rates[pregnancy_growth_curve]
    
    start_year, end_year = time_period
    num_years = end_year - start_year + 1
    selected_years = list(range(start_year, end_year + 1))
    
    # Calculate population growth
    population_growth_selected = calculate_population_growth(current_population, population_growth_rate, num_years)
    
    # Calculate pregnancies without the effect of heat
    pregnancies = calculate_pregnancies(population_growth_selected, initial_pregnancy_rate * (1 + pregnancy_growth_rate))
    
    # Calculate baseline outcomes without the effect of heat
    baseline_outcomes = [preg * current_rate for preg in pregnancies]
    
    # Calculate the temperature increase per year
    temperature_increase_per_year = calculate_temperature_increase_per_year(temp_increase_2100, start_year, end_year)
    
    # Calculate projected outcomes with the effect of heat
    projected_rates = [current_rate * (odds_ratio ** temp_increase) for temp_increase in temperature_increase_per_year]
    projected_outcomes = [rate * preg for rate, preg in zip(projected_rates, pregnancies)]
    
    # Calculate additional outcomes due to the effect of heat
    additional_outcomes = [proj - base for proj, base in zip(projected_outcomes, baseline_outcomes)]
    
    # Apply adaptation effectiveness
    effective_additional_outcomes = [outcome * (1 - adaptation_effectiveness / 100) for outcome in additional_outcomes]
    
    # Calculate the total cost
    costs = [outcome * cost_per_case for outcome in effective_additional_outcomes]
    
    return selected_years, effective_additional_outcomes, costs

# Step 3: Create the Dash Layout
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("MNCH Outcomes Due to Heat - Climate Change Projection"),
    html.Label("Select MNCH Outcome"),
    dcc.Dropdown(
        id='mnch-outcome-dropdown',
        options=[{'label': outcome, 'value': outcome} for outcome in mnch_outcomes.keys()],
        value='Preterm Birth'
    ),
    html.Label("Select RCP Scenario"),
    dcc.Dropdown(
        id='rcp-scenario-dropdown',
        options=[{'label': scenario, 'value': scenario} for scenario in rcp_scenarios.keys()],
        value='RCP 2.6'
    ),
    html.Label("Select Population Growth Curve"),
    dcc.Dropdown(
        id='population-growth-dropdown',
        options=[{'label': growth, 'value': growth} for growth in population_growth_rates.keys()],
        value='Medium Growth'
    ),
    html.Label("Select Pregnancy Growth Rate"),
    dcc.Dropdown(
        id='pregnancy-growth-dropdown',
        options=[{'label': growth, 'value': growth} for growth in pregnancy_growth_rates.keys()],
        value='Medium Growth'
    ),
    html.Label("Select Time Period"),
    dcc.RangeSlider(
        id='time-period-slider',
        min=2024,
        max=2050,
        step=1,
        marks={year: str(year) for year in years},
        value=[2024, 2030]
    ),
    html.Label("Current Population"),
    dcc.Input(
        id='current-population-input',
        type='number',
        value=7000000000
    ),
    html.Label("Initial Pregnancy Rate"),
    dcc.Input(
        id='initial-pregnancy-rate-input',
        type='number',
        value=0.02,  # 2% initial pregnancy rate
        step=0.001
    ),
    html.Label("Adaptation Effectiveness (%)"),
    dcc.Slider(
        id='adaptation-effectiveness-slider',
        min=0,
        max=100,
        step=1,
        value=50,
        marks={i: f'{i}%' for i in range(0, 101, 10)}
    ),
    html.Div(id='odds-and-rate-display'),
    dcc.Graph(id='outcome-projection-graph'),
    dcc.Graph(id='cost-projection-graph'),
    dcc.Graph(id='cost-savings-graph'),
    html.Div(id='additional-outcome-display', style={'margin-top': '20px', 'font-size': '20px'}),
    html.Button("Export Data", id="export-button"),
    dcc.Download(id="download-dataframe-csv")
])

# Step 4: Add Interactivity
@app.callback(
    Output('odds-and-rate-display', 'children'),
    Output('outcome-projection-graph', 'figure'),
    Output('cost-projection-graph', 'figure'),
    Output('cost-savings-graph', 'figure'),
    Output('additional-outcome-display', 'children'),
    Input('mnch-outcome-dropdown', 'value'),
    Input('rcp-scenario-dropdown', 'value'),
    Input('population-growth-dropdown', 'value'),
    Input('pregnancy-growth-dropdown', 'value'),
    Input('time-period-slider', 'value'),
    Input('current-population-input', 'value'),
    Input('initial-pregnancy-rate-input', 'value'),
    Input('adaptation-effectiveness-slider', 'value')
)
def update_dashboard(outcome, rcp_scenario, population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, adaptation_effectiveness):
    # Calculate selected scenario outcomes and costs
    selected_years, effective_additional_outcomes, costs = calculate_additional_outcomes_and_costs(
        outcome, rcp_scenario, population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, adaptation_effectiveness
    )
    
    # Calculate worst scenario (RCP 8.5, 0% adaptation effectiveness) outcomes and costs
    _, worst_effective_additional_outcomes, worst_costs = calculate_additional_outcomes_and_costs(
        outcome, 'RCP 8.5', population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, 0
    )
    
    # Calculate cost savings
    cost_savings = [worst - selected for worst, selected in zip(worst_costs, costs)]
    
    total_additional_outcomes = sum(effective_additional_outcomes)
    total_cost = sum(costs)
    total_cost_savings = sum(cost_savings)
    
    display_text = f"Odds Ratio per 1°C increase: {mnch_outcomes[outcome]['odds_ratio']}, Current Rate: {mnch_outcomes[outcome]['current_rate']}, Initial Pregnancy Rate: {initial_pregnancy_rate}, Adaptation Effectiveness: {adaptation_effectiveness}%, Cost per Case: ${mnch_outcomes[outcome]['cost_per_case']}"
    
    fig_outcomes = go.Figure()
    
    # Add additional outcomes
    fig_outcomes.add_trace(go.Scatter(x=selected_years, y=effective_additional_outcomes, mode='lines', name='Effective Additional Outcomes'))
    
    # Add temperature increase
    temperature_increase_per_year = calculate_temperature_increase_per_year(rcp_scenarios[rcp_scenario], time_period[0], time_period[1])
    fig_outcomes.add_trace(go.Scatter(x=selected_years, y=temperature_increase_per_year, mode='lines', name='Temperature Increase', yaxis='y2'))
    
    # Update layout for multiple y-axes
    fig_outcomes.update_layout(
        title=f"Effective Additional {outcome} Outcomes Due to Heat and Temperature Increase from {time_period[0]} to {time_period[1]}",
        xaxis_title="Year",
        yaxis=dict(
            title="Effective Additional Outcomes",
            titlefont=dict(color="#1f77b4"),
            tickfont=dict(color="#1f77b4")
        ),
        yaxis2=dict(
            title="Temperature Increase (°C)",
            titlefont=dict(color="#ff7f0e"),
            tickfont=dict(color="#ff7f0e"),
            overlaying='y',
            side='right'
        )
    )
    
    fig_costs = go.Figure()
    
    # Add costs
    fig_costs.add_trace(go.Scatter(x=selected_years, y=costs, mode='lines', name='Costs'))
    
    fig_costs.update_layout(
        title=f"Costs Associated with Effective Additional {outcome} Outcomes Due to Heat from {time_period[0]} to {time_period[1]}",
        xaxis_title="Year",
        yaxis=dict(
            title="Costs (USD)",
            titlefont=dict(color="#1f77b4"),
            tickfont=dict(color="#1f77b4")
        )
    )
    
    fig_cost_savings = go.Figure()
    
    # Add worst-case scenario costs
    fig_cost_savings.add_trace(go.Scatter(x=selected_years, y=worst_costs, mode='lines', name='Worst Case Costs (RCP 8.5, 0% Adaptation)'))
    
    # Add selected scenario costs
    fig_cost_savings.add_trace(go.Scatter(x=selected_years, y=costs, mode='lines', name='Selected Scenario Costs'))
    
    # Add cost savings
    fig_cost_savings.add_trace(go.Scatter(x=selected_years, y=cost_savings, mode='lines', name='Cost Savings', yaxis='y2'))
    
    # Update layout for multiple y-axes
    fig_cost_savings.update_layout(
        title=f"Cost Savings by Comparing Worst Scenario (RCP 8.5, 0% Adaptation) and Selected Scenario from {time_period[0]} to {time_period[1]}",
        xaxis_title="Year",
        yaxis=dict(
            title="Costs (USD)",
            titlefont=dict(color="#1f77b4"),
            tickfont=dict(color="#1f77b4")
        ),
        yaxis2=dict(
            title="Cost Savings (USD)",
            titlefont=dict(color="#2ca02c"),
            tickfont=dict(color="#2ca02c"),
            overlaying='y',
            side='right'
        )
    )
    
    additional_outcome_text = f"Total Effective Additional {outcome} Outcomes from {time_period[0]} to {time_period[1]}: {total_additional_outcomes:.2f}\nTotal Cost: ${total_cost:.2f}\nTotal Cost Savings: ${total_cost_savings:.2f}"
    
    return display_text, fig_outcomes, fig_costs, fig_cost_savings, additional_outcome_text

# Export Data Callback
@app.callback(
    Output("download-dataframe-csv", "data"),
    Input("export-button", "n_clicks"),
    prevent_initial_call=True,
)
def export_data(n_clicks):
    if n_clicks is None:
        return dash.no_update

    # Retrieve values from the update_dashboard function (need to be moved to global scope)
    outcome = 'Preterm Birth'  # or get dynamically if possible
    rcp_scenario = 'RCP 2.6'  # or get dynamically if possible
    population_growth_curve = 'Medium Growth'  # or get dynamically if possible
    pregnancy_growth_curve = 'Medium Growth'  # or get dynamically if possible
    time_period = [2024, 2030]  # or get dynamically if possible
    current_population = 7000000000  # or get dynamically if possible
    initial_pregnancy_rate = 0.02  # or get dynamically if possible
    adaptation_effectiveness = 50  # or get dynamically if possible
    
    selected_years, effective_additional_outcomes, costs = calculate_additional_outcomes_and_costs(
        outcome, rcp_scenario, population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, adaptation_effectiveness
    )
    
    _, worst_effective_additional_outcomes, worst_costs = calculate_additional_outcomes_and_costs(
        outcome, 'RCP 8.5', population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, 0
    )
    
    cost_savings = [worst - selected for worst, selected in zip(worst_costs, costs)]

    # Create a DataFrame with the current projections
    data = {
        'Year': selected_years,
        'Effective Additional Outcomes': effective_additional_outcomes,
        'Temperature Increase': calculate_temperature_increase_per_year(rcp_scenarios[rcp_scenario], time_period[0], time_period[1]),
        'Costs': costs,
        'Worst Case Costs': worst_costs,
        'Cost Savings': cost_savings
    }
    df = pd.DataFrame(data)

    return dcc.send_data_frame(df.to_csv, "projected_outcomes.csv")

# Step 5: Run the App
if __name__ == '__main__':
    app.run_server(debug=True)


In [19]:
#version 8 including cost savings
# Import necessary packages
import dash
from dash import dcc, html, Input, Output
import plotly.graph_objs as go
import pandas as pd
import numpy as np

# Step 2: Prepare the Data
mnch_outcomes = {
    'Preterm Birth': {'odds_ratio': 1.05, 'current_rate': 0.1, 'cost_per_case': 1000},
    'Low Birth Weight': {'odds_ratio': 1.03, 'current_rate': 0.08, 'cost_per_case': 800},
    'Stillbirth': {'odds_ratio': 1.04, 'current_rate': 0.005, 'cost_per_case': 5000}
}

rcp_scenarios = {
    'RCP 2.6': 1.5,  # degrees Celsius increase by 2100
    'RCP 4.5': 2.4,
    'RCP 6.0': 3.0,
    'RCP 8.5': 4.3
}

population_growth_rates = {
    'Low Growth': 0.01,  # 1% annual growth
    'Medium Growth': 0.02,  # 2% annual growth
    'High Growth': 0.03  # 3% annual growth
}

pregnancy_growth_rates = {
    'High Reduction': -0.15,  # 15% reduction
    'Medium Reduction': -0.1,  # 10% reduction
    'Low Reduction': -0.05,  # 5% reduction
    'Low Growth': 0.01,  # 1% annual growth
    'Medium Growth': 0.02,  # 2% annual growth
    'High Growth': 0.03  # 3% annual growth
}

years = list(range(2024, 2051))

def calculate_population_growth(initial_population, growth_rate, num_years):
    return [initial_population * ((1 + growth_rate) ** year) for year in range(num_years)]

def calculate_pregnancies(population, pregnancy_rate):
    return [pop * pregnancy_rate for pop in population]

def calculate_temperature_increase_per_year(temp_increase_2100, start_year, end_year):
    total_years = 2100 - 2024  # Calculate total years from 2024 to 2100
    annual_increase = temp_increase_2100 / total_years
    return [annual_increase * (year - 2024) for year in range(start_year, end_year + 1)]

def calculate_additional_outcomes_and_costs(outcome, rcp_scenario, population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, adaptation_effectiveness):
    odds_ratio = mnch_outcomes[outcome]['odds_ratio']
    current_rate = mnch_outcomes[outcome]['current_rate']
    cost_per_case = mnch_outcomes[outcome]['cost_per_case']
    temp_increase_2100 = rcp_scenarios[rcp_scenario]
    population_growth_rate = population_growth_rates[population_growth_curve]
    pregnancy_growth_rate = pregnancy_growth_rates[pregnancy_growth_curve]
    
    start_year, end_year = time_period
    num_years = end_year - start_year + 1
    selected_years = list(range(start_year, end_year + 1))
    
    # Calculate population growth
    population_growth_selected = calculate_population_growth(current_population, population_growth_rate, num_years)
    
    # Calculate pregnancies without the effect of heat
    pregnancies = calculate_pregnancies(population_growth_selected, initial_pregnancy_rate * (1 + pregnancy_growth_rate))
    
    # Calculate baseline outcomes without the effect of heat
    baseline_outcomes = [preg * current_rate for preg in pregnancies]
    
    # Calculate the temperature increase per year
    temperature_increase_per_year = calculate_temperature_increase_per_year(temp_increase_2100, start_year, end_year)
    
    # Calculate projected outcomes with the effect of heat
    projected_rates = [current_rate * (odds_ratio ** temp_increase) for temp_increase in temperature_increase_per_year]
    projected_outcomes = [rate * preg for rate, preg in zip(projected_rates, pregnancies)]
    
    # Calculate additional outcomes due to the effect of heat
    additional_outcomes = [proj - base for proj, base in zip(projected_outcomes, baseline_outcomes)]
    
    # Apply adaptation effectiveness
    effective_additional_outcomes = [outcome * (1 - adaptation_effectiveness / 100) for outcome in additional_outcomes]
    
    # Calculate the total cost
    costs = [outcome * cost_per_case for outcome in effective_additional_outcomes]
    
    return selected_years, effective_additional_outcomes, costs

# Step 3: Create the Dash Layout
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("MNCH Outcomes Due to Heat - Climate Change Projection"),
    html.Label("Select MNCH Outcome"),
    dcc.Dropdown(
        id='mnch-outcome-dropdown',
        options=[{'label': outcome, 'value': outcome} for outcome in mnch_outcomes.keys()],
        value='Preterm Birth'
    ),
    html.Label("Select RCP Scenario"),
    dcc.Dropdown(
        id='rcp-scenario-dropdown',
        options=[{'label': scenario, 'value': scenario} for scenario in rcp_scenarios.keys()],
        value='RCP 2.6'
    ),
    html.Label("Select Population Growth Curve"),
    dcc.Dropdown(
        id='population-growth-dropdown',
        options=[{'label': growth, 'value': growth} for growth in population_growth_rates.keys()],
        value='Medium Growth'
    ),
    html.Label("Select Pregnancy Growth Rate"),
    dcc.Dropdown(
        id='pregnancy-growth-dropdown',
        options=[{'label': growth, 'value': growth} for growth in pregnancy_growth_rates.keys()],
        value='Medium Growth'
    ),
    html.Label("Select Time Period"),
    dcc.RangeSlider(
        id='time-period-slider',
        min=2024,
        max=2050,
        step=1,
        marks={year: str(year) for year in years},
        value=[2024, 2030]
    ),
    html.Label("Current Population"),
    dcc.Input(
        id='current-population-input',
        type='number',
        value=7000000000
    ),
    html.Label("Initial Pregnancy Rate"),
    dcc.Input(
        id='initial-pregnancy-rate-input',
        type='number',
        value=0.02,  # 2% initial pregnancy rate
        step=0.001
    ),
    html.Label("Adaptation Effectiveness (%)"),
    dcc.Slider(
        id='adaptation-effectiveness-slider',
        min=0,
        max=100,
        step=1,
        value=50,
        marks={i: f'{i}%' for i in range(0, 101, 10)}
    ),
    html.Div(id='odds-and-rate-display'),
    dcc.Graph(id='outcome-projection-graph'),
    dcc.Graph(id='cost-projection-graph'),
    dcc.Graph(id='cost-savings-graph'),
    html.Div(id='additional-outcome-display', style={'margin-top': '20px', 'font-size': '20px'}),
    html.Button("Export Data", id="export-button"),
    dcc.Download(id="download-dataframe-csv")
])

# Step 4: Add Interactivity
@app.callback(
    Output('odds-and-rate-display', 'children'),
    Output('outcome-projection-graph', 'figure'),
    Output('cost-projection-graph', 'figure'),
    Output('cost-savings-graph', 'figure'),
    Output('additional-outcome-display', 'children'),
    Input('mnch-outcome-dropdown', 'value'),
    Input('rcp-scenario-dropdown', 'value'),
    Input('population-growth-dropdown', 'value'),
    Input('pregnancy-growth-dropdown', 'value'),
    Input('time-period-slider', 'value'),
    Input('current-population-input', 'value'),
    Input('initial-pregnancy-rate-input', 'value'),
    Input('adaptation-effectiveness-slider', 'value')
)
def update_dashboard(outcome, rcp_scenario, population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, adaptation_effectiveness):
    # Calculate selected scenario outcomes and costs
    selected_years, effective_additional_outcomes, costs = calculate_additional_outcomes_and_costs(
        outcome, rcp_scenario, population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, adaptation_effectiveness
    )
    
    # Calculate worst scenario (RCP 8.5, 0% adaptation effectiveness) outcomes and costs
    _, worst_effective_additional_outcomes, worst_costs = calculate_additional_outcomes_and_costs(
        outcome, 'RCP 8.5', population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, 0
    )
    
    # Calculate cost savings
    cost_savings = [worst - selected for worst, selected in zip(worst_costs, costs)]
    
    total_additional_outcomes = sum(effective_additional_outcomes)
    total_cost = sum(costs)
    total_cost_savings = sum(cost_savings)
    
    display_text = f"Odds Ratio per 1°C increase: {mnch_outcomes[outcome]['odds_ratio']}, Current Rate: {mnch_outcomes[outcome]['current_rate']}, Initial Pregnancy Rate: {initial_pregnancy_rate}, Adaptation Effectiveness: {adaptation_effectiveness}%, Cost per Case: ${mnch_outcomes[outcome]['cost_per_case']}"
    
    fig_outcomes = go.Figure()
    
    # Add additional outcomes
    fig_outcomes.add_trace(go.Scatter(x=selected_years, y=effective_additional_outcomes, mode='lines', name='Effective Additional Outcomes'))
    
    # Add temperature increase
    temperature_increase_per_year = calculate_temperature_increase_per_year(rcp_scenarios[rcp_scenario], time_period[0], time_period[1])
    fig_outcomes.add_trace(go.Scatter(x=selected_years, y=temperature_increase_per_year, mode='lines', name='Temperature Increase', yaxis='y2'))
    
    # Update layout for multiple y-axes
    fig_outcomes.update_layout(
        title=f"Effective Additional {outcome} Outcomes Due to Heat and Temperature Increase from {time_period[0]} to {time_period[1]}",
        xaxis_title="Year",
        yaxis=dict(
            title="Effective Additional Outcomes",
            titlefont=dict(color="#1f77b4"),
            tickfont=dict(color="#1f77b4")
        ),
        yaxis2=dict(
            title="Temperature Increase (°C)",
            titlefont=dict(color="#ff7f0e"),
            tickfont=dict(color="#ff7f0e"),
            overlaying='y',
            side='right'
        )
    )
    
    fig_costs = go.Figure()
    
    # Add costs
    fig_costs.add_trace(go.Scatter(x=selected_years, y=costs, mode='lines', name='Costs'))
    
    fig_costs.update_layout(
        title=f"Costs Associated with Effective Additional {outcome} Outcomes Due to Heat from {time_period[0]} to {time_period[1]}",
        xaxis_title="Year",
        yaxis=dict(
            title="Costs (USD)",
            titlefont=dict(color="#1f77b4"),
            tickfont=dict(color="#1f77b4")
        )
    )
    
    fig_cost_savings = go.Figure()
    
    # Add worst-case scenario costs
    fig_cost_savings.add_trace(go.Scatter(x=selected_years, y=worst_costs, mode='lines', name='Worst Case Costs (RCP 8.5, 0% Adaptation)'))
    
    # Add selected scenario costs
    fig_cost_savings.add_trace(go.Scatter(x=selected_years, y=costs, mode='lines', name=f'Selected Scenario Costs ({rcp_scenario}, {adaptation_effectiveness}% Adaptation)'))
    
    # Add cost savings
    fig_cost_savings.add_trace(go.Scatter(x=selected_years, y=cost_savings, mode='lines', name='Cost Savings', yaxis='y2'))
    
    # Update layout for multiple y-axes
    fig_cost_savings.update_layout(
        title=f"Cost Savings by Comparing Worst Scenario (RCP 8.5, 0% Adaptation) and Selected Scenario from {time_period[0]} to {time_period[1]}",
        xaxis_title="Year",
        yaxis=dict(
            title="Costs (USD)",
            titlefont=dict(color="#1f77b4"),
            tickfont=dict(color="#1f77b4")
        ),
        yaxis2=dict(
            title="Cost Savings (USD)",
            titlefont=dict(color="#2ca02c"),
            tickfont=dict(color="#2ca02c"),
            overlaying='y',
            side='right'
        )
    )
    
    additional_outcome_text = f"Total Effective Additional {outcome} Outcomes from {time_period[0]} to {time_period[1]}: {total_additional_outcomes:.2f}\nTotal Cost: ${total_cost:.2f}\nTotal Cost Savings: ${total_cost_savings:.2f}"
    
    return display_text, fig_outcomes, fig_costs, fig_cost_savings, additional_outcome_text

# Export Data Callback
@app.callback(
    Output("download-dataframe-csv", "data"),
    Input("export-button", "n_clicks"),
    prevent_initial_call=True,
)
def export_data(n_clicks):
    if n_clicks is None:
        return dash.no_update

    # Retrieve values from the update_dashboard function (need to be moved to global scope)
    outcome = 'Preterm Birth'  # or get dynamically if possible
    rcp_scenario = 'RCP 2.6'  # or get dynamically if possible
    population_growth_curve = 'Medium Growth'  # or get dynamically if possible
    pregnancy_growth_curve = 'Medium Growth'  # or get dynamically if possible
    time_period = [2024, 2030]  # or get dynamically if possible
    current_population = 7000000000  # or get dynamically if possible
    initial_pregnancy_rate = 0.02  # or get dynamically if possible
    adaptation_effectiveness = 50  # or get dynamically if possible
    
    selected_years, effective_additional_outcomes, costs = calculate_additional_outcomes_and_costs(
        outcome, rcp_scenario, population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, adaptation_effectiveness
    )
    
    _, worst_effective_additional_outcomes, worst_costs = calculate_additional_outcomes_and_costs(
        outcome, 'RCP 8.5', population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, 0
    )
    
    cost_savings = [worst - selected for worst, selected in zip(worst_costs, costs)]

    # Create a DataFrame with the current projections
    data = {
        'Year': selected_years,
        'Effective Additional Outcomes': effective_additional_outcomes,
        'Temperature Increase': calculate_temperature_increase_per_year(rcp_scenarios[rcp_scenario], time_period[0], time_period[1]),
        'Costs': costs,
        'Worst Case Costs': worst_costs,
        'Cost Savings': cost_savings
    }
    df = pd.DataFrame(data)

    return dcc.send_data_frame(df.to_csv, "projected_outcomes.csv")

# Step 5: Run the App
if __name__ == '__main__':
    app.run_server(debug=True)


In [20]:
# Import necessary packages
import dash
from dash import dcc, html, Input, Output
import plotly.graph_objs as go
import pandas as pd
import numpy as np

# Step 2: Prepare the Data
mnch_outcomes = {
    'Preterm Birth': {'odds_ratio': 1.05, 'current_rate': 0.1, 'cost_per_case': 1000},
    'Low Birth Weight': {'odds_ratio': 1.03, 'current_rate': 0.08, 'cost_per_case': 800},
    'Stillbirth': {'odds_ratio': 1.04, 'current_rate': 0.005, 'cost_per_case': 5000}
}

rcp_scenarios = {
    'RCP 2.6': 1.5,  # degrees Celsius increase by 2100
    'RCP 4.5': 2.4,
    'RCP 6.0': 3.0,
    'RCP 8.5': 4.3
}

population_growth_rates = {
    'Low Growth': 0.01,  # 1% annual growth
    'Medium Growth': 0.02,  # 2% annual growth
    'High Growth': 0.03  # 3% annual growth
}

pregnancy_growth_rates = {
    'High Reduction': -0.15,  # 15% reduction
    'Medium Reduction': -0.1,  # 10% reduction
    'Low Reduction': -0.05,  # 5% reduction
    'Low Growth': 0.01,  # 1% annual growth
    'Medium Growth': 0.02,  # 2% annual growth
    'High Growth': 0.03  # 3% annual growth
}

years = list(range(2024, 2051))

def calculate_population_growth(initial_population, growth_rate, num_years):
    return [initial_population * ((1 + growth_rate) ** year) for year in range(num_years)]

def calculate_pregnancies(population, pregnancy_rate):
    return [pop * pregnancy_rate for pop in population]

def calculate_temperature_increase_per_year(temp_increase_2100, start_year, end_year):
    total_years = 2100 - 2024  # Calculate total years from 2024 to 2100
    annual_increase = temp_increase_2100 / total_years
    return [annual_increase * (year - 2024) for year in range(start_year, end_year + 1)]

def calculate_additional_outcomes_and_costs(outcome, rcp_scenario, population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, adaptation_effectiveness):
    odds_ratio = mnch_outcomes[outcome]['odds_ratio']
    current_rate = mnch_outcomes[outcome]['current_rate']
    cost_per_case = mnch_outcomes[outcome]['cost_per_case']
    temp_increase_2100 = rcp_scenarios[rcp_scenario]
    population_growth_rate = population_growth_rates[population_growth_curve]
    pregnancy_growth_rate = pregnancy_growth_rates[pregnancy_growth_curve]
    
    start_year, end_year = time_period
    num_years = end_year - start_year + 1
    selected_years = list(range(start_year, end_year + 1))
    
    # Calculate population growth
    population_growth_selected = calculate_population_growth(current_population, population_growth_rate, num_years)
    
    # Calculate pregnancies without the effect of heat
    pregnancies = calculate_pregnancies(population_growth_selected, initial_pregnancy_rate * (1 + pregnancy_growth_rate))
    
    # Calculate baseline outcomes without the effect of heat
    baseline_outcomes = [preg * current_rate for preg in pregnancies]
    
    # Calculate the temperature increase per year
    temperature_increase_per_year = calculate_temperature_increase_per_year(temp_increase_2100, start_year, end_year)
    
    # Calculate projected outcomes with the effect of heat
    projected_rates = [current_rate * (odds_ratio ** temp_increase) for temp_increase in temperature_increase_per_year]
    projected_outcomes = [rate * preg for rate, preg in zip(projected_rates, pregnancies)]
    
    # Calculate additional outcomes due to the effect of heat
    additional_outcomes = [proj - base for proj, base in zip(projected_outcomes, baseline_outcomes)]
    
    # Apply adaptation effectiveness
    effective_additional_outcomes = [outcome * (1 - adaptation_effectiveness / 100) for outcome in additional_outcomes]
    
    # Calculate the total cost
    costs = [outcome * cost_per_case for outcome in effective_additional_outcomes]
    
    return selected_years, effective_additional_outcomes, costs, additional_outcomes

# Step 3: Create the Dash Layout
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("MNCH Outcomes Due to Heat - Climate Change Projection"),
    html.Label("Select MNCH Outcome"),
    dcc.Dropdown(
        id='mnch-outcome-dropdown',
        options=[{'label': outcome, 'value': outcome} for outcome in mnch_outcomes.keys()],
        value='Preterm Birth'
    ),
    html.Label("Select RCP Scenario"),
    dcc.Dropdown(
        id='rcp-scenario-dropdown',
        options=[{'label': scenario, 'value': scenario} for scenario in rcp_scenarios.keys()],
        value='RCP 2.6'
    ),
    html.Label("Select Population Growth Curve"),
    dcc.Dropdown(
        id='population-growth-dropdown',
        options=[{'label': growth, 'value': growth} for growth in population_growth_rates.keys()],
        value='Medium Growth'
    ),
    html.Label("Select Pregnancy Growth Rate"),
    dcc.Dropdown(
        id='pregnancy-growth-dropdown',
        options=[{'label': growth, 'value': growth} for growth in pregnancy_growth_rates.keys()],
        value='Medium Growth'
    ),
    html.Label("Select Time Period"),
    dcc.RangeSlider(
        id='time-period-slider',
        min=2024,
        max=2050,
        step=1,
        marks={year: str(year) for year in years},
        value=[2024, 2030]
    ),
    html.Label("Current Population"),
    dcc.Input(
        id='current-population-input',
        type='number',
        value=7000000000
    ),
    html.Label("Initial Pregnancy Rate"),
    dcc.Input(
        id='initial-pregnancy-rate-input',
        type='number',
        value=0.02,  # 2% initial pregnancy rate
        step=0.001
    ),
    html.Label("Adaptation Effectiveness (%)"),
    dcc.Slider(
        id='adaptation-effectiveness-slider',
        min=0,
        max=100,
        step=1,
        value=50,
        marks={i: f'{i}%' for i in range(0, 101, 10)}
    ),
    html.Div(id='odds-and-rate-display'),
    dcc.Graph(id='outcome-projection-graph'),
    dcc.Graph(id='additional-outcomes-prevented-graph'),
    dcc.Graph(id='cost-savings-graph'),
    dcc.Graph(id='cost-projection-graph'),
    html.Div(id='additional-outcome-display', style={'margin-top': '20px', 'font-size': '20px'}),
    html.Button("Export Data", id="export-button"),
    dcc.Download(id="download-dataframe-csv")
])

# Step 4: Add Interactivity
@app.callback(
    Output('odds-and-rate-display', 'children'),
    Output('outcome-projection-graph', 'figure'),
    Output('additional-outcomes-prevented-graph', 'figure'),
    Output('cost-savings-graph', 'figure'),
    Output('cost-projection-graph', 'figure'),
    Output('additional-outcome-display', 'children'),
    Input('mnch-outcome-dropdown', 'value'),
    Input('rcp-scenario-dropdown', 'value'),
    Input('population-growth-dropdown', 'value'),
    Input('pregnancy-growth-dropdown', 'value'),
    Input('time-period-slider', 'value'),
    Input('current-population-input', 'value'),
    Input('initial-pregnancy-rate-input', 'value'),
    Input('adaptation-effectiveness-slider', 'value')
)
def update_dashboard(outcome, rcp_scenario, population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, adaptation_effectiveness):
    # Calculate selected scenario outcomes and costs
    selected_years, effective_additional_outcomes, costs, additional_outcomes = calculate_additional_outcomes_and_costs(
        outcome, rcp_scenario, population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, adaptation_effectiveness
    )
    
    # Calculate worst scenario (RCP 8.5, 0% adaptation effectiveness) outcomes and costs
    _, worst_effective_additional_outcomes, worst_costs, worst_additional_outcomes = calculate_additional_outcomes_and_costs(
        outcome, 'RCP 8.5', population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, 0
    )
    
    # Calculate cost savings
    cost_savings = [worst - selected for worst, selected in zip(worst_costs, costs)]
    
    # Calculate additional outcomes prevented
    additional_outcomes_prevented = [worst - selected for worst, selected in zip(worst_additional_outcomes, effective_additional_outcomes)]
    
    total_additional_outcomes = sum(effective_additional_outcomes)
    total_cost = sum(costs)
    total_cost_savings = sum(cost_savings)
    total_additional_outcomes_prevented = sum(additional_outcomes_prevented)
    
    display_text = f"Odds Ratio per 1°C increase: {mnch_outcomes[outcome]['odds_ratio']}, Current Rate: {mnch_outcomes[outcome]['current_rate']}, Initial Pregnancy Rate: {initial_pregnancy_rate}, Adaptation Effectiveness: {adaptation_effectiveness}%, Cost per Case: ${mnch_outcomes[outcome]['cost_per_case']}"
    
    fig_outcomes = go.Figure()
    
    # Add additional outcomes
    fig_outcomes.add_trace(go.Scatter(x=selected_years, y=effective_additional_outcomes, mode='lines', name='Effective Additional Outcomes'))
    
    # Add temperature increase
    temperature_increase_per_year = calculate_temperature_increase_per_year(rcp_scenarios[rcp_scenario], time_period[0], time_period[1])
    fig_outcomes.add_trace(go.Scatter(x=selected_years, y=temperature_increase_per_year, mode='lines', name='Temperature Increase', yaxis='y2'))
    
    # Update layout for multiple y-axes
    fig_outcomes.update_layout(
        title=f"Effective Additional {outcome} Outcomes Due to Heat and Temperature Increase from {time_period[0]} to {time_period[1]}",
        xaxis_title="Year",
        yaxis=dict(
            title="Effective Additional Outcomes",
            titlefont=dict(color="#1f77b4"),
            tickfont=dict(color="#1f77b4")
        ),
        yaxis2=dict(
            title="Temperature Increase (°C)",
            titlefont=dict(color="#ff7f0e"),
            tickfont=dict(color="#ff7f0e"),
            overlaying='y',
            side='right'
        )
    )
    
    fig_additional_outcomes_prevented = go.Figure()
    
    # Add worst-case scenario additional outcomes
    fig_additional_outcomes_prevented.add_trace(go.Scatter(x=selected_years, y=worst_additional_outcomes, mode='lines', name='Worst Case Additional Outcomes (RCP 8.5, 0% Adaptation)'))
    
    # Add selected scenario additional outcomes
    fig_additional_outcomes_prevented.add_trace(go.Scatter(x=selected_years, y=effective_additional_outcomes, mode='lines', name=f'Selected Scenario Additional Outcomes ({rcp_scenario}, {adaptation_effectiveness}% Adaptation)'))
    
    # Shade the area between the graphs to represent additional outcomes prevented
    fig_additional_outcomes_prevented.add_trace(go.Scatter(
        x=selected_years + selected_years[::-1],
        y=worst_additional_outcomes + effective_additional_outcomes[::-1],
        fill='toself',
        fillcolor='rgba(0, 255, 0, 0.2)',
        line=dict(color='rgba(255, 255, 255, 0)'),
        showlegend=False,
        name='Additional Outcomes Prevented'
    ))
    
    fig_additional_outcomes_prevented.update_layout(
        title=f"Additional {outcome} Outcomes Prevented by Adaptation from {time_period[0]} to {time_period[1]}",
        xaxis_title="Year",
        yaxis=dict(
            title="Additional Outcomes",
            titlefont=dict(color="#1f77b4"),
            tickfont=dict(color="#1f77b4")
        )
    )
    
    fig_cost_savings = go.Figure()
    
    # Add worst-case scenario costs
    fig_cost_savings.add_trace(go.Scatter(x=selected_years, y=worst_costs, mode='lines', name='Worst Case Costs (RCP 8.5, 0% Adaptation)'))
    
    # Add selected scenario costs
    fig_cost_savings.add_trace(go.Scatter(x=selected_years, y=costs, mode='lines', name=f'Selected Scenario Costs ({rcp_scenario}, {adaptation_effectiveness}% Adaptation)'))
    
    # Shade the area between the graphs to represent cost savings
    fig_cost_savings.add_trace(go.Scatter(
        x=selected_years + selected_years[::-1],
        y=worst_costs + costs[::-1],
        fill='toself',
        fillcolor='rgba(0, 255, 0, 0.2)',
        line=dict(color='rgba(255, 255, 255, 0)'),
        showlegend=False,
        name='Cost Savings'
    ))
    
    fig_cost_savings.update_layout(
        title=f"Cost Savings by Comparing Worst Scenario (RCP 8.5, 0% Adaptation) and Selected Scenario from {time_period[0]} to {time_period[1]}",
        xaxis_title="Year",
        yaxis=dict(
            title="Costs (USD)",
            titlefont=dict(color="#1f77b4"),
            tickfont=dict(color="#1f77b4")
        )
    )
    
    fig_costs = go.Figure()
    
    # Add costs
    fig_costs.add_trace(go.Scatter(x=selected_years, y=costs, mode='lines', name='Costs'))
    
    fig_costs.update_layout(
        title=f"Costs Associated with Effective Additional {outcome} Outcomes Due to Heat from {time_period[0]} to {time_period[1]}",
        xaxis_title="Year",
        yaxis=dict(
            title="Costs (USD)",
            titlefont=dict(color="#1f77b4"),
            tickfont=dict(color="#1f77b4")
        )
    )
    
    additional_outcome_text = f"Total Effective Additional {outcome} Outcomes from {time_period[0]} to {time_period[1]}: {total_additional_outcomes:.2f}\nTotal Cost: ${total_cost:.2f}\nTotal Cost Savings: ${total_cost_savings:.2f}\nTotal Additional Outcomes Prevented: {total_additional_outcomes_prevented:.2f}"
    
    return display_text, fig_outcomes, fig_additional_outcomes_prevented, fig_cost_savings, fig_costs, additional_outcome_text

# Export Data Callback
@app.callback(
    Output("download-dataframe-csv", "data"),
    Input("export-button", "n_clicks"),
    prevent_initial_call=True,
)
def export_data(n_clicks):
    if n_clicks is None:
        return dash.no_update

    # Retrieve values from the update_dashboard function (need to be moved to global scope)
    outcome = 'Preterm Birth'  # or get dynamically if possible
    rcp_scenario = 'RCP 2.6'  # or get dynamically if possible
    population_growth_curve = 'Medium Growth'  # or get dynamically if possible
    pregnancy_growth_curve = 'Medium Growth'  # or get dynamically if possible
    time_period = [2024, 2030]  # or get dynamically if possible
    current_population = 7000000000  # or get dynamically if possible
    initial_pregnancy_rate = 0.02  # or get dynamically if possible
    adaptation_effectiveness = 50  # or get dynamically if possible
    
    selected_years, effective_additional_outcomes, costs, additional_outcomes = calculate_additional_outcomes_and_costs(
        outcome, rcp_scenario, population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, adaptation_effectiveness
    )
    
    _, worst_effective_additional_outcomes, worst_costs, worst_additional_outcomes = calculate_additional_outcomes_and_costs(
        outcome, 'RCP 8.5', population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, 0
    )
    
    cost_savings = [worst - selected for worst, selected in zip(worst_costs, costs)]
    additional_outcomes_prevented = [worst - selected for worst, selected in zip(worst_additional_outcomes, effective_additional_outcomes)]

    # Create a DataFrame with the current projections
    data = {
        'Year': selected_years,
        'Effective Additional Outcomes': effective_additional_outcomes,
        'Temperature Increase': calculate_temperature_increase_per_year(rcp_scenarios[rcp_scenario], time_period[0], time_period[1]),
        'Costs': costs,
        'Worst Case Costs': worst_costs,
        'Cost Savings': cost_savings,
        'Additional Outcomes Prevented': additional_outcomes_prevented
    }
    df = pd.DataFrame(data)

    return dcc.send_data_frame(df.to_csv, "projected_outcomes.csv")

# Step 5: Run the App
if __name__ == '__main__':
    app.run_server(debug=True)


In [22]:
#version 9 with mitigation and adaptation
# Import necessary packages
import dash
from dash import dcc, html, Input, Output
import plotly.graph_objs as go
import pandas as pd
import numpy as np

# Step 2: Prepare the Data
mnch_outcomes = {
    'Preterm Birth': {'odds_ratio': 1.05, 'current_rate': 0.1, 'cost_per_case': 1000},
    'Low Birth Weight': {'odds_ratio': 1.03, 'current_rate': 0.08, 'cost_per_case': 800},
    'Stillbirth': {'odds_ratio': 1.04, 'current_rate': 0.005, 'cost_per_case': 5000}
}

rcp_scenarios = {
    'RCP 2.6': 1.5,  # degrees Celsius increase by 2100
    'RCP 4.5': 2.4,
    'RCP 6.0': 3.0,
    'RCP 8.5': 4.3
}

population_growth_rates = {
    'Low Growth': 0.01,  # 1% annual growth
    'Medium Growth': 0.02,  # 2% annual growth
    'High Growth': 0.03  # 3% annual growth
}

pregnancy_growth_rates = {
    'High Reduction': -0.15,  # 15% reduction
    'Medium Reduction': -0.1,  # 10% reduction
    'Low Reduction': -0.05,  # 5% reduction
    'Low Growth': 0.01,  # 1% annual growth
    'Medium Growth': 0.02,  # 2% annual growth
    'High Growth': 0.03  # 3% annual growth
}

years = list(range(2024, 2051))

def calculate_population_growth(initial_population, growth_rate, num_years):
    return [initial_population * ((1 + growth_rate) ** year) for year in range(num_years)]

def calculate_pregnancies(population, pregnancy_rate):
    return [pop * pregnancy_rate for pop in population]

def calculate_temperature_increase_per_year(temp_increase_2100, start_year, end_year):
    total_years = 2100 - 2024  # Calculate total years from 2024 to 2100
    annual_increase = temp_increase_2100 / total_years
    return [annual_increase * (year - 2024) for year in range(start_year, end_year + 1)]

def calculate_additional_outcomes_and_costs(outcome, rcp_scenario, population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, adaptation_effectiveness):
    odds_ratio = mnch_outcomes[outcome]['odds_ratio']
    current_rate = mnch_outcomes[outcome]['current_rate']
    cost_per_case = mnch_outcomes[outcome]['cost_per_case']
    temp_increase_2100 = rcp_scenarios[rcp_scenario]
    population_growth_rate = population_growth_rates[population_growth_curve]
    pregnancy_growth_rate = pregnancy_growth_rates[pregnancy_growth_curve]
    
    start_year, end_year = time_period
    num_years = end_year - start_year + 1
    selected_years = list(range(start_year, end_year + 1))
    
    # Calculate population growth
    population_growth_selected = calculate_population_growth(current_population, population_growth_rate, num_years)
    
    # Calculate pregnancies without the effect of heat
    pregnancies = calculate_pregnancies(population_growth_selected, initial_pregnancy_rate * (1 + pregnancy_growth_rate))
    
    # Calculate baseline outcomes without the effect of heat
    baseline_outcomes = [preg * current_rate for preg in pregnancies]
    
    # Calculate the temperature increase per year
    temperature_increase_per_year = calculate_temperature_increase_per_year(temp_increase_2100, start_year, end_year)
    
    # Calculate projected outcomes with the effect of heat
    projected_rates = [current_rate * (odds_ratio ** temp_increase) for temp_increase in temperature_increase_per_year]
    projected_outcomes = [rate * preg for rate, preg in zip(projected_rates, pregnancies)]
    
    # Calculate additional outcomes due to the effect of heat
    additional_outcomes = [proj - base for proj, base in zip(projected_outcomes, baseline_outcomes)]
    
    # Apply adaptation effectiveness
    effective_additional_outcomes = [outcome * (1 - adaptation_effectiveness / 100) for outcome in additional_outcomes]
    
    # Calculate the total cost
    costs = [outcome * cost_per_case for outcome in effective_additional_outcomes]
    
    return selected_years, effective_additional_outcomes, costs, additional_outcomes

# Step 3: Create the Dash Layout
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("MNCH Outcomes Due to Heat - Climate Change Projection"),
    html.Label("Select MNCH Outcome"),
    dcc.Dropdown(
        id='mnch-outcome-dropdown',
        options=[{'label': outcome, 'value': outcome} for outcome in mnch_outcomes.keys()],
        value='Preterm Birth'
    ),
    html.Label("Select RCP Scenario"),
    dcc.Dropdown(
        id='rcp-scenario-dropdown',
        options=[{'label': scenario, 'value': scenario} for scenario in rcp_scenarios.keys()],
        value='RCP 2.6'
    ),
    html.Label("Select Population Growth Curve"),
    dcc.Dropdown(
        id='population-growth-dropdown',
        options=[{'label': growth, 'value': growth} for growth in population_growth_rates.keys()],
        value='Medium Growth'
    ),
    html.Label("Select Pregnancy Growth Rate"),
    dcc.Dropdown(
        id='pregnancy-growth-dropdown',
        options=[{'label': growth, 'value': growth} for growth in pregnancy_growth_rates.keys()],
        value='Medium Growth'
    ),
    html.Label("Select Time Period"),
    dcc.RangeSlider(
        id='time-period-slider',
        min=2024,
        max=2050,
        step=1,
        marks={year: str(year) for year in years},
        value=[2024, 2030]
    ),
    html.Label("Current Population"),
    dcc.Input(
        id='current-population-input',
        type='number',
        value=7000000000
    ),
    html.Label("Initial Pregnancy Rate"),
    dcc.Input(
        id='initial-pregnancy-rate-input',
        type='number',
        value=0.02,  # 2% initial pregnancy rate
        step=0.001
    ),
    html.Label("Adaptation Effectiveness (%)"),
    dcc.Slider(
        id='adaptation-effectiveness-slider',
        min=0,
        max=100,
        step=1,
        value=50,
        marks={i: f'{i}%' for i in range(0, 101, 10)}
    ),
    html.Div(id='odds-and-rate-display'),
    dcc.Graph(id='outcome-projection-graph'),
    dcc.Graph(id='additional-outcomes-prevented-graph'),
    dcc.Graph(id='cost-projection-graph'),
    dcc.Graph(id='cost-savings-graph'),
    html.Div(id='additional-outcome-display', style={'margin-top': '20px', 'font-size': '20px'}),
    html.Button("Export Data", id="export-button"),
    dcc.Download(id="download-dataframe-csv")
])

# Step 4: Add Interactivity
@app.callback(
    Output('odds-and-rate-display', 'children'),
    Output('outcome-projection-graph', 'figure'),
    Output('additional-outcomes-prevented-graph', 'figure'),
    Output('cost-projection-graph', 'figure'),
    Output('cost-savings-graph', 'figure'),
    Output('additional-outcome-display', 'children'),
    Input('mnch-outcome-dropdown', 'value'),
    Input('rcp-scenario-dropdown', 'value'),
    Input('population-growth-dropdown', 'value'),
    Input('pregnancy-growth-dropdown', 'value'),
    Input('time-period-slider', 'value'),
    Input('current-population-input', 'value'),
    Input('initial-pregnancy-rate-input', 'value'),
    Input('adaptation-effectiveness-slider', 'value')
)
def update_dashboard(outcome, rcp_scenario, population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, adaptation_effectiveness):
    # Calculate selected scenario outcomes and costs
    selected_years, effective_additional_outcomes, costs, additional_outcomes = calculate_additional_outcomes_and_costs(
        outcome, rcp_scenario, population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, adaptation_effectiveness
    )
    
    # Calculate worst scenario (RCP 8.5, 0% adaptation effectiveness) outcomes and costs
    _, worst_effective_additional_outcomes, worst_costs, worst_additional_outcomes = calculate_additional_outcomes_and_costs(
        outcome, 'RCP 8.5', population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, 0
    )
    
    # Calculate 0% adaptation scenario outcomes and costs for comparison
    _, zero_adaptation_outcomes, zero_adaptation_costs, _ = calculate_additional_outcomes_and_costs(
        outcome, rcp_scenario, population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, 0
    )
    
    # Calculate cost savings
    mitigation_savings = [worst - selected for worst, selected in zip(worst_costs, costs)]
    adaptation_savings = [zero - selected for zero, selected in zip(zero_adaptation_costs, costs)]
    
    # Calculate additional outcomes prevented
    mitigation_prevented = [worst - selected for worst, selected in zip(worst_additional_outcomes, effective_additional_outcomes)]
    adaptation_prevented = [zero - selected for zero, selected in zip(zero_adaptation_outcomes, effective_additional_outcomes)]
    
    total_additional_outcomes = sum(effective_additional_outcomes)
    total_cost = sum(costs)
    total_mitigation_savings = sum(mitigation_savings)
    total_adaptation_savings = sum(adaptation_savings)
    total_mitigation_prevented = sum(mitigation_prevented)
    total_adaptation_prevented = sum(adaptation_prevented)
    
    display_text = f"Odds Ratio per 1°C increase: {mnch_outcomes[outcome]['odds_ratio']}, Current Rate: {mnch_outcomes[outcome]['current_rate']}, Initial Pregnancy Rate: {initial_pregnancy_rate}, Adaptation Effectiveness: {adaptation_effectiveness}%, Cost per Case: ${mnch_outcomes[outcome]['cost_per_case']}"
    
    fig_outcomes = go.Figure()
    
    # Add additional outcomes
    fig_outcomes.add_trace(go.Scatter(x=selected_years, y=effective_additional_outcomes, mode='lines', name='Effective Additional Outcomes'))
    
    # Add temperature increase
    temperature_increase_per_year = calculate_temperature_increase_per_year(rcp_scenarios[rcp_scenario], time_period[0], time_period[1])
    fig_outcomes.add_trace(go.Scatter(x=selected_years, y=temperature_increase_per_year, mode='lines', name='Temperature Increase', yaxis='y2'))
    
    # Update layout for multiple y-axes
    fig_outcomes.update_layout(
        title=f"Effective Additional {outcome} Outcomes Due to Heat and Temperature Increase from {time_period[0]} to {time_period[1]}",
        xaxis_title="Year",
        yaxis=dict(
            title="Effective Additional Outcomes",
            titlefont=dict(color="#1f77b4"),
            tickfont=dict(color="#1f77b4")
        ),
        yaxis2=dict(
            title="Temperature Increase (°C)",
            titlefont=dict(color="#ff7f0e"),
            tickfont=dict(color="#ff7f0e"),
            overlaying='y',
            side='right'
        )
    )
    
    fig_additional_outcomes_prevented = go.Figure()
    
    # Add worst-case scenario additional outcomes
    fig_additional_outcomes_prevented.add_trace(go.Scatter(x=selected_years, y=worst_additional_outcomes, mode='lines', name='Worst Case Additional Outcomes (RCP 8.5, 0% Adaptation)'))
    
    # Add selected scenario additional outcomes
    fig_additional_outcomes_prevented.add_trace(go.Scatter(x=selected_years, y=effective_additional_outcomes, mode='lines', name=f'Selected Scenario Additional Outcomes ({rcp_scenario}, {adaptation_effectiveness}% Adaptation)'))
    
    # Shade the area between the graphs to represent mitigation and adaptation
    fig_additional_outcomes_prevented.add_trace(go.Scatter(
        x=selected_years + selected_years[::-1],
        y=worst_additional_outcomes + effective_additional_outcomes[::-1],
        fill='toself',
        fillcolor='rgba(0, 255, 0, 0.2)',
        line=dict(color='rgba(255, 255, 255, 0)'),
        showlegend=True,
        name='Mitigation'
    ))
    fig_additional_outcomes_prevented.add_trace(go.Scatter(
        x=selected_years + selected_years[::-1],
        y=zero_adaptation_outcomes + effective_additional_outcomes[::-1],
        fill='toself',
        fillcolor='rgba(0, 0, 255, 0.2)',
        line=dict(color='rgba(255, 255, 255, 0)'),
        showlegend=True,
        name='Adaptation'
    ))
    
    fig_additional_outcomes_prevented.update_layout(
        title=f"Additional {outcome} Outcomes Prevented by Mitigation and Adaptation from {time_period[0]} to {time_period[1]}",
        xaxis_title="Year",
        yaxis=dict(
            title="Additional Outcomes",
            titlefont=dict(color="#1f77b4"),
            tickfont=dict(color="#1f77b4")
        )
    )
    
    fig_costs = go.Figure()
    
    # Add costs
    fig_costs.add_trace(go.Scatter(x=selected_years, y=costs, mode='lines', name='Costs'))
    
    fig_costs.update_layout(
        title=f"Costs Associated with Effective Additional {outcome} Outcomes Due to Heat from {time_period[0]} to {time_period[1]}",
        xaxis_title="Year",
        yaxis=dict(
            title="Costs (USD)",
            titlefont=dict(color="#1f77b4"),
            tickfont=dict(color="#1f77b4")
        )
    )
    
    fig_cost_savings = go.Figure()
    
    # Add worst-case scenario costs
    fig_cost_savings.add_trace(go.Scatter(x=selected_years, y=worst_costs, mode='lines', name='Worst Case Costs (RCP 8.5, 0% Adaptation)'))
    
    # Add selected scenario costs
    fig_cost_savings.add_trace(go.Scatter(x=selected_years, y=costs, mode='lines', name=f'Selected Scenario Costs ({rcp_scenario}, {adaptation_effectiveness}% Adaptation)'))
    
    # Shade the area between the graphs to represent mitigation and adaptation
    fig_cost_savings.add_trace(go.Scatter(
        x=selected_years + selected_years[::-1],
        y=worst_costs + costs[::-1],
        fill='toself',
        fillcolor='rgba(0, 255, 0, 0.2)',
        line=dict(color='rgba(255, 255, 255, 0)'),
        showlegend=True,
        name='Mitigation'
    ))
    fig_cost_savings.add_trace(go.Scatter(
        x=selected_years + selected_years[::-1],
        y=zero_adaptation_costs + costs[::-1],
        fill='toself',
        fillcolor='rgba(0, 0, 255, 0.2)',
        line=dict(color='rgba(255, 255, 255, 0)'),
        showlegend=True,
        name='Adaptation'
    ))
    
    fig_cost_savings.update_layout(
        title=f"Cost Savings by Comparing Worst Scenario (RCP 8.5, 0% Adaptation) and Selected Scenario from {time_period[0]} to {time_period[1]}",
        xaxis_title="Year",
        yaxis=dict(
            title="Costs (USD)",
            titlefont=dict(color="#1f77b4"),
            tickfont=dict(color="#1f77b4")
        )
    )
    
    additional_outcome_text = f"Total Effective Additional {outcome} Outcomes from {time_period[0]} to {time_period[1]}: {total_additional_outcomes:.2f}\nTotal Cost: ${total_cost:.2f}\nTotal Mitigation Savings: ${total_mitigation_savings:.2f}\nTotal Adaptation Savings: ${total_adaptation_savings:.2f}\nTotal Mitigation Prevented: {total_mitigation_prevented:.2f}\nTotal Adaptation Prevented: {total_adaptation_prevented:.2f}"
    
    return display_text, fig_outcomes, fig_additional_outcomes_prevented, fig_costs, fig_cost_savings, additional_outcome_text

# Export Data Callback
@app.callback(
    Output("download-dataframe-csv", "data"),
    Input("export-button", "n_clicks"),
    prevent_initial_call=True,
)
def export_data(n_clicks):
    if n_clicks is None:
        return dash.no_update

    # Retrieve values from the update_dashboard function (need to be moved to global scope)
    outcome = 'Preterm Birth'  # or get dynamically if possible
    rcp_scenario = 'RCP 2.6'  # or get dynamically if possible
    population_growth_curve = 'Medium Growth'  # or get dynamically if possible
    pregnancy_growth_curve = 'Medium Growth'  # or get dynamically if possible
    time_period = [2024, 2030]  # or get dynamically if possible
    current_population = 7000000000  # or get dynamically if possible
    initial_pregnancy_rate = 0.02  # or get dynamically if possible
    adaptation_effectiveness = 50  # or get dynamically if possible
    
    selected_years, effective_additional_outcomes, costs, additional_outcomes = calculate_additional_outcomes_and_costs(
        outcome, rcp_scenario, population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, adaptation_effectiveness
    )
    
    _, worst_effective_additional_outcomes, worst_costs, worst_additional_outcomes = calculate_additional_outcomes_and_costs(
        outcome, 'RCP 8.5', population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, 0
    )
    
    _, zero_adaptation_outcomes, zero_adaptation_costs, _ = calculate_additional_outcomes_and_costs(
        outcome, rcp_scenario, population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, 0
    )
    
    mitigation_savings = [worst - selected for worst, selected in zip(worst_costs, costs)]
    adaptation_savings = [zero - selected for zero, selected in zip(zero_adaptation_costs, costs)]
    mitigation_prevented = [worst - selected for worst, selected in zip(worst_additional_outcomes, effective_additional_outcomes)]
    adaptation_prevented = [zero - selected for zero, selected in zip(zero_adaptation_outcomes, effective_additional_outcomes)]

    # Create a DataFrame with the current projections
    data = {
        'Year': selected_years,
        'Effective Additional Outcomes': effective_additional_outcomes,
        'Temperature Increase': calculate_temperature_increase_per_year(rcp_scenarios[rcp_scenario], time_period[0], time_period[1]),
        'Costs': costs,
        'Worst Case Costs': worst_costs,
        'Zero Adaptation Costs': zero_adaptation_costs,
        'Mitigation Savings': mitigation_savings,
        'Adaptation Savings': adaptation_savings,
        'Mitigation Prevented': mitigation_prevented,
        'Adaptation Prevented': adaptation_prevented
    }
    df = pd.DataFrame(data)

    return dcc.send_data_frame(df.to_csv, "projected_outcomes.csv")

# Step 5: Run the App
if __name__ == '__main__':
    app.run_server(debug=True)


In [24]:
# Import necessary packages
import dash
from dash import dcc, html, Input, Output
import plotly.graph_objs as go
import pandas as pd
import numpy as np

# Step 2: Prepare the Data
mnch_outcomes = {
    'Preterm Birth': {'odds_ratio': 1.05, 'current_rate': 0.1, 'cost_per_case': 1000},
    'Low Birth Weight': {'odds_ratio': 1.03, 'current_rate': 0.08, 'cost_per_case': 800},
    'Stillbirth': {'odds_ratio': 1.04, 'current_rate': 0.005, 'cost_per_case': 5000}
}

rcp_scenarios = {
    'RCP 2.6': 1.5,  # degrees Celsius increase by 2100
    'RCP 4.5': 2.4,
    'RCP 6.0': 3.0,
    'RCP 8.5': 4.3
}

population_growth_rates = {
    'Low Growth': 0.01,  # 1% annual growth
    'Medium Growth': 0.02,  # 2% annual growth
    'High Growth': 0.03  # 3% annual growth
}

pregnancy_growth_rates = {
    'High Reduction': -0.15,  # 15% reduction
    'Medium Reduction': -0.1,  # 10% reduction
    'Low Reduction': -0.05,  # 5% reduction
    'Low Growth': 0.01,  # 1% annual growth
    'Medium Growth': 0.02,  # 2% annual growth
    'High Growth': 0.03  # 3% annual growth
}

years = list(range(2024, 2051))

def calculate_population_growth(initial_population, growth_rate, num_years):
    return [initial_population * ((1 + growth_rate) ** year) for year in range(num_years)]

def calculate_pregnancies(population, pregnancy_rate):
    return [pop * pregnancy_rate for pop in population]

def calculate_temperature_increase_per_year(temp_increase_2100, start_year, end_year):
    total_years = 2100 - 2024  # Calculate total years from 2024 to 2100
    annual_increase = temp_increase_2100 / total_years
    return [annual_increase * (year - 2024) for year in range(start_year, end_year + 1)]

def calculate_additional_outcomes_and_costs(outcome, rcp_scenario, population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, adaptation_effectiveness):
    odds_ratio = mnch_outcomes[outcome]['odds_ratio']
    current_rate = mnch_outcomes[outcome]['current_rate']
    cost_per_case = mnch_outcomes[outcome]['cost_per_case']
    temp_increase_2100 = rcp_scenarios[rcp_scenario]
    population_growth_rate = population_growth_rates[population_growth_curve]
    pregnancy_growth_rate = pregnancy_growth_rates[pregnancy_growth_curve]
    
    start_year, end_year = time_period
    num_years = end_year - start_year + 1
    selected_years = list(range(start_year, end_year + 1))
    
    # Calculate population growth
    population_growth_selected = calculate_population_growth(current_population, population_growth_rate, num_years)
    
    # Calculate pregnancies without the effect of heat
    pregnancies = calculate_pregnancies(population_growth_selected, initial_pregnancy_rate * (1 + pregnancy_growth_rate))
    
    # Calculate baseline outcomes without the effect of heat
    baseline_outcomes = [preg * current_rate for preg in pregnancies]
    
    # Calculate the temperature increase per year
    temperature_increase_per_year = calculate_temperature_increase_per_year(temp_increase_2100, start_year, end_year)
    
    # Calculate projected outcomes with the effect of heat
    projected_rates = [current_rate * (odds_ratio ** temp_increase) for temp_increase in temperature_increase_per_year]
    projected_outcomes = [rate * preg for rate, preg in zip(projected_rates, pregnancies)]
    
    # Calculate additional outcomes due to the effect of heat
    additional_outcomes = [proj - base for proj, base in zip(projected_outcomes, baseline_outcomes)]
    
    # Apply adaptation effectiveness
    effective_additional_outcomes = [outcome * (1 - adaptation_effectiveness / 100) for outcome in additional_outcomes]
    
    # Calculate the total cost
    costs = [outcome * cost_per_case for outcome in effective_additional_outcomes]
    
    return selected_years, effective_additional_outcomes, costs, additional_outcomes

# Step 3: Create the Dash Layout
app = dash.Dash(__name__)

app.layout = html.Div([
    html.Link(rel='stylesheet', href='/assets/style.css'),
    html.H1("MNCH Outcomes Due to Heat - Climate Change Projection"),
    html.Div(className='container', children=[
        html.Div(className='box', children=[
            html.H2("Select MNCH Outcome"),
            dcc.Dropdown(
                id='mnch-outcome-dropdown',
                options=[{'label': outcome, 'value': outcome} for outcome in mnch_outcomes.keys()],
                value='Preterm Birth',
                className='dropdown'
            )
        ]),
        html.Div(className='box', children=[
            html.H2("Select RCP Scenario"),
            dcc.Dropdown(
                id='rcp-scenario-dropdown',
                options=[{'label': scenario, 'value': scenario} for scenario in rcp_scenarios.keys()],
                value='RCP 2.6',
                className='dropdown'
            )
        ]),
        html.Div(className='box', children=[
            html.H2("Select Population Growth Curve"),
            dcc.Dropdown(
                id='population-growth-dropdown',
                options=[{'label': growth, 'value': growth} for growth in population_growth_rates.keys()],
                value='Medium Growth',
                className='dropdown'
            )
        ]),
        html.Div(className='box', children=[
            html.H2("Select Pregnancy Growth Rate"),
            dcc.Dropdown(
                id='pregnancy-growth-dropdown',
                options=[{'label': growth, 'value': growth} for growth in pregnancy_growth_rates.keys()],
                value='Medium Growth',
                className='dropdown'
            )
        ]),
        html.Div(className='box', children=[
            html.H2("Select Time Period"),
            dcc.RangeSlider(
                id='time-period-slider',
                min=2024,
                max=2050,
                step=1,
                marks={year: str(year) for year in years},
                value=[2024, 2030],
                className='slider'
            )
        ]),
        html.Div(className='box', children=[
            html.H2("Current Population"),
            dcc.Input(
                id='current-population-input',
                type='number',
                value=7000000000,
                className='input'
            )
        ]),
        html.Div(className='box', children=[
            html.H2("Initial Pregnancy Rate"),
            dcc.Input(
                id='initial-pregnancy-rate-input',
                type='number',
                value=0.02,  # 2% initial pregnancy rate
                step=0.001,
                className='input'
            )
        ]),
        html.Div(className='box', children=[
            html.H2("Adaptation Effectiveness (%)"),
            dcc.Slider(
                id='adaptation-effectiveness-slider',
                min=0,
                max=100,
                step=1,
                value=50,
                marks={i: f'{i}%' for i in range(0, 101, 10)},
                className='slider'
            )
        ]),
        html.Div(className='box', id='odds-and-rate-display', children=[]),
        html.Div(className='graph-container', children=[
            dcc.Graph(id='outcome-projection-graph'),
            dcc.Graph(id='additional-outcomes-prevented-graph'),
            dcc.Graph(id='cost-projection-graph'),
            dcc.Graph(id='cost-savings-graph')
        ]),
        html.Div(className='box', id='additional-outcome-display', style={'margin-top': '20px', 'font-size': '20px'}, children=[]),
        html.Button("Export Data", id="export-button", className='button'),
        dcc.Download(id="download-dataframe-csv")
    ])
])

# Step 4: Add Interactivity
@app.callback(
    Output('odds-and-rate-display', 'children'),
    Output('outcome-projection-graph', 'figure'),
    Output('additional-outcomes-prevented-graph', 'figure'),
    Output('cost-projection-graph', 'figure'),
    Output('cost-savings-graph', 'figure'),
    Output('additional-outcome-display', 'children'),
    Input('mnch-outcome-dropdown', 'value'),
    Input('rcp-scenario-dropdown', 'value'),
    Input('population-growth-dropdown', 'value'),
    Input('pregnancy-growth-dropdown', 'value'),
    Input('time-period-slider', 'value'),
    Input('current-population-input', 'value'),
    Input('initial-pregnancy-rate-input', 'value'),
    Input('adaptation-effectiveness-slider', 'value')
)
def update_dashboard(outcome, rcp_scenario, population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, adaptation_effectiveness):
    # Calculate selected scenario outcomes and costs
    selected_years, effective_additional_outcomes, costs, additional_outcomes = calculate_additional_outcomes_and_costs(
        outcome, rcp_scenario, population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, adaptation_effectiveness
    )
    
    # Calculate worst scenario (RCP 8.5, 0% adaptation effectiveness) outcomes and costs
    _, worst_effective_additional_outcomes, worst_costs, worst_additional_outcomes = calculate_additional_outcomes_and_costs(
        outcome, 'RCP 8.5', population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, 0
    )
    
    # Calculate 0% adaptation scenario outcomes and costs for comparison
    _, zero_adaptation_outcomes, zero_adaptation_costs, _ = calculate_additional_outcomes_and_costs(
        outcome, rcp_scenario, population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, 0
    )
    
    # Calculate cost savings
    mitigation_savings = [worst - selected for worst, selected in zip(worst_costs, costs)]
    adaptation_savings = [zero - selected for zero, selected in zip(zero_adaptation_costs, costs)]
    
    # Calculate additional outcomes prevented
    mitigation_prevented = [worst - selected for worst, selected in zip(worst_additional_outcomes, effective_additional_outcomes)]
    adaptation_prevented = [zero - selected for zero, selected in zip(zero_adaptation_outcomes, effective_additional_outcomes)]
    
    total_additional_outcomes = sum(effective_additional_outcomes)
    total_cost = sum(costs)
    total_mitigation_savings = sum(mitigation_savings)
    total_adaptation_savings = sum(adaptation_savings)
    total_mitigation_prevented = sum(mitigation_prevented)
    total_adaptation_prevented = sum(adaptation_prevented)
    
    display_text = f"Odds Ratio per 1°C increase: {mnch_outcomes[outcome]['odds_ratio']}, Current Rate: {mnch_outcomes[outcome]['current_rate']}, Initial Pregnancy Rate: {initial_pregnancy_rate}, Adaptation Effectiveness: {adaptation_effectiveness}%, Cost per Case: ${mnch_outcomes[outcome]['cost_per_case']}"
    
    fig_outcomes = go.Figure()
    
    # Add additional outcomes
    fig_outcomes.add_trace(go.Scatter(x=selected_years, y=effective_additional_outcomes, mode='lines', name='Effective Additional Outcomes'))
    
    # Add temperature increase
    temperature_increase_per_year = calculate_temperature_increase_per_year(rcp_scenarios[rcp_scenario], time_period[0], time_period[1])
    fig_outcomes.add_trace(go.Scatter(x=selected_years, y=temperature_increase_per_year, mode='lines', name='Temperature Increase', yaxis='y2'))
    
    # Update layout for multiple y-axes
    fig_outcomes.update_layout(
        title=f"Effective Additional {outcome} Outcomes Due to Heat and Temperature Increase from {time_period[0]} to {time_period[1]}",
        xaxis_title="Year",
        yaxis=dict(
            title="Effective Additional Outcomes",
            titlefont=dict(color="#1f77b4"),
            tickfont=dict(color="#1f77b4")
        ),
        yaxis2=dict(
            title="Temperature Increase (°C)",
            titlefont=dict(color="#ff7f0e"),
            tickfont=dict(color="#ff7f0e"),
            overlaying='y',
            side='right'
        )
    )
    
    fig_additional_outcomes_prevented = go.Figure()
    
    # Add worst-case scenario additional outcomes
    fig_additional_outcomes_prevented.add_trace(go.Scatter(x=selected_years, y=worst_additional_outcomes, mode='lines', name='Worst Case Additional Outcomes (RCP 8.5, 0% Adaptation)'))
    
    # Add selected scenario additional outcomes
    fig_additional_outcomes_prevented.add_trace(go.Scatter(x=selected_years, y=effective_additional_outcomes, mode='lines', name=f'Selected Scenario Additional Outcomes ({rcp_scenario}, {adaptation_effectiveness}% Adaptation)'))
    
    # Shade the area between the graphs to represent mitigation and adaptation
    fig_additional_outcomes_prevented.add_trace(go.Scatter(
        x=selected_years + selected_years[::-1],
        y=worst_additional_outcomes + effective_additional_outcomes[::-1],
        fill='toself',
        fillcolor='rgba(0, 255, 0, 0.2)',
        line=dict(color='rgba(255, 255, 255, 0)'),
        showlegend=True,
        name='Mitigation'
    ))
    fig_additional_outcomes_prevented.add_trace(go.Scatter(
        x=selected_years + selected_years[::-1],
        y=zero_adaptation_outcomes + effective_additional_outcomes[::-1],
        fill='toself',
        fillcolor='rgba(0, 0, 255, 0.2)',
        line=dict(color='rgba(255, 255, 255, 0)'),
        showlegend=True,
        name='Adaptation'
    ))
    
    fig_additional_outcomes_prevented.update_layout(
        title=f"Additional {outcome} Outcomes Prevented by Mitigation and Adaptation from {time_period[0]} to {time_period[1]}",
        xaxis_title="Year",
        yaxis=dict(
            title="Additional Outcomes",
            titlefont=dict(color="#1f77b4"),
            tickfont=dict(color="#1f77b4")
        )
    )
    
    fig_costs = go.Figure()
    
    # Add costs
    fig_costs.add_trace(go.Scatter(x=selected_years, y=costs, mode='lines', name='Costs'))
    
    fig_costs.update_layout(
        title=f"Costs Associated with Effective Additional {outcome} Outcomes Due to Heat from {time_period[0]} to {time_period[1]}",
        xaxis_title="Year",
        yaxis=dict(
            title="Costs (USD)",
            titlefont=dict(color="#1f77b4"),
            tickfont=dict(color="#1f77b4")
        )
    )
    
    fig_cost_savings = go.Figure()
    
    # Add worst-case scenario costs
    fig_cost_savings.add_trace(go.Scatter(x=selected_years, y=worst_costs, mode='lines', name='Worst Case Costs (RCP 8.5, 0% Adaptation)'))
    
    # Add selected scenario costs
    fig_cost_savings.add_trace(go.Scatter(x=selected_years, y=costs, mode='lines', name=f'Selected Scenario Costs ({rcp_scenario}, {adaptation_effectiveness}% Adaptation)'))
    
    # Shade the area between the graphs to represent mitigation and adaptation
    fig_cost_savings.add_trace(go.Scatter(
        x=selected_years + selected_years[::-1],
        y=worst_costs + costs[::-1],
        fill='toself',
        fillcolor='rgba(0, 255, 0, 0.2)',
        line=dict(color='rgba(255, 255, 255, 0)'),
        showlegend=True,
        name='Mitigation'
    ))
    fig_cost_savings.add_trace(go.Scatter(
        x=selected_years + selected_years[::-1],
        y=zero_adaptation_costs + costs[::-1],
        fill='toself',
        fillcolor='rgba(0, 0, 255, 0.2)',
        line=dict(color='rgba(255, 255, 255, 0)'),
        showlegend=True,
        name='Adaptation'
    ))
    
    fig_cost_savings.update_layout(
        title=f"Cost Savings by Comparing Worst Scenario (RCP 8.5, 0% Adaptation) and Selected Scenario from {time_period[0]} to {time_period[1]}",
        xaxis_title="Year",
        yaxis=dict(
            title="Costs (USD)",
            titlefont=dict(color="#1f77b4"),
            tickfont=dict(color="#1f77b4")
        )
    )
    
    additional_outcome_text = f"Total Effective Additional {outcome} Outcomes from {time_period[0]} to {time_period[1]}: {total_additional_outcomes:.2f}\nTotal Cost: ${total_cost:.2f}\nTotal Mitigation Savings: ${total_mitigation_savings:.2f}\nTotal Adaptation Savings: ${total_adaptation_savings:.2f}\nTotal Mitigation Prevented: {total_mitigation_prevented:.2f}\nTotal Adaptation Prevented: {total_adaptation_prevented:.2f}"
    
    return display_text, fig_outcomes, fig_additional_outcomes_prevented, fig_costs, fig_cost_savings, additional_outcome_text

# Export Data Callback
@app.callback(
    Output("download-dataframe-csv", "data"),
    Input("export-button", "n_clicks"),
    prevent_initial_call=True,
)
def export_data(n_clicks):
    if n_clicks is None:
        return dash.no_update

    # Retrieve values from the update_dashboard function (need to be moved to global scope)
    outcome = 'Preterm Birth'  # or get dynamically if possible
    rcp_scenario = 'RCP 2.6'  # or get dynamically if possible
    population_growth_curve = 'Medium Growth'  # or get dynamically if possible
    pregnancy_growth_curve = 'Medium Growth'  # or get dynamically if possible
    time_period = [2024, 2030]  # or get dynamically if possible
    current_population = 7000000000  # or get dynamically if possible
    initial_pregnancy_rate = 0.02  # or get dynamically if possible
    adaptation_effectiveness = 50  # or get dynamically if possible
    
    selected_years, effective_additional_outcomes, costs, additional_outcomes = calculate_additional_outcomes_and_costs(
        outcome, rcp_scenario, population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, adaptation_effectiveness
    )
    
    _, worst_effective_additional_outcomes, worst_costs, worst_additional_outcomes = calculate_additional_outcomes_and_costs(
        outcome, 'RCP 8.5', population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, 0
    )
    
    _, zero_adaptation_outcomes, zero_adaptation_costs, _ = calculate_additional_outcomes_and_costs(
        outcome, rcp_scenario, population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, 0
    )
    
    mitigation_savings = [worst - selected for worst, selected in zip(worst_costs, costs)]
    adaptation_savings = [zero - selected for zero, selected in zip(zero_adaptation_costs, costs)]
    mitigation_prevented = [worst - selected for worst, selected in zip(worst_additional_outcomes, effective_additional_outcomes)]
    adaptation_prevented = [zero - selected for zero, selected in zip(zero_adaptation_outcomes, effective_additional_outcomes)]

    # Create a DataFrame with the current projections
    data = {
        'Year': selected_years,
        'Effective Additional Outcomes': effective_additional_outcomes,
        'Temperature Increase': calculate_temperature_increase_per_year(rcp_scenarios[rcp_scenario], time_period[0], time_period[1]),
        'Costs': costs,
        'Worst Case Costs': worst_costs,
        'Zero Adaptation Costs': zero_adaptation_costs,
        'Mitigation Savings': mitigation_savings,
        'Adaptation Savings': adaptation_savings,
        'Mitigation Prevented': mitigation_prevented,
        'Adaptation Prevented': adaptation_prevented
    }
    df = pd.DataFrame(data)

    return dcc.send_data_frame(df.to_csv, "projected_outcomes.csv")

# Step 5: Run the App
if __name__ == '__main__':
    app.run_server(debug=True)


In [28]:
#version 10 with improved layout
# Import necessary packages
import dash
from dash import dcc, html, Input, Output
import plotly.graph_objs as go
import pandas as pd
import numpy as np

# Step 2: Prepare the Data
mnch_outcomes = {
    'Preterm Birth': {'odds_ratio': 1.05, 'current_rate': 0.1, 'cost_per_case': 1000},
    'Low Birth Weight': {'odds_ratio': 1.03, 'current_rate': 0.08, 'cost_per_case': 800},
    'Stillbirth': {'odds_ratio': 1.04, 'current_rate': 0.005, 'cost_per_case': 5000}
}

rcp_scenarios = {
    'RCP 2.6': 1.5,  # degrees Celsius increase by 2100
    'RCP 4.5': 2.4,
    'RCP 6.0': 3.0,
    'RCP 8.5': 4.3
}

population_growth_rates = {
    'Low Growth': 0.01,  # 1% annual growth
    'Medium Growth': 0.02,  # 2% annual growth
    'High Growth': 0.03  # 3% annual growth
}

pregnancy_growth_rates = {
    'High Reduction': -0.15,  # 15% reduction
    'Medium Reduction': -0.1,  # 10% reduction
    'Low Reduction': -0.05,  # 5% reduction
    'Low Growth': 0.01,  # 1% annual growth
    'Medium Growth': 0.02,  # 2% annual growth
    'High Growth': 0.03  # 3% annual growth
}

years = list(range(2024, 2051))

def calculate_population_growth(initial_population, growth_rate, num_years):
    return [initial_population * ((1 + growth_rate) ** year) for year in range(num_years)]

def calculate_pregnancies(population, pregnancy_rate):
    return [pop * pregnancy_rate for pop in population]

def calculate_temperature_increase_per_year(temp_increase_2100, start_year, end_year):
    total_years = 2100 - 2024  # Calculate total years from 2024 to 2100
    annual_increase = temp_increase_2100 / total_years
    return [annual_increase * (year - 2024) for year in range(start_year, end_year + 1)]

def calculate_additional_outcomes_and_costs(outcome, rcp_scenario, population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, adaptation_effectiveness):
    odds_ratio = mnch_outcomes[outcome]['odds_ratio']
    current_rate = mnch_outcomes[outcome]['current_rate']
    cost_per_case = mnch_outcomes[outcome]['cost_per_case']
    temp_increase_2100 = rcp_scenarios[rcp_scenario]
    population_growth_rate = population_growth_rates[population_growth_curve]
    pregnancy_growth_rate = pregnancy_growth_rates[pregnancy_growth_curve]
    
    start_year, end_year = time_period
    num_years = end_year - start_year + 1
    selected_years = list(range(start_year, end_year + 1))
    
    # Calculate population growth
    population_growth_selected = calculate_population_growth(current_population, population_growth_rate, num_years)
    
    # Calculate pregnancies without the effect of heat
    pregnancies = calculate_pregnancies(population_growth_selected, initial_pregnancy_rate * (1 + pregnancy_growth_rate))
    
    # Calculate baseline outcomes without the effect of heat
    baseline_outcomes = [preg * current_rate for preg in pregnancies]
    
    # Calculate the temperature increase per year
    temperature_increase_per_year = calculate_temperature_increase_per_year(temp_increase_2100, start_year, end_year)
    
    # Calculate projected outcomes with the effect of heat
    projected_rates = [current_rate * (odds_ratio ** temp_increase) for temp_increase in temperature_increase_per_year]
    projected_outcomes = [rate * preg for rate, preg in zip(projected_rates, pregnancies)]
    
    # Calculate additional outcomes due to the effect of heat
    additional_outcomes = [proj - base for proj, base in zip(projected_outcomes, baseline_outcomes)]
    
    # Apply adaptation effectiveness
    effective_additional_outcomes = [outcome * (1 - adaptation_effectiveness / 100) for outcome in additional_outcomes]
    
    # Calculate the total cost
    costs = [outcome * cost_per_case for outcome in effective_additional_outcomes]
    
    return selected_years, effective_additional_outcomes, costs, additional_outcomes

# Step 3: Create the Dash Layout
app = dash.Dash(__name__)

app.layout = html.Div([
    html.Link(rel='stylesheet', href='/assets/style.css'),
    html.H1("MNCH Outcomes Due to Heat - Climate Change Projection"),
    html.Div(className='container', children=[
        html.Div(className='left-box', children=[
            html.H2("Input Parameters"),
            html.Div(className='box', children=[
                html.H3("Select MNCH Outcome"),
                dcc.Dropdown(
                    id='mnch-outcome-dropdown',
                    options=[{'label': outcome, 'value': outcome} for outcome in mnch_outcomes.keys()],
                    value='Preterm Birth',
                    className='dropdown'
                )
            ]),
            html.Div(className='box', children=[
                html.H3("Select RCP Scenario"),
                dcc.Dropdown(
                    id='rcp-scenario-dropdown',
                    options=[{'label': scenario, 'value': scenario} for scenario in rcp_scenarios.keys()],
                    value='RCP 2.6',
                    className='dropdown'
                )
            ]),
            html.Div(className='box', children=[
                html.H3("Select Population Growth Curve"),
                dcc.Dropdown(
                    id='population-growth-dropdown',
                    options=[{'label': growth, 'value': growth} for growth in population_growth_rates.keys()],
                    value='Medium Growth',
                    className='dropdown'
                )
            ]),
            html.Div(className='box', children=[
                html.H3("Select Pregnancy Growth Rate"),
                dcc.Dropdown(
                    id='pregnancy-growth-dropdown',
                    options=[{'label': growth, 'value': growth} for growth in pregnancy_growth_rates.keys()],
                    value='Medium Growth',
                    className='dropdown'
                )
            ]),
            html.Div(className='box', children=[
                html.H3("Current Population"),
                dcc.Input(
                    id='current-population-input',
                    type='number',
                    value=7000000000,
                    className='input'
                )
            ]),
            html.Div(className='box', children=[
                html.H3("Initial Pregnancy Rate"),
                dcc.Input(
                    id='initial-pregnancy-rate-input',
                    type='number',
                    value=0.02,  # 2% initial pregnancy rate
                    step=0.001,
                    className='input'
                )
            ])
        ]),
        html.Div(className='right-box', children=[
            html.Div(className='box', children=[
                html.H3("Adaptation Effectiveness (%)"),
                dcc.Slider(
                    id='adaptation-effectiveness-slider',
                    min=0,
                    max=100,
                    step=1,
                    value=50,
                    marks={i: f'{i}%' for i in range(0, 101, 10)},
                    className='slider'
                )
            ]),
            html.Div(className='box', children=[
                html.H3("Select Time Period"),
                dcc.RangeSlider(
                    id='time-period-slider',
                    min=2024,
                    max=2050,
                    step=1,
                    marks={year: str(year) for year in years},
                    value=[2024, 2030],
                    className='slider'
                )
            ])
        ]),
        html.Div(className='key-indicators', children=[
            html.Div(className='key-indicator', id='odds-and-rate-display'),
            html.Div(className='key-indicator', id='additional-outcome-display')
        ]),
        html.Div(className='graph-container', children=[
            dcc.Graph(id='outcome-projection-graph'),
            dcc.Graph(id='additional-outcomes-prevented-graph'),
            dcc.Graph(id='cost-projection-graph'),
            dcc.Graph(id='cost-savings-graph')
        ]),
        html.Button("Export Data", id="export-button", className='button'),
        dcc.Download(id="download-dataframe-csv")
    ])
])

# Step 4: Add Interactivity
@app.callback(
    Output('odds-and-rate-display', 'children'),
    Output('outcome-projection-graph', 'figure'),
    Output('additional-outcomes-prevented-graph', 'figure'),
    Output('cost-projection-graph', 'figure'),
    Output('cost-savings-graph', 'figure'),
    Output('additional-outcome-display', 'children'),
    Input('mnch-outcome-dropdown', 'value'),
    Input('rcp-scenario-dropdown', 'value'),
    Input('population-growth-dropdown', 'value'),
    Input('pregnancy-growth-dropdown', 'value'),
    Input('time-period-slider', 'value'),
    Input('current-population-input', 'value'),
    Input('initial-pregnancy-rate-input', 'value'),
    Input('adaptation-effectiveness-slider', 'value')
)
def update_dashboard(outcome, rcp_scenario, population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, adaptation_effectiveness):
    # Calculate selected scenario outcomes and costs
    selected_years, effective_additional_outcomes, costs, additional_outcomes = calculate_additional_outcomes_and_costs(
        outcome, rcp_scenario, population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, adaptation_effectiveness
    )
    
    # Calculate worst scenario (RCP 8.5, 0% adaptation effectiveness) outcomes and costs
    _, worst_effective_additional_outcomes, worst_costs, worst_additional_outcomes = calculate_additional_outcomes_and_costs(
        outcome, 'RCP 8.5', population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, 0
    )
    
    # Calculate 0% adaptation scenario outcomes and costs for comparison
    _, zero_adaptation_outcomes, zero_adaptation_costs, _ = calculate_additional_outcomes_and_costs(
        outcome, rcp_scenario, population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, 0
    )
    
    # Calculate cost savings
    mitigation_savings = [worst - selected for worst, selected in zip(worst_costs, costs)]
    adaptation_savings = [zero - selected for zero, selected in zip(zero_adaptation_costs, costs)]
    
    # Calculate additional outcomes prevented
    mitigation_prevented = [worst - selected for worst, selected in zip(worst_additional_outcomes, effective_additional_outcomes)]
    adaptation_prevented = [zero - selected for zero, selected in zip(zero_adaptation_outcomes, effective_additional_outcomes)]
    
    total_additional_outcomes = sum(effective_additional_outcomes)
    total_cost = sum(costs)
    total_mitigation_savings = sum(mitigation_savings)
    total_adaptation_savings = sum(adaptation_savings)
    total_mitigation_prevented = sum(mitigation_prevented)
    total_adaptation_prevented = sum(adaptation_prevented)
    
    display_text = f"Odds Ratio per 1°C increase: {mnch_outcomes[outcome]['odds_ratio']}, Current Rate: {mnch_outcomes[outcome]['current_rate']}, Initial Pregnancy Rate: {initial_pregnancy_rate}, Adaptation Effectiveness: {adaptation_effectiveness}%, Cost per Case: ${mnch_outcomes[outcome]['cost_per_case']}"
    
    fig_outcomes = go.Figure()
    
    # Add additional outcomes
    fig_outcomes.add_trace(go.Scatter(x=selected_years, y=effective_additional_outcomes, mode='lines', name='Effective Additional Outcomes'))
    
    # Add temperature increase
    temperature_increase_per_year = calculate_temperature_increase_per_year(rcp_scenarios[rcp_scenario], time_period[0], time_period[1])
    fig_outcomes.add_trace(go.Scatter(x=selected_years, y=temperature_increase_per_year, mode='lines', name='Temperature Increase', yaxis='y2'))
    
    # Update layout for multiple y-axes
    fig_outcomes.update_layout(
        title=f"Effective Additional {outcome} Outcomes Due to Heat and Temperature Increase from {time_period[0]} to {time_period[1]}",
        xaxis_title="Year",
        yaxis=dict(
            title="Effective Additional Outcomes",
            titlefont=dict(color="#1f77b4"),
            tickfont=dict(color="#1f77b4")
        ),
        yaxis2=dict(
            title="Temperature Increase (°C)",
            titlefont=dict(color="#ff7f0e"),
            tickfont=dict(color="#ff7f0e"),
            overlaying='y',
            side='right'
        )
    )
    
    fig_additional_outcomes_prevented = go.Figure()
    
    # Add worst-case scenario additional outcomes
    fig_additional_outcomes_prevented.add_trace(go.Scatter(x=selected_years, y=worst_additional_outcomes, mode='lines', name='Worst Case Additional Outcomes (RCP 8.5, 0% Adaptation)'))
    
    # Add selected scenario additional outcomes
    fig_additional_outcomes_prevented.add_trace(go.Scatter(x=selected_years, y=effective_additional_outcomes, mode='lines', name=f'Selected Scenario Additional Outcomes ({rcp_scenario}, {adaptation_effectiveness}% Adaptation)'))
    
    # Shade the area between the graphs to represent mitigation and adaptation
    fig_additional_outcomes_prevented.add_trace(go.Scatter(
        x=selected_years + selected_years[::-1],
        y=worst_additional_outcomes + effective_additional_outcomes[::-1],
        fill='toself',
        fillcolor='rgba(0, 255, 0, 0.2)',
        line=dict(color='rgba(255, 255, 255, 0)'),
        showlegend=True,
        name='Mitigation'
    ))
    fig_additional_outcomes_prevented.add_trace(go.Scatter(
        x=selected_years + selected_years[::-1],
        y=zero_adaptation_outcomes + effective_additional_outcomes[::-1],
        fill='toself',
        fillcolor='rgba(0, 0, 255, 0.2)',
        line=dict(color='rgba(255, 255, 255, 0)'),
        showlegend=True,
        name='Adaptation'
    ))
    
    fig_additional_outcomes_prevented.update_layout(
        title=f"Additional {outcome} Outcomes Prevented by Mitigation and Adaptation from {time_period[0]} to {time_period[1]}",
        xaxis_title="Year",
        yaxis=dict(
            title="Additional Outcomes",
            titlefont=dict(color="#1f77b4"),
            tickfont=dict(color="#1f77b4")
        )
    )
    
    fig_costs = go.Figure()
    
    # Add costs
    fig_costs.add_trace(go.Scatter(x=selected_years, y=costs, mode='lines', name='Costs'))
    
    fig_costs.update_layout(
        title=f"Costs Associated with Effective Additional {outcome} Outcomes Due to Heat from {time_period[0]} to {time_period[1]}",
        xaxis_title="Year",
        yaxis=dict(
            title="Costs (USD)",
            titlefont=dict(color="#1f77b4"),
            tickfont=dict(color="#1f77b4")
        )
    )
    
    fig_cost_savings = go.Figure()
    
    # Add worst-case scenario costs
    fig_cost_savings.add_trace(go.Scatter(x=selected_years, y=worst_costs, mode='lines', name='Worst Case Costs (RCP 8.5, 0% Adaptation)'))
    
    # Add selected scenario costs
    fig_cost_savings.add_trace(go.Scatter(x=selected_years, y=costs, mode='lines', name=f'Selected Scenario Costs ({rcp_scenario}, {adaptation_effectiveness}% Adaptation)'))
    
    # Shade the area between the graphs to represent mitigation and adaptation
    fig_cost_savings.add_trace(go.Scatter(
        x=selected_years + selected_years[::-1],
        y=worst_costs + costs[::-1],
        fill='toself',
        fillcolor='rgba(0, 255, 0, 0.2)',
        line=dict(color='rgba(255, 255, 255, 0)'),
        showlegend=True,
        name='Mitigation'
    ))
    fig_cost_savings.add_trace(go.Scatter(
        x=selected_years + selected_years[::-1],
        y=zero_adaptation_costs + costs[::-1],
        fill='toself',
        fillcolor='rgba(0, 0, 255, 0.2)',
        line=dict(color='rgba(255, 255, 255, 0)'),
        showlegend=True,
        name='Adaptation'
    ))
    
    fig_cost_savings.update_layout(
        title=f"Cost Savings by Comparing Worst Scenario (RCP 8.5, 0% Adaptation) and Selected Scenario from {time_period[0]} to {time_period[1]}",
        xaxis_title="Year",
        yaxis=dict(
            title="Costs (USD)",
            titlefont=dict(color="#1f77b4"),
            tickfont=dict(color="#1f77b4")
        )
    )
    
    additional_outcome_text = f"Odds Ratio per 1°C increase: {mnch_outcomes[outcome]['odds_ratio']}<br>Current Rate: {mnch_outcomes[outcome]['current_rate']}<br>Initial Pregnancy Rate: {initial_pregnancy_rate}<br>Adaptation Effectiveness: {adaptation_effectiveness}%<br>Cost per Case: ${mnch_outcomes[outcome]['cost_per_case']}"
    
    total_outcomes_text = f"Total Effective Additional {outcome} Outcomes from {time_period[0]} to {time_period[1]}: {int(round(total_additional_outcomes))}<br>Total Cost: ${round(total_cost, 2):,}<br>Total Mitigation Savings: ${round(total_mitigation_savings, 2):,}<br>Total Adaptation Savings: ${round(total_adaptation_savings, 2):,}<br>Total Mitigation Prevented: {int(round(total_mitigation_prevented))}<br>Total Adaptation Prevented: {int(round(total_adaptation_prevented))}"
    
    return display_text, fig_outcomes, fig_additional_outcomes_prevented, fig_costs, fig_cost_savings, total_outcomes_text

# Export Data Callback
@app.callback(
    Output("download-dataframe-csv", "data"),
    Input("export-button", "n_clicks"),
    prevent_initial_call=True,
)
def export_data(n_clicks):
    if n_clicks is None:
        return dash.no_update

    # Retrieve values from the update_dashboard function (need to be moved to global scope)
    outcome = 'Preterm Birth'  # or get dynamically if possible
    rcp_scenario = 'RCP 2.6'  # or get dynamically if possible
    population_growth_curve = 'Medium Growth'  # or get dynamically if possible
    pregnancy_growth_curve = 'Medium Growth'  # or get dynamically if possible
    time_period = [2024, 2030]  # or get dynamically if possible
    current_population = 7000000000  # or get dynamically if possible
    initial_pregnancy_rate = 0.02  # or get dynamically if possible
    adaptation_effectiveness = 50  # or get dynamically if possible
    
    selected_years, effective_additional_outcomes, costs, additional_outcomes = calculate_additional_outcomes_and_costs(
        outcome, rcp_scenario, population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, adaptation_effectiveness
    )
    
    _, worst_effective_additional_outcomes, worst_costs, worst_additional_outcomes = calculate_additional_outcomes_and_costs(
        outcome, 'RCP 8.5', population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, 0
    )
    
    _, zero_adaptation_outcomes, zero_adaptation_costs, _ = calculate_additional_outcomes_and_costs(
        outcome, rcp_scenario, population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, 0
    )
    
    mitigation_savings = [worst - selected for worst, selected in zip(worst_costs, costs)]
    adaptation_savings = [zero - selected for zero, selected in zip(zero_adaptation_costs, costs)]
    mitigation_prevented = [worst - selected for worst, selected in zip(worst_additional_outcomes, effective_additional_outcomes)]
    adaptation_prevented = [zero - selected for zero, selected in zip(zero_adaptation_outcomes, effective_additional_outcomes)]

    # Create a DataFrame with the current projections
    data = {
        'Year': selected_years,
        'Effective Additional Outcomes': effective_additional_outcomes,
        'Temperature Increase': calculate_temperature_increase_per_year(rcp_scenarios[rcp_scenario], time_period[0], time_period[1]),
        'Costs': costs,
        'Worst Case Costs': worst_costs,
        'Zero Adaptation Costs': zero_adaptation_costs,
        'Mitigation Savings': mitigation_savings,
        'Adaptation Savings': adaptation_savings,
        'Mitigation Prevented': mitigation_prevented,
        'Adaptation Prevented': adaptation_prevented
    }
    df = pd.DataFrame(data)

    return dcc.send_data_frame(df.to_csv, "projected_outcomes.csv")

# Step 5: Run the App
if __name__ == '__main__':
    app.run_server(debug=True)



In [29]:
#version 11 with improved layout
# Import necessary packages
import dash
from dash import dcc, html, Input, Output
import plotly.graph_objs as go
import pandas as pd
import numpy as np

# Step 2: Prepare the Data
mnch_outcomes = {
    'Preterm Birth': {'odds_ratio': 1.05, 'current_rate': 0.1, 'cost_per_case': 1000},
    'Low Birth Weight': {'odds_ratio': 1.03, 'current_rate': 0.08, 'cost_per_case': 800},
    'Stillbirth': {'odds_ratio': 1.04, 'current_rate': 0.005, 'cost_per_case': 5000}
}

rcp_scenarios = {
    'RCP 2.6': 1.5,  # degrees Celsius increase by 2100
    'RCP 4.5': 2.4,
    'RCP 6.0': 3.0,
    'RCP 8.5': 4.3
}

population_growth_rates = {
    'Low Growth': 0.01,  # 1% annual growth
    'Medium Growth': 0.02,  # 2% annual growth
    'High Growth': 0.03  # 3% annual growth
}

pregnancy_growth_rates = {
    'High Reduction': -0.15,  # 15% reduction
    'Medium Reduction': -0.1,  # 10% reduction
    'Low Reduction': -0.05,  # 5% reduction
    'Low Growth': 0.01,  # 1% annual growth
    'Medium Growth': 0.02,  # 2% annual growth
    'High Growth': 0.03  # 3% annual growth
}

years = list(range(2024, 2051))

def calculate_population_growth(initial_population, growth_rate, num_years):
    return [initial_population * ((1 + growth_rate) ** year) for year in range(num_years)]

def calculate_pregnancies(population, pregnancy_rate):
    return [pop * pregnancy_rate for pop in population]

def calculate_temperature_increase_per_year(temp_increase_2100, start_year, end_year):
    total_years = 2100 - 2024  # Calculate total years from 2024 to 2100
    annual_increase = temp_increase_2100 / total_years
    return [annual_increase * (year - 2024) for year in range(start_year, end_year + 1)]

def calculate_additional_outcomes_and_costs(outcome, rcp_scenario, population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, adaptation_effectiveness):
    odds_ratio = mnch_outcomes[outcome]['odds_ratio']
    current_rate = mnch_outcomes[outcome]['current_rate']
    cost_per_case = mnch_outcomes[outcome]['cost_per_case']
    temp_increase_2100 = rcp_scenarios[rcp_scenario]
    population_growth_rate = population_growth_rates[population_growth_curve]
    pregnancy_growth_rate = pregnancy_growth_rates[pregnancy_growth_curve]
    
    start_year, end_year = time_period
    num_years = end_year - start_year + 1
    selected_years = list(range(start_year, end_year + 1))
    
    # Calculate population growth
    population_growth_selected = calculate_population_growth(current_population, population_growth_rate, num_years)
    
    # Calculate pregnancies without the effect of heat
    pregnancies = calculate_pregnancies(population_growth_selected, initial_pregnancy_rate * (1 + pregnancy_growth_rate))
    
    # Calculate baseline outcomes without the effect of heat
    baseline_outcomes = [preg * current_rate for preg in pregnancies]
    
    # Calculate the temperature increase per year
    temperature_increase_per_year = calculate_temperature_increase_per_year(temp_increase_2100, start_year, end_year)
    
    # Calculate projected outcomes with the effect of heat
    projected_rates = [current_rate * (odds_ratio ** temp_increase) for temp_increase in temperature_increase_per_year]
    projected_outcomes = [rate * preg for rate, preg in zip(projected_rates, pregnancies)]
    
    # Calculate additional outcomes due to the effect of heat
    additional_outcomes = [proj - base for proj, base in zip(projected_outcomes, baseline_outcomes)]
    
    # Apply adaptation effectiveness
    effective_additional_outcomes = [outcome * (1 - adaptation_effectiveness / 100) for outcome in additional_outcomes]
    
    # Calculate the total cost
    costs = [outcome * cost_per_case for outcome in effective_additional_outcomes]
    
    return selected_years, effective_additional_outcomes, costs, additional_outcomes

# Step 3: Create the Dash Layout
app = dash.Dash(__name__)

app.layout = html.Div([
    html.Link(rel='stylesheet', href='/assets/style.css'),
    html.H1("MNCH Outcomes Due to Heat"),
    html.H2("Climate Change Projection", className='subtitle'),
    html.Div(className='container', children=[
        html.Div(className='input-container', children=[
            html.Div(className='input-box', children=[
                html.H3("Select MNCH Outcome"),
                dcc.Dropdown(
                    id='mnch-outcome-dropdown',
                    options=[{'label': outcome, 'value': outcome} for outcome in mnch_outcomes.keys()],
                    value='Preterm Birth',
                    className='dropdown'
                )
            ]),
            html.Div(className='input-box', children=[
                html.H3("Select RCP Scenario"),
                dcc.Dropdown(
                    id='rcp-scenario-dropdown',
                    options=[{'label': scenario, 'value': scenario} for scenario in rcp_scenarios.keys()],
                    value='RCP 2.6',
                    className='dropdown'
                )
            ]),
            html.Div(className='input-box', children=[
                html.H3("Select Population Growth Curve"),
                dcc.Dropdown(
                    id='population-growth-dropdown',
                    options=[{'label': growth, 'value': growth} for growth in population_growth_rates.keys()],
                    value='Medium Growth',
                    className='dropdown'
                )
            ]),
            html.Div(className='input-box', children=[
                html.H3("Select Pregnancy Growth Rate"),
                dcc.Dropdown(
                    id='pregnancy-growth-dropdown',
                    options=[{'label': growth, 'value': growth} for growth in pregnancy_growth_rates.keys()],
                    value='Medium Growth',
                    className='dropdown'
                )
            ]),
            html.Div(className='input-box', children=[
                html.H3("Current Population"),
                dcc.Input(
                    id='current-population-input',
                    type='number',
                    value=7000000000,
                    className='input'
                )
            ]),
            html.Div(className='input-box', children=[
                html.H3("Initial Pregnancy Rate"),
                dcc.Input(
                    id='initial-pregnancy-rate-input',
                    type='number',
                    value=0.02,  # 2% initial pregnancy rate
                    step=0.001,
                    className='input'
                )
            ])
        ]),
        html.Div(className='slider-box', children=[
            html.H3("Adaptation Effectiveness (%)"),
            dcc.Slider(
                id='adaptation-effectiveness-slider',
                min=0,
                max=100,
                step=1,
                value=50,
                marks={i: f'{i}%' for i in range(0, 101, 10)},
                className='slider'
            )
        ]),
        html.Div(className='slider-box', children=[
            html.H3("Select Time Period"),
            dcc.RangeSlider(
                id='time-period-slider',
                min=2024,
                max=2050,
                step=1,
                marks={year: str(year) for year in years},
                value=[2024, 2030],
                className='slider'
            )
        ]),
        html.H2("Outputs", className='subtitle'),
        html.Div(className='key-indicators', children=[
            html.Div(className='key-indicator', children=[
                html.H2("Odds Ratio per 1°C increase"),
                html.P(id='odds-and-rate-display')
            ]),
            html.Div(className='key-indicator', children=[
                html.H2("Current Rate"),
                html.P(id='current-rate-display')
            ]),
            html.Div(className='key-indicator', children=[
                html.H2("Initial Pregnancy Rate"),
                html.P(id='initial-pregnancy-rate-display')
            ]),
            html.Div(className='key-indicator', children=[
                html.H2("Adaptation Effectiveness"),
                html.P(id='adaptation-effectiveness-display')
            ]),
            html.Div(className='key-indicator', children=[
                html.H2("Cost per Case"),
                html.P(id='cost-per-case-display')
            ]),
            html.Div(className='key-indicator', children=[
                html.H2("Total Additional Outcomes"),
                html.P(id='total-additional-outcomes-display')
            ]),
            html.Div(className='key-indicator', children=[
                html.H2("Total Cost"),
                html.P(id='total-cost-display')
            ]),
            html.Div(className='key-indicator', children=[
                html.H2("Total Mitigation Savings"),
                html.P(id='total-mitigation-savings-display')
            ]),
            html.Div(className='key-indicator', children=[
                html.H2("Total Adaptation Savings"),
                html.P(id='total-adaptation-savings-display')
            ]),
            html.Div(className='key-indicator', children=[
                html.H2("Total Mitigation Prevented"),
                html.P(id='total-mitigation-prevented-display')
            ]),
            html.Div(className='key-indicator', children=[
                html.H2("Total Adaptation Prevented"),
                html.P(id='total-adaptation-prevented-display')
            ])
        ]),
        html.Div(className='graph-container', children=[
            dcc.Graph(id='outcome-projection-graph'),
            dcc.Graph(id='additional-outcomes-prevented-graph'),
            dcc.Graph(id='cost-projection-graph'),
            dcc.Graph(id='cost-savings-graph')
        ]),
        html.Button("Export Data", id="export-button", className='button'),
        dcc.Download(id="download-dataframe-csv")
    ])
])

# Step 4: Add Interactivity
@app.callback(
    Output('odds-and-rate-display', 'children'),
    Output('current-rate-display', 'children'),
    Output('initial-pregnancy-rate-display', 'children'),
    Output('adaptation-effectiveness-display', 'children'),
    Output('cost-per-case-display', 'children'),
    Output('total-additional-outcomes-display', 'children'),
    Output('total-cost-display', 'children'),
    Output('total-mitigation-savings-display', 'children'),
    Output('total-adaptation-savings-display', 'children'),
    Output('total-mitigation-prevented-display', 'children'),
    Output('total-adaptation-prevented-display', 'children'),
    Output('outcome-projection-graph', 'figure'),
    Output('additional-outcomes-prevented-graph', 'figure'),
    Output('cost-projection-graph', 'figure'),
    Output('cost-savings-graph', 'figure'),
    Input('mnch-outcome-dropdown', 'value'),
    Input('rcp-scenario-dropdown', 'value'),
    Input('population-growth-dropdown', 'value'),
    Input('pregnancy-growth-dropdown', 'value'),
    Input('time-period-slider', 'value'),
    Input('current-population-input', 'value'),
    Input('initial-pregnancy-rate-input', 'value'),
    Input('adaptation-effectiveness-slider', 'value')
)
def update_dashboard(outcome, rcp_scenario, population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, adaptation_effectiveness):
    # Calculate selected scenario outcomes and costs
    selected_years, effective_additional_outcomes, costs, additional_outcomes = calculate_additional_outcomes_and_costs(
        outcome, rcp_scenario, population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, adaptation_effectiveness
    )
    
    # Calculate worst scenario (RCP 8.5, 0% adaptation effectiveness) outcomes and costs
    _, worst_effective_additional_outcomes, worst_costs, worst_additional_outcomes = calculate_additional_outcomes_and_costs(
        outcome, 'RCP 8.5', population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, 0
    )
    
    # Calculate 0% adaptation scenario outcomes and costs for comparison
    _, zero_adaptation_outcomes, zero_adaptation_costs, _ = calculate_additional_outcomes_and_costs(
        outcome, rcp_scenario, population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, 0
    )
    
    # Calculate cost savings
    mitigation_savings = [worst - selected for worst, selected in zip(worst_costs, costs)]
    adaptation_savings = [zero - selected for zero, selected in zip(zero_adaptation_costs, costs)]
    
    # Calculate additional outcomes prevented
    mitigation_prevented = [worst - selected for worst, selected in zip(worst_additional_outcomes, effective_additional_outcomes)]
    adaptation_prevented = [zero - selected for zero, selected in zip(zero_adaptation_outcomes, effective_additional_outcomes)]
    
    total_additional_outcomes = sum(effective_additional_outcomes)
    total_cost = sum(costs)
    total_mitigation_savings = sum(mitigation_savings)
    total_adaptation_savings = sum(adaptation_savings)
    total_mitigation_prevented = sum(mitigation_prevented)
    total_adaptation_prevented = sum(adaptation_prevented)
    
    display_text = f"{mnch_outcomes[outcome]['odds_ratio']}"
    current_rate_text = f"{mnch_outcomes[outcome]['current_rate']}"
    initial_pregnancy_rate_text = f"{initial_pregnancy_rate}"
    adaptation_effectiveness_text = f"{adaptation_effectiveness}%"
    cost_per_case_text = f"${mnch_outcomes[outcome]['cost_per_case']}"
    total_additional_outcomes_text = f"{int(round(total_additional_outcomes))}"
    total_cost_text = f"${round(total_cost, 2):,}"
    total_mitigation_savings_text = f"${round(total_mitigation_savings, 2):,}"
    total_adaptation_savings_text = f"${round(total_adaptation_savings, 2):,}"
    total_mitigation_prevented_text = f"{int(round(total_mitigation_prevented))}"
    total_adaptation_prevented_text = f"{int(round(total_adaptation_prevented))}"
    
    fig_outcomes = go.Figure()
    
    # Add additional outcomes
    fig_outcomes.add_trace(go.Scatter(x=selected_years, y=effective_additional_outcomes, mode='lines', name='Selected Scenario', line=dict(color='blue')))
    
    # Add temperature increase
    temperature_increase_per_year = calculate_temperature_increase_per_year(rcp_scenarios[rcp_scenario], time_period[0], time_period[1])
    fig_outcomes.add_trace(go.Scatter(x=selected_years, y=temperature_increase_per_year, mode='lines', name='Temperature Increase', yaxis='y2'))
    
    # Update layout for multiple y-axes
    fig_outcomes.update_layout(
        title=f"Effective Additional {outcome} Outcomes Due to Heat and Temperature Increase from {time_period[0]} to {time_period[1]}",
        xaxis_title="Year",
        yaxis=dict(
            title="Effective Additional Outcomes",
            titlefont=dict(color="#1f77b4"),
            tickfont=dict(color="#1f77b4")
        ),
        yaxis2=dict(
            title="Temperature Increase (°C)",
            titlefont=dict(color="#ff7f0e"),
            tickfont=dict(color="#ff7f0e"),
            overlaying='y',
            side='right'
        ),
        legend=dict(yanchor="bottom", y=-0.5, xanchor="center", x=0.5)
    )
    
    fig_additional_outcomes_prevented = go.Figure()
    
    # Add worst-case scenario additional outcomes
    fig_additional_outcomes_prevented.add_trace(go.Scatter(x=selected_years, y=worst_additional_outcomes, mode='lines', name='Worst Case', line=dict(color='red')))
    
    # Add selected scenario additional outcomes
    fig_additional_outcomes_prevented.add_trace(go.Scatter(x=selected_years, y=effective_additional_outcomes, mode='lines', name=f'Selected Scenario ({rcp_scenario})', line=dict(color='blue')))
    
    # Shade the area between the graphs to represent mitigation and adaptation
    fig_additional_outcomes_prevented.add_trace(go.Scatter(
        x=selected_years + selected_years[::-1],
        y=worst_additional_outcomes + effective_additional_outcomes[::-1],
        fill='toself',
        fillcolor='rgba(0, 255, 0, 0.2)',
        line=dict(color='rgba(255, 255, 255, 0)'),
        showlegend=True,
        name='Mitigation'
    ))
    fig_additional_outcomes_prevented.add_trace(go.Scatter(
        x=selected_years + selected_years[::-1],
        y=zero_adaptation_outcomes + effective_additional_outcomes[::-1],
        fill='toself',
        fillcolor='rgba(0, 0, 255, 0.2)',
        line=dict(color='rgba(255, 255, 255, 0)'),
        showlegend=True,
        name='Adaptation'
    ))
    
    fig_additional_outcomes_prevented.update_layout(
        title=f"Additional {outcome} Outcomes Prevented by Mitigation and Adaptation from {time_period[0]} to {time_period[1]}",
        xaxis_title="Year",
        yaxis=dict(
            title="Additional Outcomes",
            titlefont=dict(color="#1f77b4"),
            tickfont=dict(color="#1f77b4")
        ),
        legend=dict(yanchor="bottom", y=-0.5, xanchor="center", x=0.5)
    )
    
    fig_costs = go.Figure()
    
    # Add costs
    fig_costs.add_trace(go.Scatter(x=selected_years, y=costs, mode='lines', name='Costs', line=dict(color='blue')))
    
    fig_costs.update_layout(
        title=f"Costs Associated with Effective Additional {outcome} Outcomes Due to Heat from {time_period[0]} to {time_period[1]}",
        xaxis_title="Year",
        yaxis=dict(
            title="Costs (USD)",
            titlefont=dict(color="#1f77b4"),
            tickfont=dict(color="#1f77b4")
        ),
        legend=dict(yanchor="bottom", y=-0.5, xanchor="center", x=0.5)
    )
    
    fig_cost_savings = go.Figure()
    
    # Add worst-case scenario costs
    fig_cost_savings.add_trace(go.Scatter(x=selected_years, y=worst_costs, mode='lines', name='Worst Case', line=dict(color='red')))
    
    # Add selected scenario costs
    fig_cost_savings.add_trace(go.Scatter(x=selected_years, y=costs, mode='lines', name=f'Selected Scenario ({rcp_scenario})', line=dict(color='blue')))
    
    # Shade the area between the graphs to represent mitigation and adaptation
    fig_cost_savings.add_trace(go.Scatter(
        x=selected_years + selected_years[::-1],
        y=worst_costs + costs[::-1],
        fill='toself',
        fillcolor='rgba(0, 255, 0, 0.2)',
        line=dict(color='rgba(255, 255, 255, 0)'),
        showlegend=True,
        name='Mitigation'
    ))
    fig_cost_savings.add_trace(go.Scatter(
        x=selected_years + selected_years[::-1],
        y=zero_adaptation_costs + costs[::-1],
        fill='toself',
        fillcolor='rgba(0, 0, 255, 0.2)',
        line=dict(color='rgba(255, 255, 255, 0)'),
        showlegend=True,
        name='Adaptation'
    ))
    
    fig_cost_savings.update_layout(
        title=f"Cost Savings by Comparing Worst Scenario (RCP 8.5, 0% Adaptation) and Selected Scenario from {time_period[0]} to {time_period[1]}",
        xaxis_title="Year",
        yaxis=dict(
            title="Costs (USD)",
            titlefont=dict(color="#1f77b4"),
            tickfont=dict(color="#1f77b4")
        ),
        legend=dict(yanchor="bottom", y=-0.5, xanchor="center", x=0.5)
    )
    
    return display_text, current_rate_text, initial_pregnancy_rate_text, adaptation_effectiveness_text, cost_per_case_text, total_additional_outcomes_text, total_cost_text, total_mitigation_savings_text, total_adaptation_savings_text, total_mitigation_prevented_text, total_adaptation_prevented_text, fig_outcomes, fig_additional_outcomes_prevented, fig_costs, fig_cost_savings

# Export Data Callback
@app.callback(
    Output("download-dataframe-csv", "data"),
    Input("export-button", "n_clicks"),
    prevent_initial_call=True,
)
def export_data(n_clicks):
    if n_clicks is None:
        return dash.no_update

    # Retrieve values from the update_dashboard function (need to be moved to global scope)
    outcome = 'Preterm Birth'  # or get dynamically if possible
    rcp_scenario = 'RCP 2.6'  # or get dynamically if possible
    population_growth_curve = 'Medium Growth'  # or get dynamically if possible
    pregnancy_growth_curve = 'Medium Growth'  # or get dynamically if possible
    time_period = [2024, 2030]  # or get dynamically if possible
    current_population = 7000000000  # or get dynamically if possible
    initial_pregnancy_rate = 0.02  # or get dynamically if possible
    adaptation_effectiveness = 50  # or get dynamically if possible
    
    selected_years, effective_additional_outcomes, costs, additional_outcomes = calculate_additional_outcomes_and_costs(
        outcome, rcp_scenario, population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, adaptation_effectiveness
    )
    
    _, worst_effective_additional_outcomes, worst_costs, worst_additional_outcomes = calculate_additional_outcomes_and_costs(
        outcome, 'RCP 8.5', population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, 0
    )
    
    _, zero_adaptation_outcomes, zero_adaptation_costs, _ = calculate_additional_outcomes_and_costs(
        outcome, rcp_scenario, population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, 0
    )
    
    mitigation_savings = [worst - selected for worst, selected in zip(worst_costs, costs)]
    adaptation_savings = [zero - selected for zero, selected in zip(zero_adaptation_costs, costs)]
    mitigation_prevented = [worst - selected for worst, selected in zip(worst_additional_outcomes, effective_additional_outcomes)]
    adaptation_prevented = [zero - selected for zero, selected in zip(zero_adaptation_outcomes, effective_additional_outcomes)]

    # Create a DataFrame with the current projections
    data = {
        'Year': selected_years,
        'Effective Additional Outcomes': effective_additional_outcomes,
        'Temperature Increase': calculate_temperature_increase_per_year(rcp_scenarios[rcp_scenario], time_period[0], time_period[1]),
        'Costs': costs,
        'Worst Case Costs': worst_costs,
        'Zero Adaptation Costs': zero_adaptation_costs,
        'Mitigation Savings': mitigation_savings,
        'Adaptation Savings': adaptation_savings,
        'Mitigation Prevented': mitigation_prevented,
        'Adaptation Prevented': adaptation_prevented
    }
    df = pd.DataFrame(data)

    return dcc.send_data_frame(df.to_csv, "projected_outcomes.csv")

# Step 5: Run the App
if __name__ == '__main__':
    app.run_server(debug=True)


In [46]:
#version 12 with improved layout
# Import necessary packages
import dash
from dash import dcc, html, Input, Output
import plotly.graph_objs as go
import pandas as pd
import numpy as np

# Step 2: Prepare the Data
mnch_outcomes = {
    'Preterm Birth': {'odds_ratio': 1.05, 'current_rate': 0.1, 'cost_per_case': 1000},
    'Low Birth Weight': {'odds_ratio': 1.03, 'current_rate': 0.08, 'cost_per_case': 800},
    'Stillbirth': {'odds_ratio': 1.04, 'current_rate': 0.005, 'cost_per_case': 5000}
}
mnch_outcomes = {
    'Preterm Birth': {'odds_ratio': 1.04, 'current_rate': 0.1, 'cost_per_case': 1000, 'ci_lower': 1.03, 'ci_upper': 1.06},
    'Low Birth Weight': {'odds_ratio': 1.03, 'current_rate': 0.08, 'cost_per_case': 800, 'ci_lower': 1.02, 'ci_upper': 1.05},
    'Stillbirth': {'odds_ratio': 1.14, 'current_rate': 0.005, 'cost_per_case': 5000, 'ci_lower': 0.99, 'ci_upper': 1.32},
    'Congenital Anomalies': {'odds_ratio': 1.29, 'current_rate': 0.02, 'cost_per_case': 10000, 'ci_lower': 1.21, 'ci_upper': 1.38},
    'Gestational Diabetes Mellitus': {'odds_ratio': 1.28, 'current_rate': 0.05, 'cost_per_case': 2000, 'ci_lower': 1.05, 'ci_upper': 1.74},
    'Hypertensive Disorders': {'odds_ratio': 1.16, 'current_rate': 0.1, 'cost_per_case': 1500, 'ci_lower': 1.10, 'ci_upper': 1.22},
    'Prelabour Rupture of Membranes': {'odds_ratio': 1.63, 'current_rate': 0.02, 'cost_per_case': 1200, 'ci_lower': 1.23, 'ci_upper': 2.16},
    'Infections': {'odds_ratio': 1.15, 'current_rate': 0.02, 'cost_per_case': 800, 'ci_lower': 1.03, 'ci_upper': 1.40},
    'Placental Abruption': {'odds_ratio': 1.16, 'current_rate': 0.005, 'cost_per_case': 2500, 'ci_lower': 1.03, 'ci_upper': 1.40},
    'Neonatal Admissions': {'odds_ratio': 1.33, 'current_rate': 0.02, 'cost_per_case': 1500, 'ci_lower': 1.27, 'ci_upper': 1.38}
}
mnch_outcomes = {
    'Gestational Diabetes': {'odds_ratio': 1.07, 'current_rate': 0.05, 'cost_per_case': 2000, 'ci_lower': 1.01, 'ci_upper': 1.54},
    'Hypertension during Pregnancy': {'odds_ratio': 1.12, 'current_rate': 0.1, 'cost_per_case': 1500, 'ci_lower': 1.03, 'ci_upper': 1.21},
    'Maternal Admission': {'odds_ratio': 1.01, 'current_rate': 0.02, 'cost_per_case': 1000, 'ci_lower': 1.00, 'ci_upper': 1.03},
    'Infections': {'odds_ratio': 1.29, 'current_rate': 0.02, 'cost_per_case': 800, 'ci_lower': 1.05, 'ci_upper': 1.58},
    'Prelabour Rupture of Membranes': {'odds_ratio': 1.08, 'current_rate': 0.02, 'cost_per_case': 1200, 'ci_lower': 1.07, 'ci_upper': 1.09},
    'Antenatal Bleeding': {'odds_ratio': 1.16, 'current_rate': 0.005, 'cost_per_case': 2500, 'ci_lower': 1.13, 'ci_upper': 1.40},
    'Cardiovascular Event': {'odds_ratio': 1.11, 'current_rate': 0.01, 'cost_per_case': 3000, 'ci_lower': 1.06, 'ci_upper': 1.15},
    'Stillbirth': {'odds_ratio': 1.14, 'current_rate': 0.005, 'cost_per_case': 5000, 'ci_lower': 0.99, 'ci_upper': 1.32},
    'Congenital Anomalies': {'odds_ratio': 1.13, 'current_rate': 0.02, 'cost_per_case': 10000, 'ci_lower': 0.99, 'ci_upper': 1.85},
    'Spontaneous Abortion': {'odds_ratio': 1.31, 'current_rate': 0.01, 'cost_per_case': 5000, 'ci_lower': 0.99, 'ci_upper': 3.30},
    'Non-reassuring Fetal Status': {'odds_ratio': 1.21, 'current_rate': 0.02, 'cost_per_case': 800, 'ci_lower': 1.12, 'ci_upper': 1.32},
    'Composite Outcome': {'odds_ratio': 1.42, 'current_rate': 0.1, 'cost_per_case': 2000, 'ci_lower': 1.00, 'ci_upper': 2.03},
    'Preterm Birth': {'odds_ratio': 1.04, 'current_rate': 0.1, 'cost_per_case': 1000, 'ci_lower': 1.03, 'ci_upper': 1.06},
    'Low Birth Weight': {'odds_ratio': 1.13, 'current_rate': 0.08, 'cost_per_case': 800, 'ci_lower': 1.02, 'ci_upper': 1.26},
    'Small for Gestational Age': {'odds_ratio': 1.10, 'current_rate': 0.02, 'cost_per_case': 800, 'ci_lower': 1.02, 'ci_upper': 1.18},
    'Neonatal Admission': {'odds_ratio': 1.22, 'current_rate': 0.02, 'cost_per_case': 1500, 'ci_lower': 1.02, 'ci_upper': 1.43},
    'Neonatal Morbidity': {'odds_ratio': 1.04, 'current_rate': 0.02, 'cost_per_case': 800, 'ci_lower': 1.03, 'ci_upper': 1.06},
    'Obstetric Complication': {'odds_ratio': 1.05, 'current_rate': 0.1, 'cost_per_case': 1500, 'ci_lower': 1.03, 'ci_upper': 1.06}
}

rcp_scenarios = {
    'RCP 2.6': 1.5,  # degrees Celsius increase by 2100
    'RCP 4.5': 2.4,
    'RCP 6.0': 3.0,
    'RCP 8.5': 4.3
}

population_growth_rates = {
    'Low Growth': 0.01,  # 1% annual growth
    'Medium Growth': 0.02,  # 2% annual growth
    'High Growth': 0.03  # 3% annual growth
}

pregnancy_growth_rates = {
    'High Reduction': -0.15,  # 15% reduction
    'Medium Reduction': -0.1,  # 10% reduction
    'Low Reduction': -0.05,  # 5% reduction
    'Low Growth': 0.01,  # 1% annual growth
    'Medium Growth': 0.02,  # 2% annual growth
    'High Growth': 0.03  # 3% annual growth
}

years = list(range(2024, 2051))

def calculate_population_growth(initial_population, growth_rate, num_years):
    return [initial_population * ((1 + growth_rate) ** year) for year in range(num_years)]

def calculate_pregnancies(population, pregnancy_rate):
    return [pop * pregnancy_rate for pop in population]

def calculate_temperature_increase_per_year(temp_increase_2100, start_year, end_year):
    total_years = 2100 - 2024  # Calculate total years from 2024 to 2100
    annual_increase = temp_increase_2100 / total_years
    return [annual_increase * (year - 2024) for year in range(start_year, end_year + 1)]

def calculate_additional_outcomes_and_costs(outcome, rcp_scenario, population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, adaptation_effectiveness):
    odds_ratio = mnch_outcomes[outcome]['odds_ratio']
    current_rate = mnch_outcomes[outcome]['current_rate']
    cost_per_case = mnch_outcomes[outcome]['cost_per_case']
    temp_increase_2100 = rcp_scenarios[rcp_scenario]
    population_growth_rate = population_growth_rates[population_growth_curve]
    pregnancy_growth_rate = pregnancy_growth_rates[pregnancy_growth_curve]
    
    start_year, end_year = time_period
    num_years = end_year - start_year + 1
    selected_years = list(range(start_year, end_year + 1))
    
    # Calculate population growth
    population_growth_selected = calculate_population_growth(current_population, population_growth_rate, num_years)
    
    # Calculate pregnancies without the effect of heat
    pregnancies = calculate_pregnancies(population_growth_selected, initial_pregnancy_rate * (1 + pregnancy_growth_rate))
    
    # Calculate baseline outcomes without the effect of heat
    baseline_outcomes = [preg * current_rate for preg in pregnancies]
    
    # Calculate the temperature increase per year
    temperature_increase_per_year = calculate_temperature_increase_per_year(temp_increase_2100, start_year, end_year)
    
    # Calculate projected outcomes with the effect of heat
    projected_rates = [current_rate * (odds_ratio ** temp_increase) for temp_increase in temperature_increase_per_year]
    projected_outcomes = [rate * preg for rate, preg in zip(projected_rates, pregnancies)]
    
    # Calculate additional outcomes due to the effect of heat
    additional_outcomes = [proj - base for proj, base in zip(projected_outcomes, baseline_outcomes)]
    
    # Apply adaptation effectiveness correctly
    effective_additional_outcomes = [outcome * (1 - adaptation_effectiveness / 100) for outcome in additional_outcomes]
    
    # Calculate the total cost
    costs = [outcome * cost_per_case for outcome in effective_additional_outcomes]
    
    return selected_years, effective_additional_outcomes, costs, additional_outcomes

# Step 3: Create the Dash Layout
app = dash.Dash(__name__)

app.layout = html.Div([
    html.Link(rel='stylesheet', href='/assets/style.css'),
    html.H1("MNCH Outcomes Due to Heat"),
    html.H2("Climate Change Projection", className='subtitle'),
    html.Div(className='container', children=[
        html.Div(className='input-container', children=[
            html.Div(className='input-box', children=[
                html.H3("Select MNCH Outcome"),
                dcc.Dropdown(
                    id='mnch-outcome-dropdown',
                    options=[{'label': outcome, 'value': outcome} for outcome in mnch_outcomes.keys()],
                    value='Preterm Birth',
                    className='dropdown'
                )
            ]),
            html.Div(className='input-box', children=[
                html.H3("Select RCP Scenario"),
                dcc.Dropdown(
                    id='rcp-scenario-dropdown',
                    options=[{'label': scenario, 'value': scenario} for scenario in rcp_scenarios.keys()],
                    value='RCP 2.6',
                    className='dropdown'
                )
            ]),
            html.Div(className='input-box', children=[
                html.H3("Select Population Growth Curve"),
                dcc.Dropdown(
                    id='population-growth-dropdown',
                    options=[{'label': growth, 'value': growth} for growth in population_growth_rates.keys()],
                    value='Medium Growth',
                    className='dropdown'
                )
            ]),
            html.Div(className='input-box', children=[
                html.H3("Select Pregnancy Growth Rate"),
                dcc.Dropdown(
                    id='pregnancy-growth-dropdown',
                    options=[{'label': growth, 'value': growth} for growth in pregnancy_growth_rates.keys()],
                    value='Medium Growth',
                    className='dropdown'
                )
            ]),
            html.Div(className='input-box', children=[
                html.H3("Current Population"),
                dcc.Input(
                    id='current-population-input',
                    type='number',
                    value=7000000000,
                    className='input'
                )
            ]),
            html.Div(className='input-box', children=[
                html.H3("Initial Pregnancy Rate"),
                dcc.Input(
                    id='initial-pregnancy-rate-input',
                    type='number',
                    value=0.02,  # 2% initial pregnancy rate
                    step=0.001,
                    className='input'
                )
            ])
        ]),
        html.Div(className='slider-box', children=[
            html.H3("Adaptation Effectiveness (%)"),
            dcc.Slider(
                id='adaptation-effectiveness-slider',
                min=0,
                max=100,
                step=1,
                value=50,
                marks={i: f'{i}%' for i in range(0, 101, 10)},
                className='slider'
            )
        ]),
        html.Div(className='slider-box', children=[
            html.H3("Select Time Period"),
            dcc.RangeSlider(
                id='time-period-slider',
                min=2024,
                max=2050,
                step=1,
                marks={year: str(year) for year in years},
                value=[2024, 2030],
                className='slider'
            )
        ]),
        html.H2("Outputs", className='subtitle'),
        html.Div(className='key-indicators', children=[
            html.Div(className='key-indicator', children=[
                html.H2("Odds Ratio per 1°C increase"),
                html.P(id='odds-and-rate-display')
            ]),
            html.Div(className='key-indicator', children=[
                html.H2("Current Rate"),
                html.P(id='current-rate-display')
            ]),
            html.Div(className='key-indicator', children=[
                html.H2("Initial Pregnancy Rate"),
                html.P(id='initial-pregnancy-rate-display')
            ]),
            html.Div(className='key-indicator', children=[
                html.H2("Adaptation Effectiveness"),
                html.P(id='adaptation-effectiveness-display')
            ]),
            html.Div(className='key-indicator', children=[
                html.H2("Cost per Case"),
                html.P(id='cost-per-case-display')
            ]),
            html.Div(className='key-indicator', children=[
                html.H2("Total Additional Outcomes"),
                html.P(id='total-additional-outcomes-display')
            ]),
            html.Div(className='key-indicator', children=[
                html.H2("Total Cost"),
                html.P(id='total-cost-display')
            ]),
            html.Div(className='key-indicator', children=[
                html.H2("Total Mitigation Savings"),
                html.P(id='total-mitigation-savings-display')
            ]),
            html.Div(className='key-indicator', children=[
                html.H2("Total Adaptation Savings"),
                html.P(id='total-adaptation-savings-display')
            ]),
            html.Div(className='key-indicator', children=[
                html.H2("Total Mitigation Prevented"),
                html.P(id='total-mitigation-prevented-display')
            ]),
            html.Div(className='key-indicator', children=[
                html.H2("Total Adaptation Prevented"),
                html.P(id='total-adaptation-prevented-display')
            ])
        ]),
        html.Div(className='graph-container', children=[
            dcc.Graph(id='outcome-projection-graph'),
            dcc.Graph(id='additional-outcomes-prevented-graph'),
            dcc.Graph(id='cost-projection-graph'),
            dcc.Graph(id='cost-savings-graph')
        ]),
        html.Button("Export Data", id="export-button", className='button'),
        dcc.Download(id="download-dataframe-csv")
    ])
])

# Step 4: Add Interactivity
@app.callback(
    Output('odds-and-rate-display', 'children'),
    Output('current-rate-display', 'children'),
    Output('initial-pregnancy-rate-display', 'children'),
    Output('adaptation-effectiveness-display', 'children'),
    Output('cost-per-case-display', 'children'),
    Output('total-additional-outcomes-display', 'children'),
    Output('total-cost-display', 'children'),
    Output('total-mitigation-savings-display', 'children'),
    Output('total-adaptation-savings-display', 'children'),
    Output('total-mitigation-prevented-display', 'children'),
    Output('total-adaptation-prevented-display', 'children'),
    Output('outcome-projection-graph', 'figure'),
    Output('additional-outcomes-prevented-graph', 'figure'),
    Output('cost-projection-graph', 'figure'),
    Output('cost-savings-graph', 'figure'),
    Input('mnch-outcome-dropdown', 'value'),
    Input('rcp-scenario-dropdown', 'value'),
    Input('population-growth-dropdown', 'value'),
    Input('pregnancy-growth-dropdown', 'value'),
    Input('time-period-slider', 'value'),
    Input('current-population-input', 'value'),
    Input('initial-pregnancy-rate-input', 'value'),
    Input('adaptation-effectiveness-slider', 'value')
)
def update_dashboard(outcome, rcp_scenario, population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, adaptation_effectiveness):
    # Calculate selected scenario outcomes and costs
    selected_years, effective_additional_outcomes, costs, additional_outcomes = calculate_additional_outcomes_and_costs(
        outcome, rcp_scenario, population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, adaptation_effectiveness
    )
    
    # Calculate worst scenario (RCP 8.5, 0% adaptation effectiveness) outcomes and costs
    _, worst_effective_additional_outcomes, worst_costs, worst_additional_outcomes = calculate_additional_outcomes_and_costs(
        outcome, 'RCP 8.5', population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, 0
    )
    
    # Calculate 0% adaptation scenario outcomes and costs for comparison
    _, zero_adaptation_outcomes, zero_adaptation_costs, _ = calculate_additional_outcomes_and_costs(
        outcome, rcp_scenario, population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, 0
    )
    
    # Calculate cost savings
    mitigation_savings = [worst - selected for worst, selected in zip(worst_costs, costs)]
    adaptation_savings = [zero - selected for zero, selected in zip(zero_adaptation_costs, costs)]
    
    # Calculate additional outcomes prevented
    mitigation_prevented = [worst - selected for worst, selected in zip(worst_additional_outcomes, effective_additional_outcomes)]
    adaptation_prevented = [zero - selected for zero, selected in zip(zero_adaptation_outcomes, effective_additional_outcomes)]
    
    total_additional_outcomes = sum(effective_additional_outcomes)
    total_cost = sum(costs)
    total_mitigation_savings = sum(mitigation_savings)
    total_adaptation_savings = sum(adaptation_savings)
    total_mitigation_prevented = sum(mitigation_prevented)
    total_adaptation_prevented = sum(adaptation_prevented)
    
    display_text = f"{mnch_outcomes[outcome]['odds_ratio']} (95% CI: {mnch_outcomes[outcome]['ci_lower']} - {mnch_outcomes[outcome]['ci_upper']})"
    current_rate_text = f"{mnch_outcomes[outcome]['current_rate']}"
    initial_pregnancy_rate_text = f"{initial_pregnancy_rate}"
    adaptation_effectiveness_text = f"{adaptation_effectiveness}%"
    cost_per_case_text = f"${mnch_outcomes[outcome]['cost_per_case']}"
    total_additional_outcomes_text = f"{int(round(total_additional_outcomes))}"
    total_cost_text = f"${round(total_cost, 2):,}"
    total_mitigation_savings_text = f"${round(total_mitigation_savings, 2):,}"
    total_adaptation_savings_text = f"${round(total_adaptation_savings, 2):,}"
    total_mitigation_prevented_text = f"{int(round(total_mitigation_prevented))}"
    total_adaptation_prevented_text = f"{int(round(total_adaptation_prevented))}"
    
    fig_outcomes = go.Figure()
    
    # Add additional outcomes
    fig_outcomes.add_trace(go.Scatter(x=selected_years, y=effective_additional_outcomes, mode='lines', name=f'Selected Scenario ({rcp_scenario}, {adaptation_effectiveness}%)', line=dict(color='blue')))
    
    # Add temperature increase
    temperature_increase_per_year = calculate_temperature_increase_per_year(rcp_scenarios[rcp_scenario], time_period[0], time_period[1])
    fig_outcomes.add_trace(go.Scatter(x=selected_years, y=temperature_increase_per_year, mode='lines', name='Temperature Increase', yaxis='y2',line=dict(color='red')))
    
    # Update layout for multiple y-axes
    fig_outcomes.update_layout(
        title=f"Effective Additional {outcome} Outcomes Due to Heat and Temperature Increase from {time_period[0]} to {time_period[1]}",
        xaxis_title="Year",
        yaxis=dict(
            title="Effective Additional Outcomes",
            titlefont=dict(color="#1f77b4"),
            tickfont=dict(color="#1f77b4")
        ),
        yaxis2=dict(
            title="Temperature Increase (°C)",
            titlefont=dict(color="#ff7f0e"),
            tickfont=dict(color="#ff7f0e"),
            overlaying='y',
            side='right'
        ),
        legend=dict(yanchor="bottom", y=-0.6, xanchor="center", x=0.5)
    )
    
    fig_additional_outcomes_prevented = go.Figure()
    
    # Add worst-case scenario additional outcomes
    fig_additional_outcomes_prevented.add_trace(go.Scatter(x=selected_years, y=worst_additional_outcomes, mode='lines', name='Worst Case (RCP 8.5, 0% Adaptation)', line=dict(color='red')))
    
    # Add selected scenario additional outcomes
    fig_additional_outcomes_prevented.add_trace(go.Scatter(x=selected_years, y=effective_additional_outcomes, mode='lines', name=f'Selected Scenario ({rcp_scenario}, {adaptation_effectiveness}%)', line=dict(color='blue')))
    
    # Shade the area between the graphs to represent mitigation and adaptation
    fig_additional_outcomes_prevented.add_trace(go.Scatter(
        x=selected_years + selected_years[::-1],
        y=worst_additional_outcomes + effective_additional_outcomes[::-1],
        fill='toself',
        fillcolor='rgba(0, 255, 0, 0.2)',
        line=dict(color='rgba(255, 255, 255, 0)'),
        showlegend=True,
        name='Mitigation'
    ))
    fig_additional_outcomes_prevented.add_trace(go.Scatter(
        x=selected_years + selected_years[::-1],
        y=zero_adaptation_outcomes + effective_additional_outcomes[::-1],
        fill='toself',
        fillcolor='rgba(0, 0, 255, 0.2)',
        line=dict(color='rgba(255, 255, 255, 0)'),
        showlegend=True,
        name='Adaptation'
    ))
    
    fig_additional_outcomes_prevented.update_layout(
        title=f"Additional {outcome} Outcomes Prevented by Mitigation and Adaptation from {time_period[0]} to {time_period[1]}",
        xaxis_title="Year",
        yaxis=dict(
            title="Additional Outcomes",
            titlefont=dict(color="#1f77b4"),
            tickfont=dict(color="#1f77b4")
        ),
        legend=dict(yanchor="bottom", y=-0.5, xanchor="center", x=0.5)
    )
    
    fig_costs = go.Figure()
    
    # Add costs
    fig_costs.add_trace(go.Scatter(x=selected_years, y=costs, mode='lines', name=f'Selected Scenario ({rcp_scenario}, {adaptation_effectiveness}%)', line=dict(color='blue')))
    
    fig_costs.update_layout(
        title=f"Costs Associated with Effective Additional {outcome} Outcomes Due to Heat from {time_period[0]} to {time_period[1]}",
        xaxis_title="Year",
        yaxis=dict(
            title="Costs (USD)",
            titlefont=dict(color="#1f77b4"),
            tickfont=dict(color="#1f77b4")
        ),
        legend=dict(yanchor="bottom", y=-0.5, xanchor="center", x=0.5)
    )
    
    fig_cost_savings = go.Figure()
    
    # Add worst-case scenario costs
    fig_cost_savings.add_trace(go.Scatter(x=selected_years, y=worst_costs, mode='lines', name='Worst Case (RCP 8.5, 0% Adaptation)', line=dict(color='red')))
    
    # Add selected scenario costs
    fig_cost_savings.add_trace(go.Scatter(x=selected_years, y=costs, mode='lines', name=f'Selected Scenario ({rcp_scenario}, {adaptation_effectiveness}%)', line=dict(color='blue')))
    
    # Shade the area between the graphs to represent mitigation and adaptation
    fig_cost_savings.add_trace(go.Scatter(
        x=selected_years + selected_years[::-1],
        y=worst_costs + costs[::-1],
        fill='toself',
        fillcolor='rgba(0, 255, 0, 0.2)',
        line=dict(color='rgba(255, 255, 255, 0)'),
        showlegend=True,
        name='Mitigation'
    ))
    fig_cost_savings.add_trace(go.Scatter(
        x=selected_years + selected_years[::-1],
        y=zero_adaptation_costs + costs[::-1],
        fill='toself',
        fillcolor='rgba(0, 0, 255, 0.2)',
        line=dict(color='rgba(255, 255, 255, 0)'),
        showlegend=True,
        name='Adaptation'
    ))
    
    fig_cost_savings.update_layout(
        title=f"Cost Savings by Comparing Worst Scenario (RCP 8.5, 0% Adaptation) and Selected Scenario from {time_period[0]} to {time_period[1]}",
        xaxis_title="Year",
        yaxis=dict(
            title="Costs (USD)",
            titlefont=dict(color="#1f77b4"),
            tickfont=dict(color="#1f77b4")
        ),
        legend=dict(yanchor="bottom", y=-0.5, xanchor="center", x=0.5)
    )
    
    return display_text, current_rate_text, initial_pregnancy_rate_text, adaptation_effectiveness_text, cost_per_case_text, total_additional_outcomes_text, total_cost_text, total_mitigation_savings_text, total_adaptation_savings_text, total_mitigation_prevented_text, total_adaptation_prevented_text, fig_outcomes, fig_additional_outcomes_prevented, fig_costs, fig_cost_savings

# Export Data Callback
@app.callback(
    Output("download-dataframe-csv", "data"),
    Input("export-button", "n_clicks"),
    prevent_initial_call=True,
)
def export_data(n_clicks):
    if n_clicks is None:
        return dash.no_update

    # Retrieve values from the update_dashboard function (need to be moved to global scope)
    outcome = 'Preterm Birth'  # or get dynamically if possible
    rcp_scenario = 'RCP 2.6'  # or get dynamically if possible
    population_growth_curve = 'Medium Growth'  # or get dynamically if possible
    pregnancy_growth_curve = 'Medium Growth'  # or get dynamically if possible
    time_period = [2024, 2030]  # or get dynamically if possible
    current_population = 7000000000  # or get dynamically if possible
    initial_pregnancy_rate = 0.02  # or get dynamically if possible
    adaptation_effectiveness = 50  # or get dynamically if possible
    
    selected_years, effective_additional_outcomes, costs, additional_outcomes = calculate_additional_outcomes_and_costs(
        outcome, rcp_scenario, population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, adaptation_effectiveness
    )
    
    _, worst_effective_additional_outcomes, worst_costs, worst_additional_outcomes = calculate_additional_outcomes_and_costs(
        outcome, 'RCP 8.5', population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, 0
    )
    
    _, zero_adaptation_outcomes, zero_adaptation_costs, _ = calculate_additional_outcomes_and_costs(
        outcome, rcp_scenario, population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, 0
    )
    
    mitigation_savings = [worst - selected for worst, selected in zip(worst_costs, costs)]
    adaptation_savings = [zero - selected for zero, selected in zip(zero_adaptation_costs, costs)]
    mitigation_prevented = [worst - selected for worst, selected in zip(worst_additional_outcomes, effective_additional_outcomes)]
    adaptation_prevented = [zero - selected for zero, selected in zip(zero_adaptation_outcomes, effective_additional_outcomes)]

    # Create a DataFrame with the current projections
    data = {
        'Year': selected_years,
        'Effective Additional Outcomes': effective_additional_outcomes,
        'Temperature Increase': calculate_temperature_increase_per_year(rcp_scenarios[rcp_scenario], time_period[0], time_period[1]),
        'Costs': costs,
        'Worst Case Costs': worst_costs,
        'Zero Adaptation Costs': zero_adaptation_costs,
        'Mitigation Savings': mitigation_savings,
        'Adaptation Savings': adaptation_savings,
        'Mitigation Prevented': mitigation_prevented,
        'Adaptation Prevented': adaptation_prevented
    }
    df = pd.DataFrame(data)

    return dcc.send_data_frame(df.to_csv, "projected_outcomes.csv")

# Step 5: Run the App
if __name__ == '__main__':
    app.run_server(debug=True)


In [47]:
#version 13 with attributable fraction
# Import necessary packages
import dash
from dash import dcc, html, Input, Output
import plotly.graph_objs as go
import pandas as pd
import numpy as np

# Step 2: Prepare the Data
mnch_outcomes = {
    'Preterm Birth': {'odds_ratio': 1.04, 'current_rate': 0.1, 'cost_per_case': 1000, 'ci_lower': 1.03, 'ci_upper': 1.06},
    'Low Birth Weight': {'odds_ratio': 1.03, 'current_rate': 0.08, 'cost_per_case': 800, 'ci_lower': 1.02, 'ci_upper': 1.05},
    'Stillbirth': {'odds_ratio': 1.14, 'current_rate': 0.005, 'cost_per_case': 5000, 'ci_lower': 0.99, 'ci_upper': 1.32},
    'Congenital Anomalies': {'odds_ratio': 1.29, 'current_rate': 0.02, 'cost_per_case': 10000, 'ci_lower': 1.21, 'ci_upper': 1.38},
    'Gestational Diabetes Mellitus': {'odds_ratio': 1.28, 'current_rate': 0.05, 'cost_per_case': 2000, 'ci_lower': 1.05, 'ci_upper': 1.74},
    'Hypertensive Disorders': {'odds_ratio': 1.16, 'current_rate': 0.1, 'cost_per_case': 1500, 'ci_lower': 1.10, 'ci_upper': 1.22},
    'Prelabour Rupture of Membranes': {'odds_ratio': 1.63, 'current_rate': 0.02, 'cost_per_case': 1200, 'ci_lower': 1.23, 'ci_upper': 2.16},
    'Infections': {'odds_ratio': 1.15, 'current_rate': 0.02, 'cost_per_case': 800, 'ci_lower': 1.03, 'ci_upper': 1.40},
    'Placental Abruption': {'odds_ratio': 1.16, 'current_rate': 0.005, 'cost_per_case': 2500, 'ci_lower': 1.03, 'ci_upper': 1.40},
    'Neonatal Admissions': {'odds_ratio': 1.33, 'current_rate': 0.02, 'cost_per_case': 1500, 'ci_lower': 1.27, 'ci_upper': 1.38}
}

rcp_scenarios = {
    'RCP 2.6': 1.5,  # degrees Celsius increase by 2100
    'RCP 4.5': 2.4,
    'RCP 6.0': 3.0,
    'RCP 8.5': 4.3
}

population_growth_rates = {
    'Low Growth': 0.01,  # 1% annual growth
    'Medium Growth': 0.02,  # 2% annual growth
    'High Growth': 0.03  # 3% annual growth
}

pregnancy_growth_rates = {
    'High Reduction': -0.15,  # 15% reduction
    'Medium Reduction': -0.1,  # 10% reduction
    'Low Reduction': -0.05,  # 5% reduction
    'Low Growth': 0.01,  # 1% annual growth
    'Medium Growth': 0.02,  # 2% annual growth
    'High Growth': 0.03  # 3% annual growth
}

years = list(range(2024, 2051))

def calculate_population_growth(initial_population, growth_rate, num_years):
    return [initial_population * ((1 + growth_rate) ** year) for year in range(num_years)]

def calculate_pregnancies(population, pregnancy_rate):
    return [pop * pregnancy_rate for pop in population]

def calculate_temperature_increase_per_year(temp_increase_2100, start_year, end_year):
    total_years = 2100 - 2024  # Calculate total years from 2024 to 2100
    annual_increase = temp_increase_2100 / total_years
    return [annual_increase * (year - 2024) for year in range(start_year, end_year + 1)]

def calculate_additional_outcomes_and_costs(outcome, rcp_scenario, population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, adaptation_effectiveness):
    odds_ratio = mnch_outcomes[outcome]['odds_ratio']
    current_rate = mnch_outcomes[outcome]['current_rate']
    cost_per_case = mnch_outcomes[outcome]['cost_per_case']
    temp_increase_2100 = rcp_scenarios[rcp_scenario]
    population_growth_rate = population_growth_rates[population_growth_curve]
    pregnancy_growth_rate = pregnancy_growth_rates[pregnancy_growth_curve]
    
    start_year, end_year = time_period
    num_years = end_year - start_year + 1
    selected_years = list(range(start_year, end_year + 1))
    
    # Calculate population growth
    population_growth_selected = calculate_population_growth(current_population, population_growth_rate, num_years)
    
    # Calculate pregnancies without the effect of heat
    pregnancies = calculate_pregnancies(population_growth_selected, initial_pregnancy_rate * (1 + pregnancy_growth_rate))
    
    # Calculate baseline outcomes without the effect of heat
    baseline_outcomes = [preg * current_rate for preg in pregnancies]
    
    # Calculate the temperature increase per year
    temperature_increase_per_year = calculate_temperature_increase_per_year(temp_increase_2100, start_year, end_year)
    
    # Calculate projected outcomes with the effect of heat
    projected_rates = [current_rate * (odds_ratio ** temp_increase) for temp_increase in temperature_increase_per_year]
    projected_outcomes = [rate * preg for rate, preg in zip(projected_rates, pregnancies)]
    
    # Calculate additional outcomes due to the effect of heat
    additional_outcomes = [proj - base for proj, base in zip(projected_outcomes, baseline_outcomes)]
    
    # Apply adaptation effectiveness
    effective_additional_outcomes = [outcome * (1 - adaptation_effectiveness / 100) for outcome in additional_outcomes]
    
    # Calculate the total cost
    costs = [outcome * cost_per_case for outcome in effective_additional_outcomes]
    
    return selected_years, effective_additional_outcomes, costs, additional_outcomes

# Calculate Attributable Fraction
def calculate_attributable_fraction(odds_ratio):
    return (odds_ratio - 1) / odds_ratio

# Step 3: Create the Dash Layout
app = dash.Dash(__name__)

app.layout = html.Div([
    html.Link(rel='stylesheet', href='/assets/style.css'),
    html.H1("MNCH Outcomes Due to Heat"),
    html.H2("Climate Change Projection", className='subtitle'),
    html.Div(className='container', children=[
        html.Div(className='input-container', children=[
            html.Div(className='input-box', children=[
                html.H3("Select MNCH Outcome"),
                dcc.Dropdown(
                    id='mnch-outcome-dropdown',
                    options=[{'label': outcome, 'value': outcome} for outcome in mnch_outcomes.keys()],
                    value='Preterm Birth',
                    className='dropdown'
                )
            ]),
            html.Div(className='input-box', children=[
                html.H3("Select RCP Scenario"),
                dcc.Dropdown(
                    id='rcp-scenario-dropdown',
                    options=[{'label': scenario, 'value': scenario} for scenario in rcp_scenarios.keys()],
                    value='RCP 2.6',
                    className='dropdown'
                )
            ]),
            html.Div(className='input-box', children=[
                html.H3("Select Population Growth Curve"),
                dcc.Dropdown(
                    id='population-growth-dropdown',
                    options=[{'label': growth, 'value': growth} for growth in population_growth_rates.keys()],
                    value='Medium Growth',
                    className='dropdown'
                )
            ]),
            html.Div(className='input-box', children=[
                html.H3("Select Pregnancy Growth Rate"),
                dcc.Dropdown(
                    id='pregnancy-growth-dropdown',
                    options=[{'label': growth, 'value': growth} for growth in pregnancy_growth_rates.keys()],
                    value='Medium Growth',
                    className='dropdown'
                )
            ]),
            html.Div(className='input-box', children=[
                html.H3("Current Population"),
                dcc.Input(
                    id='current-population-input',
                    type='number',
                    value=7000000000,
                    className='input'
                )
            ]),
            html.Div(className='input-box', children=[
                html.H3("Initial Pregnancy Rate"),
                dcc.Input(
                    id='initial-pregnancy-rate-input',
                    type='number',
                    value=0.02,  # 2% initial pregnancy rate
                    step=0.001,
                    className='input'
                )
            ])
        ]),
        html.Div(className='slider-box', children=[
            html.H3("Adaptation Effectiveness (%)"),
            dcc.Slider(
                id='adaptation-effectiveness-slider',
                min=0,
                max=100,
                step=1,
                value=50,
                marks={i: f'{i}%' for i in range(0, 101, 10)},
                className='slider'
            )
        ]),
        html.Div(className='slider-box', children=[
            html.H3("Select Time Period"),
            dcc.RangeSlider(
                id='time-period-slider',
                min=2024,
                max=2050,
                step=1,
                marks={year: str(year) for year in years},
                value=[2024, 2030],
                className='slider'
            )
        ]),
        html.H2("Outputs", className='subtitle'),
        html.Div(className='key-indicators', children=[
            html.Div(className='key-indicator', children=[
                html.H2("Odds Ratio per 1°C increase"),
                html.P(id='odds-and-rate-display')
            ]),
            html.Div(className='key-indicator', children=[
                html.H2("Current Rate"),
                html.P(id='current-rate-display')
            ]),
            html.Div(className='key-indicator', children=[
                html.H2("Initial Pregnancy Rate"),
                html.P(id='initial-pregnancy-rate-display')
            ]),
            html.Div(className='key-indicator', children=[
                html.H2("Adaptation Effectiveness"),
                html.P(id='adaptation-effectiveness-display')
            ]),
            html.Div(className='key-indicator', children=[
                html.H2("Cost per Case"),
                html.P(id='cost-per-case-display')
            ]),
            html.Div(className='key-indicator', children=[
                html.H2("Attributable Fraction"),
                html.P(id='attributable-fraction-display')
            ]),
            html.Div(className='key-indicator', children=[
                html.H2("Total Additional Outcomes"),
                html.P(id='total-additional-outcomes-display')
            ]),
            html.Div(className='key-indicator', children=[
                html.H2("Total Cost"),
                html.P(id='total-cost-display')
            ]),
            html.Div(className='key-indicator', children=[
                html.H2("Total Mitigation Savings"),
                html.P(id='total-mitigation-savings-display')
            ]),
            html.Div(className='key-indicator', children=[
                html.H2("Total Adaptation Savings"),
                html.P(id='total-adaptation-savings-display')
            ]),
            html.Div(className='key-indicator', children=[
                html.H2("Total Mitigation Prevented"),
                html.P(id='total-mitigation-prevented-display')
            ]),
            html.Div(className='key-indicator', children=[
                html.H2("Total Adaptation Prevented"),
                html.P(id='total-adaptation-prevented-display')
            ])
        ]),
        html.Div(className='graph-container', children=[
            dcc.Graph(id='outcome-projection-graph'),
            dcc.Graph(id='additional-outcomes-prevented-graph'),
            dcc.Graph(id='cost-projection-graph'),
            dcc.Graph(id='cost-savings-graph')
        ]),
        html.Button("Export Data", id="export-button", className='button'),
        dcc.Download(id="download-dataframe-csv")
    ])
])

# Step 4: Add Interactivity
@app.callback(
    Output('odds-and-rate-display', 'children'),
    Output('current-rate-display', 'children'),
    Output('initial-pregnancy-rate-display', 'children'),
    Output('adaptation-effectiveness-display', 'children'),
    Output('cost-per-case-display', 'children'),
    Output('attributable-fraction-display', 'children'),
    Output('total-additional-outcomes-display', 'children'),
    Output('total-cost-display', 'children'),
    Output('total-mitigation-savings-display', 'children'),
    Output('total-adaptation-savings-display', 'children'),
    Output('total-mitigation-prevented-display', 'children'),
    Output('total-adaptation-prevented-display', 'children'),
    Output('outcome-projection-graph', 'figure'),
    Output('additional-outcomes-prevented-graph', 'figure'),
    Output('cost-projection-graph', 'figure'),
    Output('cost-savings-graph', 'figure'),
    Input('mnch-outcome-dropdown', 'value'),
    Input('rcp-scenario-dropdown', 'value'),
    Input('population-growth-dropdown', 'value'),
    Input('pregnancy-growth-dropdown', 'value'),
    Input('time-period-slider', 'value'),
    Input('current-population-input', 'value'),
    Input('initial-pregnancy-rate-input', 'value'),
    Input('adaptation-effectiveness-slider', 'value')
)
def update_dashboard(outcome, rcp_scenario, population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, adaptation_effectiveness):
    odds_ratio = mnch_outcomes[outcome]['odds_ratio']
    
    # Calculate Attributable Fraction
    attributable_fraction = calculate_attributable_fraction(odds_ratio)
    
    # Calculate selected scenario outcomes and costs
    selected_years, effective_additional_outcomes, costs, additional_outcomes = calculate_additional_outcomes_and_costs(
        outcome, rcp_scenario, population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, adaptation_effectiveness
    )
    
    # Calculate worst scenario (RCP 8.5, 0% adaptation effectiveness) outcomes and costs
    _, worst_effective_additional_outcomes, worst_costs, worst_additional_outcomes = calculate_additional_outcomes_and_costs(
        outcome, 'RCP 8.5', population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, 0
    )
    
    # Calculate 0% adaptation scenario outcomes and costs for comparison
    _, zero_adaptation_outcomes, zero_adaptation_costs, _ = calculate_additional_outcomes_and_costs(
        outcome, rcp_scenario, population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, 0
    )
    
    # Calculate cost savings
    mitigation_savings = [worst - selected for worst, selected in zip(worst_costs, costs)]
    adaptation_savings = [zero - selected for zero, selected in zip(zero_adaptation_costs, costs)]
    
    # Calculate additional outcomes prevented
    mitigation_prevented = [worst - selected for worst, selected in zip(worst_additional_outcomes, effective_additional_outcomes)]
    adaptation_prevented = [zero - selected for zero, selected in zip(zero_adaptation_outcomes, effective_additional_outcomes)]
    
    total_additional_outcomes = sum(effective_additional_outcomes)
    total_cost = sum(costs)
    total_mitigation_savings = sum(mitigation_savings)
    total_adaptation_savings = sum(adaptation_savings)
    total_mitigation_prevented = sum(mitigation_prevented)
    total_adaptation_prevented = sum(adaptation_prevented)
    
    display_text = f"{mnch_outcomes[outcome]['odds_ratio']} (95% CI: {mnch_outcomes[outcome]['ci_lower']} - {mnch_outcomes[outcome]['ci_upper']})"
    current_rate_text = f"{mnch_outcomes[outcome]['current_rate']}"
    initial_pregnancy_rate_text = f"{initial_pregnancy_rate}"
    adaptation_effectiveness_text = f"{adaptation_effectiveness}%"
    cost_per_case_text = f"${mnch_outcomes[outcome]['cost_per_case']}"
    attributable_fraction_text = f"{round(attributable_fraction * 100, 2)}%"
    total_additional_outcomes_text = f"{int(round(total_additional_outcomes))}"
    total_cost_text = f"${round(total_cost, 2):,}"
    total_mitigation_savings_text = f"${round(total_mitigation_savings, 2):,}"
    total_adaptation_savings_text = f"${round(total_adaptation_savings, 2):,}"
    total_mitigation_prevented_text = f"{int(round(total_mitigation_prevented))}"
    total_adaptation_prevented_text = f"{int(round(total_adaptation_prevented))}"
    
    fig_outcomes = go.Figure()
    
    # Add additional outcomes
    fig_outcomes.add_trace(go.Scatter(x=selected_years, y=effective_additional_outcomes, mode='lines', name=f'Selected Scenario ({rcp_scenario}, {adaptation_effectiveness}%)', line=dict(color='blue')))
    
    # Add temperature increase
    temperature_increase_per_year = calculate_temperature_increase_per_year(rcp_scenarios[rcp_scenario], time_period[0], time_period[1])
    fig_outcomes.add_trace(go.Scatter(x=selected_years, y=temperature_increase_per_year, mode='lines', name='Temperature Increase', yaxis='y2',line=dict(color='red')))
    
    # Update layout for multiple y-axes
    fig_outcomes.update_layout(
        title=f"Effective Additional {outcome} Outcomes Due to Heat and Temperature Increase from {time_period[0]} to {time_period[1]}",
        xaxis_title="Year",
        yaxis=dict(
            title="Effective Additional Outcomes",
            titlefont=dict(color="#1f77b4"),
            tickfont=dict(color="#1f77b4")
        ),
        yaxis2=dict(
            title="Temperature Increase (°C)",
            titlefont=dict(color="#ff7f0e"),
            tickfont=dict(color="#ff7f0e"),
            overlaying='y',
            side='right'
        ),
        legend=dict(yanchor="bottom", y=-0.6, xanchor="center", x=0.5)
    )
    
    fig_additional_outcomes_prevented = go.Figure()
    
    # Add worst-case scenario additional outcomes
    fig_additional_outcomes_prevented.add_trace(go.Scatter(x=selected_years, y=worst_additional_outcomes, mode='lines', name='Worst Case (RCP 8.5, 0% Adaptation)', line=dict(color='red')))
    
    # Add selected scenario additional outcomes
    fig_additional_outcomes_prevented.add_trace(go.Scatter(x=selected_years, y=effective_additional_outcomes, mode='lines', name=f'Selected Scenario ({rcp_scenario}, {adaptation_effectiveness}%)', line=dict(color='blue')))
    
    # Shade the area between the graphs to represent mitigation and adaptation
    fig_additional_outcomes_prevented.add_trace(go.Scatter(
        x=selected_years + selected_years[::-1],
        y=worst_additional_outcomes + effective_additional_outcomes[::-1],
        fill='toself',
        fillcolor='rgba(0, 255, 0, 0.2)',
        line=dict(color='rgba(255, 255, 255, 0)'),
        showlegend=True,
        name='Mitigation'
    ))
    fig_additional_outcomes_prevented.add_trace(go.Scatter(
        x=selected_years + selected_years[::-1],
        y=zero_adaptation_outcomes + effective_additional_outcomes[::-1],
        fill='toself',
        fillcolor='rgba(0, 0, 255, 0.2)',
        line=dict(color='rgba(255, 255, 255, 0)'),
        showlegend=True,
        name='Adaptation'
    ))
    
    fig_additional_outcomes_prevented.update_layout(
        title=f"Additional {outcome} Outcomes Prevented by Mitigation and Adaptation from {time_period[0]} to {time_period[1]}",
        xaxis_title="Year",
        yaxis=dict(
            title="Additional Outcomes",
            titlefont=dict(color="#1f77b4"),
            tickfont=dict(color="#1f77b4")
        ),
        legend=dict(yanchor="bottom", y=-0.5, xanchor="center", x=0.5)
    )
    
    fig_costs = go.Figure()
    
    # Add costs
    fig_costs.add_trace(go.Scatter(x=selected_years, y=costs, mode='lines', name=f'Selected Scenario ({rcp_scenario}, {adaptation_effectiveness}%)', line=dict(color='blue')))
    
    fig_costs.update_layout(
        title=f"Costs Associated with Effective Additional {outcome} Outcomes Due to Heat from {time_period[0]} to {time_period[1]}",
        xaxis_title="Year",
        yaxis=dict(
            title="Costs (USD)",
            titlefont=dict(color="#1f77b4"),
            tickfont=dict(color="#1f77b4")
        ),
        legend=dict(yanchor="bottom", y=-0.5, xanchor="center", x=0.5)
    )
    
    fig_cost_savings = go.Figure()
    
    # Add worst-case scenario costs
    fig_cost_savings.add_trace(go.Scatter(x=selected_years, y=worst_costs, mode='lines', name='Worst Case (RCP 8.5, 0% Adaptation)', line=dict(color='red')))
    
    # Add selected scenario costs
    fig_cost_savings.add_trace(go.Scatter(x=selected_years, y=costs, mode='lines', name=f'Selected Scenario ({rcp_scenario}, {adaptation_effectiveness}%)', line=dict(color='blue')))
    
    # Shade the area between the graphs to represent mitigation and adaptation
    fig_cost_savings.add_trace(go.Scatter(
        x=selected_years + selected_years[::-1],
        y=worst_costs + costs[::-1],
        fill='toself',
        fillcolor='rgba(0, 255, 0, 0.2)',
        line=dict(color='rgba(255, 255, 255, 0)'),
        showlegend=True,
        name='Mitigation'
    ))
    fig_cost_savings.add_trace(go.Scatter(
        x=selected_years + selected_years[::-1],
        y=zero_adaptation_costs + costs[::-1],
        fill='toself',
        fillcolor='rgba(0, 0, 255, 0.2)',
        line=dict(color='rgba(255, 255, 255, 0)'),
        showlegend=True,
        name='Adaptation'
    ))
    
    fig_cost_savings.update_layout(
        title=f"Cost Savings by Comparing Worst Scenario (RCP 8.5, 0% Adaptation) and Selected Scenario from {time_period[0]} to {time_period[1]}",
        xaxis_title="Year",
        yaxis=dict(
            title="Costs (USD)",
            titlefont=dict(color="#1f77b4"),
            tickfont=dict(color="#1f77b4")
        ),
        legend=dict(yanchor="bottom", y=-0.5, xanchor="center", x=0.5)
    )
    
    return display_text, current_rate_text, initial_pregnancy_rate_text, adaptation_effectiveness_text, cost_per_case_text, attributable_fraction_text, total_additional_outcomes_text, total_cost_text, total_mitigation_savings_text, total_adaptation_savings_text, total_mitigation_prevented_text, total_adaptation_prevented_text, fig_outcomes, fig_additional_outcomes_prevented, fig_costs, fig_cost_savings

# Export Data Callback
@app.callback(
    Output("download-dataframe-csv", "data"),
    Input("export-button", "n_clicks"),
    prevent_initial_call=True,
)
def export_data(n_clicks):
    if n_clicks is None:
        return dash.no_update

    # Retrieve values from the update_dashboard function (need to be moved to global scope)
    outcome = 'Preterm Birth'  # or get dynamically if possible
    rcp_scenario = 'RCP 2.6'  # or get dynamically if possible
    population_growth_curve = 'Medium Growth'  # or get dynamically if possible
    pregnancy_growth_curve = 'Medium Growth'  # or get dynamically if possible
    time_period = [2024, 2030]  # or get dynamically if possible
    current_population = 7000000000  # or get dynamically if possible
    initial_pregnancy_rate = 0.02  # or get dynamically if possible
    adaptation_effectiveness = 50  # or get dynamically if possible
    
    selected_years, effective_additional_outcomes, costs, additional_outcomes = calculate_additional_outcomes_and_costs(
        outcome, rcp_scenario, population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, adaptation_effectiveness
    )
    
    _, worst_effective_additional_outcomes, worst_costs, worst_additional_outcomes = calculate_additional_outcomes_and_costs(
        outcome, 'RCP 8.5', population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, 0
    )
    
    _, zero_adaptation_outcomes, zero_adaptation_costs, _ = calculate_additional_outcomes_and_costs(
        outcome, rcp_scenario, population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, 0
    )
    
    mitigation_savings = [worst - selected for worst, selected in zip(worst_costs, costs)]
    adaptation_savings = [zero - selected for zero, selected in zip(zero_adaptation_costs, costs)]
    mitigation_prevented = [worst - selected for worst, selected in zip(worst_additional_outcomes, effective_additional_outcomes)]
    adaptation_prevented = [zero - selected for zero, selected in zip(zero_adaptation_outcomes, effective_additional_outcomes)]

    # Create a DataFrame with the current projections
    data = {
        'Year': selected_years,
        'Effective Additional Outcomes': effective_additional_outcomes,
        'Temperature Increase': calculate_temperature_increase_per_year(rcp_scenarios[rcp_scenario], time_period[0], time_period[1]),
        'Costs': costs,
        'Worst Case Costs': worst_costs,
        'Zero Adaptation Costs': zero_adaptation_costs,
        'Mitigation Savings': mitigation_savings,
        'Adaptation Savings': adaptation_savings,
        'Mitigation Prevented': mitigation_prevented,
        'Adaptation Prevented': adaptation_prevented
    }
    df = pd.DataFrame(data)

    return dcc.send_data_frame(df.to_csv, "projected_outcomes.csv")

# Step 5: Run the App
if __name__ == '__main__':
    app.run_server(debug=True)



In [44]:
#version 14 with confidence intervals
# Import necessary packages
import dash
from dash import dcc, html, Input, Output
import plotly.graph_objs as go
import pandas as pd
import numpy as np

# Step 2: Prepare the Data
mnch_outcomes = {
    'Preterm Birth': {'odds_ratio': 1.04, 'current_rate': 0.1, 'cost_per_case': 1000, 'ci_lower': 1.03, 'ci_upper': 1.06},
    'Low Birth Weight': {'odds_ratio': 1.03, 'current_rate': 0.08, 'cost_per_case': 800, 'ci_lower': 1.02, 'ci_upper': 1.05},
    'Stillbirth': {'odds_ratio': 1.14, 'current_rate': 0.005, 'cost_per_case': 5000, 'ci_lower': 0.99, 'ci_upper': 1.32},
    'Congenital Anomalies': {'odds_ratio': 1.29, 'current_rate': 0.02, 'cost_per_case': 10000, 'ci_lower': 1.21, 'ci_upper': 1.38},
    'Gestational Diabetes Mellitus': {'odds_ratio': 1.28, 'current_rate': 0.05, 'cost_per_case': 2000, 'ci_lower': 1.05, 'ci_upper': 1.74},
    'Hypertensive Disorders': {'odds_ratio': 1.16, 'current_rate': 0.1, 'cost_per_case': 1500, 'ci_lower': 1.10, 'ci_upper': 1.22},
    'Prelabour Rupture of Membranes': {'odds_ratio': 1.63, 'current_rate': 0.02, 'cost_per_case': 1200, 'ci_lower': 1.23, 'ci_upper': 2.16},
    'Infections': {'odds_ratio': 1.15, 'current_rate': 0.02, 'cost_per_case': 800, 'ci_lower': 1.03, 'ci_upper': 1.40},
    'Placental Abruption': {'odds_ratio': 1.16, 'current_rate': 0.005, 'cost_per_case': 2500, 'ci_lower': 1.03, 'ci_upper': 1.40},
    'Neonatal Admissions': {'odds_ratio': 1.33, 'current_rate': 0.02, 'cost_per_case': 1500, 'ci_lower': 1.27, 'ci_upper': 1.38}
}

rcp_scenarios = {
    'RCP 2.6': 1.5,  # degrees Celsius increase by 2100
    'RCP 4.5': 2.4,
    'RCP 6.0': 3.0,
    'RCP 8.5': 4.3
}

population_growth_rates = {
    'Low Growth': 0.01,  # 1% annual growth
    'Medium Growth': 0.02,  # 2% annual growth
    'High Growth': 0.03  # 3% annual growth
}

pregnancy_growth_rates = {
    'High Reduction': -0.15,  # 15% reduction
    'Medium Reduction': -0.1,  # 10% reduction
    'Low Reduction': -0.05,  # 5% reduction
    'Low Growth': 0.01,  # 1% annual growth
    'Medium Growth': 0.02,  # 2% annual growth
    'High Growth': 0.03  # 3% annual growth
}

years = list(range(2024, 2051))

def calculate_population_growth(initial_population, growth_rate, num_years):
    return [initial_population * ((1 + growth_rate) ** year) for year in range(num_years)]

def calculate_pregnancies(population, pregnancy_rate):
    return [pop * pregnancy_rate for pop in population]

def calculate_temperature_increase_per_year(temp_increase_2100, start_year, end_year):
    total_years = 2100 - 2024  # Calculate total years from 2024 to 2100
    annual_increase = temp_increase_2100 / total_years
    return [annual_increase * (year - 2024) for year in range(start_year, end_year + 1)]

def calculate_additional_outcomes_and_costs(outcome, rcp_scenario, population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, adaptation_effectiveness):
    odds_ratio = mnch_outcomes[outcome]['odds_ratio']
    ci_lower = mnch_outcomes[outcome]['ci_lower']
    ci_upper = mnch_outcomes[outcome]['ci_upper']
    current_rate = mnch_outcomes[outcome]['current_rate']
    cost_per_case = mnch_outcomes[outcome]['cost_per_case']
    temp_increase_2100 = rcp_scenarios[rcp_scenario]
    population_growth_rate = population_growth_rates[population_growth_curve]
    pregnancy_growth_rate = pregnancy_growth_rates[pregnancy_growth_curve]
    
    start_year, end_year = time_period
    num_years = end_year - start_year + 1
    selected_years = list(range(start_year, end_year + 1))
    
    # Calculate population growth
    population_growth_selected = calculate_population_growth(current_population, population_growth_rate, num_years)
    
    # Calculate pregnancies without the effect of heat
    pregnancies = calculate_pregnancies(population_growth_selected, initial_pregnancy_rate * (1 + pregnancy_growth_rate))
    
    # Calculate baseline outcomes without the effect of heat
    baseline_outcomes = [preg * current_rate for preg in pregnancies]
    
    # Calculate the temperature increase per year
    temperature_increase_per_year = calculate_temperature_increase_per_year(temp_increase_2100, start_year, end_year)
    
    # Calculate projected outcomes with the effect of heat
    projected_rates = [current_rate * (odds_ratio ** temp_increase) for temp_increase in temperature_increase_per_year]
    projected_outcomes = [rate * preg for rate, preg in zip(projected_rates, pregnancies)]
    
    # Calculate additional outcomes due to the effect of heat
    additional_outcomes = [proj - base for proj, base in zip(projected_outcomes, baseline_outcomes)]
    
    # Apply adaptation effectiveness
    effective_additional_outcomes = [outcome * (1 - adaptation_effectiveness / 100) for outcome in additional_outcomes]
    
    # Calculate confidence intervals for effective additional outcomes
    effective_additional_outcomes_lower = [outcome * (1 - adaptation_effectiveness / 100) * ci_lower / odds_ratio for outcome in additional_outcomes]
    effective_additional_outcomes_upper = [outcome * (1 - adaptation_effectiveness / 100) * ci_upper / odds_ratio for outcome in additional_outcomes]
    
    # Calculate the total cost
    costs = [outcome * cost_per_case for outcome in effective_additional_outcomes]
    costs_lower = [outcome * cost_per_case for outcome in effective_additional_outcomes_lower]
    costs_upper = [outcome * cost_per_case for outcome in effective_additional_outcomes_upper]
    
    return selected_years, effective_additional_outcomes, costs, additional_outcomes, effective_additional_outcomes_lower, effective_additional_outcomes_upper, costs_lower, costs_upper

# Calculate Attributable Fraction
def calculate_attributable_fraction(odds_ratio):
    return (odds_ratio - 1) / odds_ratio

# Step 3: Create the Dash Layout
app = dash.Dash(__name__)

app.layout = html.Div([
    html.Link(rel='stylesheet', href='/assets/style.css'),
    html.H1("MNCH Outcomes Due to Heat"),
    html.H2("Climate Change Projection", className='subtitle'),
    html.Div(className='container', children=[
        html.Div(className='input-container', children=[
            html.Div(className='input-box', children=[
                html.H3("Select MNCH Outcome"),
                dcc.Dropdown(
                    id='mnch-outcome-dropdown',
                    options=[{'label': outcome, 'value': outcome} for outcome in mnch_outcomes.keys()],
                    value='Preterm Birth',
                    className='dropdown'
                )
            ]),
            html.Div(className='input-box', children=[
                html.H3("Select RCP Scenario"),
                dcc.Dropdown(
                    id='rcp-scenario-dropdown',
                    options=[{'label': scenario, 'value': scenario} for scenario in rcp_scenarios.keys()],
                    value='RCP 2.6',
                    className='dropdown'
                )
            ]),
            html.Div(className='input-box', children=[
                html.H3("Select Population Growth Curve"),
                dcc.Dropdown(
                    id='population-growth-dropdown',
                    options=[{'label': growth, 'value': growth} for growth in population_growth_rates.keys()],
                    value='Medium Growth',
                    className='dropdown'
                )
            ]),
            html.Div(className='input-box', children=[
                html.H3("Select Pregnancy Growth Rate"),
                dcc.Dropdown(
                    id='pregnancy-growth-dropdown',
                    options=[{'label': growth, 'value': growth} for growth in pregnancy_growth_rates.keys()],
                    value='Medium Growth',
                    className='dropdown'
                )
            ]),
            html.Div(className='input-box', children=[
                html.H3("Current Population"),
                dcc.Input(
                    id='current-population-input',
                    type='number',
                    value=7000000000,
                    className='input'
                )
            ]),
            html.Div(className='input-box', children=[
                html.H3("Initial Pregnancy Rate"),
                dcc.Input(
                    id='initial-pregnancy-rate-input',
                    type='number',
                    value=0.02,  # 2% initial pregnancy rate
                    step=0.001,
                    className='input'
                )
            ])
        ]),
        html.Div(className='slider-box', children=[
            html.H3("Adaptation Effectiveness (%)"),
            dcc.Slider(
                id='adaptation-effectiveness-slider',
                min=0,
                max=100,
                step=1,
                value=50,
                marks={i: f'{i}%' for i in range(0, 101, 10)},
                className='slider'
            )
        ]),
        html.Div(className='slider-box', children=[
            html.H3("Select Time Period"),
            dcc.RangeSlider(
                id='time-period-slider',
                min=2024,
                max=2050,
                step=1,
                marks={year: str(year) for year in years},
                value=[2024, 2030],
                className='slider'
            )
        ]),
        html.H2("Outputs", className='subtitle'),
        html.Div(className='key-indicators', children=[
            html.Div(className='key-indicator', children=[
                html.H2("Odds Ratio per 1°C increase"),
                html.P(id='odds-and-rate-display')
            ]),
            html.Div(className='key-indicator', children=[
                html.H2("Current Rate"),
                html.P(id='current-rate-display')
            ]),
            html.Div(className='key-indicator', children=[
                html.H2("Initial Pregnancy Rate"),
                html.P(id='initial-pregnancy-rate-display')
            ]),
            html.Div(className='key-indicator', children=[
                html.H2("Adaptation Effectiveness"),
                html.P(id='adaptation-effectiveness-display')
            ]),
            html.Div(className='key-indicator', children=[
                html.H2("Cost per Case"),
                html.P(id='cost-per-case-display')
            ]),
            html.Div(className='key-indicator', children=[
                html.H2("Attributable Fraction"),
                html.P(id='attributable-fraction-display')
            ]),
            html.Div(className='key-indicator', children=[
                html.H2("Total Additional Outcomes"),
                html.P(id='total-additional-outcomes-display')
            ]),
            html.Div(className='key-indicator', children=[
                html.H2("Total Cost"),
                html.P(id='total-cost-display')
            ]),
            html.Div(className='key-indicator', children=[
                html.H2("Total Mitigation Savings"),
                html.P(id='total-mitigation-savings-display')
            ]),
            html.Div(className='key-indicator', children=[
                html.H2("Total Adaptation Savings"),
                html.P(id='total-adaptation-savings-display')
            ]),
            html.Div(className='key-indicator', children=[
                html.H2("Total Mitigation Prevented"),
                html.P(id='total-mitigation-prevented-display')
            ]),
            html.Div(className='key-indicator', children=[
                html.H2("Total Adaptation Prevented"),
                html.P(id='total-adaptation-prevented-display')
            ])
        ]),
        html.Div(className='graph-container', children=[
            dcc.Graph(id='outcome-projection-graph'),
            dcc.Graph(id='additional-outcomes-prevented-graph'),
            dcc.Graph(id='cost-projection-graph'),
            dcc.Graph(id='cost-savings-graph')
        ]),
        html.Button("Export Data", id="export-button", className='button'),
        dcc.Download(id="download-dataframe-csv")
    ])
])

# Step 4: Add Interactivity
@app.callback(
    Output('odds-and-rate-display', 'children'),
    Output('current-rate-display', 'children'),
    Output('initial-pregnancy-rate-display', 'children'),
    Output('adaptation-effectiveness-display', 'children'),
    Output('cost-per-case-display', 'children'),
    Output('attributable-fraction-display', 'children'),
    Output('total-additional-outcomes-display', 'children'),
    Output('total-cost-display', 'children'),
    Output('total-mitigation-savings-display', 'children'),
    Output('total-adaptation-savings-display', 'children'),
    Output('total-mitigation-prevented-display', 'children'),
    Output('total-adaptation-prevented-display', 'children'),
    Output('outcome-projection-graph', 'figure'),
    Output('additional-outcomes-prevented-graph', 'figure'),
    Output('cost-projection-graph', 'figure'),
    Output('cost-savings-graph', 'figure'),
    Input('mnch-outcome-dropdown', 'value'),
    Input('rcp-scenario-dropdown', 'value'),
    Input('population-growth-dropdown', 'value'),
    Input('pregnancy-growth-dropdown', 'value'),
    Input('time-period-slider', 'value'),
    Input('current-population-input', 'value'),
    Input('initial-pregnancy-rate-input', 'value'),
    Input('adaptation-effectiveness-slider', 'value')
)
def update_dashboard(outcome, rcp_scenario, population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, adaptation_effectiveness):
    odds_ratio = mnch_outcomes[outcome]['odds_ratio']
    
    # Calculate Attributable Fraction
    attributable_fraction = calculate_attributable_fraction(odds_ratio)
    
    # Calculate selected scenario outcomes and costs
    selected_years, effective_additional_outcomes, costs, additional_outcomes, effective_additional_outcomes_lower, effective_additional_outcomes_upper, costs_lower, costs_upper = calculate_additional_outcomes_and_costs(
        outcome, rcp_scenario, population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, adaptation_effectiveness
    )
    
    # Calculate worst scenario (RCP 8.5, 0% adaptation effectiveness) outcomes and costs
    _, worst_effective_additional_outcomes, worst_costs, worst_additional_outcomes, _, _, _, _ = calculate_additional_outcomes_and_costs(
        outcome, 'RCP 8.5', population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, 0
    )
    
    # Calculate 0% adaptation scenario outcomes and costs for comparison
    _, zero_adaptation_outcomes, zero_adaptation_costs, _, _, _, _, _ = calculate_additional_outcomes_and_costs(
        outcome, rcp_scenario, population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, 0
    )
    
    # Calculate cost savings
    mitigation_savings = [worst - selected for worst, selected in zip(worst_costs, costs)]
    adaptation_savings = [zero - selected for zero, selected in zip(zero_adaptation_costs, costs)]
    
    # Calculate additional outcomes prevented
    mitigation_prevented = [worst - selected for worst, selected in zip(worst_additional_outcomes, effective_additional_outcomes)]
    adaptation_prevented = [zero - selected for zero, selected in zip(zero_adaptation_outcomes, effective_additional_outcomes)]
    
    total_additional_outcomes = sum(effective_additional_outcomes)
    total_cost = sum(costs)
    total_mitigation_savings = sum(mitigation_savings)
    total_adaptation_savings = sum(adaptation_savings)
    total_mitigation_prevented = sum(mitigation_prevented)
    total_adaptation_prevented = sum(adaptation_prevented)
    
    display_text = f"{mnch_outcomes[outcome]['odds_ratio']} (95% CI: {mnch_outcomes[outcome]['ci_lower']} - {mnch_outcomes[outcome]['ci_upper']})"
    current_rate_text = f"{mnch_outcomes[outcome]['current_rate']}"
    initial_pregnancy_rate_text = f"{initial_pregnancy_rate}"
    adaptation_effectiveness_text = f"{adaptation_effectiveness}%"
    cost_per_case_text = f"${mnch_outcomes[outcome]['cost_per_case']}"
    attributable_fraction_text = f"{round(attributable_fraction * 100, 2)}%"
    total_additional_outcomes_text = f"{int(round(total_additional_outcomes))}"
    total_cost_text = f"${round(total_cost, 2):,}"
    total_mitigation_savings_text = f"${round(total_mitigation_savings, 2):,}"
    total_adaptation_savings_text = f"${round(total_adaptation_savings, 2):,}"
    total_mitigation_prevented_text = f"{int(round(total_mitigation_prevented))}"
    total_adaptation_prevented_text = f"{int(round(total_adaptation_prevented))}"
    
    fig_outcomes = go.Figure()
    
    # Add additional outcomes
    fig_outcomes.add_trace(go.Scatter(x=selected_years, y=effective_additional_outcomes, mode='lines', name=f'Selected Scenario ({rcp_scenario}, {adaptation_effectiveness}%)', line=dict(color='blue')))
    
    # Add confidence interval shading for effective additional outcomes
    fig_outcomes.add_trace(go.Scatter(
        x=selected_years + selected_years[::-1],
        y=effective_additional_outcomes_upper + effective_additional_outcomes_lower[::-1],
        fill='toself',
        fillcolor='rgba(0, 0, 255, 0.2)',
        line=dict(color='rgba(255, 255, 255, 0)'),
        showlegend=False,
        name='Confidence Interval'
    ))
    
    # Add temperature increase
    temperature_increase_per_year = calculate_temperature_increase_per_year(rcp_scenarios[rcp_scenario], time_period[0], time_period[1])
    fig_outcomes.add_trace(go.Scatter(x=selected_years, y=temperature_increase_per_year, mode='lines', name='Temperature Increase', yaxis='y2'))
    
    # Update layout for multiple y-axes
    fig_outcomes.update_layout(
        title=f"Effective Additional {outcome} Outcomes Due to Heat and Temperature Increase from {time_period[0]} to {time_period[1]}",
        xaxis_title="Year",
        yaxis=dict(
            title="Effective Additional Outcomes",
            titlefont=dict(color="#1f77b4"),
            tickfont=dict(color="#1f77b4")
        ),
        yaxis2=dict(
            title="Temperature Increase (°C)",
            titlefont=dict(color="#ff7f0e"),
            tickfont=dict(color="#ff7f0e"),
            overlaying='y',
            side='right'
        ),
        legend=dict(yanchor="bottom", y=-0.6, xanchor="center", x=0.5)
    )
    
    fig_additional_outcomes_prevented = go.Figure()
    
    # Add worst-case scenario additional outcomes
    fig_additional_outcomes_prevented.add_trace(go.Scatter(x=selected_years, y=worst_additional_outcomes, mode='lines', name='Worst Case (RCP 8.5, 0% Adaptation)', line=dict(color='red')))
    
    # Add selected scenario additional outcomes
    fig_additional_outcomes_prevented.add_trace(go.Scatter(x=selected_years, y=effective_additional_outcomes, mode='lines', name=f'Selected Scenario ({rcp_scenario}, {adaptation_effectiveness}%)', line=dict(color='blue')))
    
    # Shade the area between the graphs to represent mitigation and adaptation
    fig_additional_outcomes_prevented.add_trace(go.Scatter(
        x=selected_years + selected_years[::-1],
        y=worst_additional_outcomes + effective_additional_outcomes[::-1],
        fill='toself',
        fillcolor='rgba(0, 255, 0, 0.2)',
        line=dict(color='rgba(255, 255, 255, 0)'),
        showlegend=True,
        name='Mitigation'
    ))
    fig_additional_outcomes_prevented.add_trace(go.Scatter(
        x=selected_years + selected_years[::-1],
        y=zero_adaptation_outcomes + effective_additional_outcomes[::-1],
        fill='toself',
        fillcolor='rgba(0, 0, 255, 0.2)',
        line=dict(color='rgba(255, 255, 255, 0)'),
        showlegend=True,
        name='Adaptation'
    ))
    
    fig_additional_outcomes_prevented.update_layout(
        title=f"Additional {outcome} Outcomes Prevented by Mitigation and Adaptation from {time_period[0]} to {time_period[1]}",
        xaxis_title="Year",
        yaxis=dict(
            title="Additional Outcomes",
            titlefont=dict(color="#1f77b4"),
            tickfont=dict(color="#1f77b4")
        ),
        legend=dict(yanchor="bottom", y=-0.5, xanchor="center", x=0.5)
    )
    
    fig_costs = go.Figure()
    
    # Add costs
    fig_costs.add_trace(go.Scatter(x=selected_years, y=costs, mode='lines', name=f'Selected Scenario ({rcp_scenario}, {adaptation_effectiveness}%)', line=dict(color='blue')))
    
    # Add confidence interval shading for costs
    fig_costs.add_trace(go.Scatter(
        x=selected_years + selected_years[::-1],
        y=costs_upper + costs_lower[::-1],
        fill='toself',
        fillcolor='rgba(0, 0, 255, 0.2)',
        line=dict(color='rgba(255, 255, 255, 0)'),
        showlegend=False,
        name='Confidence Interval'
    ))
    
    fig_costs.update_layout(
        title=f"Costs Associated with Effective Additional {outcome} Outcomes Due to Heat from {time_period[0]} to {time_period[1]}",
        xaxis_title="Year",
        yaxis=dict(
            title="Costs (USD)",
            titlefont=dict(color="#1f77b4"),
            tickfont=dict(color="#1f77b4")
        ),
        legend=dict(yanchor="bottom", y=-0.5, xanchor="center", x=0.5)
    )
    
    fig_cost_savings = go.Figure()
    
    # Add worst-case scenario costs
    fig_cost_savings.add_trace(go.Scatter(x=selected_years, y=worst_costs, mode='lines', name='Worst Case (RCP 8.5, 0% Adaptation)', line=dict(color='red')))
    
    # Add selected scenario costs
    fig_cost_savings.add_trace(go.Scatter(x=selected_years, y=costs, mode='lines', name=f'Selected Scenario ({rcp_scenario}, {adaptation_effectiveness}%)', line=dict(color='blue')))
    
    # Shade the area between the graphs to represent mitigation and adaptation
    fig_cost_savings.add_trace(go.Scatter(
        x=selected_years + selected_years[::-1],
        y=worst_costs + costs[::-1],
        fill='toself',
        fillcolor='rgba(0, 255, 0, 0.2)',
        line=dict(color='rgba(255, 255, 255, 0)'),
        showlegend=True,
        name='Mitigation'
    ))
    fig_cost_savings.add_trace(go.Scatter(
        x=selected_years + selected_years[::-1],
        y=zero_adaptation_costs + costs[::-1],
        fill='toself',
        fillcolor='rgba(0, 0, 255, 0.2)',
        line=dict(color='rgba(255, 255, 255, 0)'),
        showlegend=True,
        name='Adaptation'
    ))
    
    fig_cost_savings.update_layout(
        title=f"Cost Savings by Comparing Worst Scenario (RCP 8.5, 0% Adaptation) and Selected Scenario from {time_period[0]} to {time_period[1]}",
        xaxis_title="Year",
        yaxis=dict(
            title="Costs (USD)",
            titlefont=dict(color="#1f77b4"),
            tickfont=dict(color="#1f77b4")
        ),
        legend=dict(yanchor="bottom", y=-0.5, xanchor="center", x=0.5)
    )
    
    return display_text, current_rate_text, initial_pregnancy_rate_text, adaptation_effectiveness_text, cost_per_case_text, attributable_fraction_text, total_additional_outcomes_text, total_cost_text, total_mitigation_savings_text, total_adaptation_savings_text, total_mitigation_prevented_text, total_adaptation_prevented_text, fig_outcomes, fig_additional_outcomes_prevented, fig_costs, fig_cost_savings

# Export Data Callback
@app.callback(
    Output("download-dataframe-csv", "data"),
    Input("export-button", "n_clicks"),
    prevent_initial_call=True,
)
def export_data(n_clicks):
    if n_clicks is None:
        return dash.no_update

    # Retrieve values from the update_dashboard function (need to be moved to global scope)
    outcome = 'Preterm Birth'  # or get dynamically if possible
    rcp_scenario = 'RCP 2.6'  # or get dynamically if possible
    population_growth_curve = 'Medium Growth'  # or get dynamically if possible
    pregnancy_growth_curve = 'Medium Growth'  # or get dynamically if possible
    time_period = [2024, 2030]  # or get dynamically if possible
    current_population = 7000000000  # or get dynamically if possible
    initial_pregnancy_rate = 0.02  # or get dynamically if possible
    adaptation_effectiveness = 50  # or get dynamically if possible
    
    selected_years, effective_additional_outcomes, costs, additional_outcomes, _, _, _, _ = calculate_additional_outcomes_and_costs(
        outcome, rcp_scenario, population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, adaptation_effectiveness
    )
    
    _, worst_effective_additional_outcomes, worst_costs, worst_additional_outcomes, _, _, _, _ = calculate_additional_outcomes_and_costs(
        outcome, 'RCP 8.5', population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, 0
    )
    
    _, zero_adaptation_outcomes, zero_adaptation_costs, _, _, _, _ = calculate_additional_outcomes_and_costs(
        outcome, rcp_scenario, population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, 0
    )
    
    mitigation_savings = [worst - selected for worst, selected in zip(worst_costs, costs)]
    adaptation_savings = [zero - selected for zero, selected in zip(zero_adaptation_costs, costs)]
    mitigation_prevented = [worst - selected for worst, selected in zip(worst_additional_outcomes, effective_additional_outcomes)]
    adaptation_prevented = [zero - selected for zero, selected in zip(zero_adaptation_outcomes, effective_additional_outcomes)]

    # Create a DataFrame with the current projections
    data = {
        'Year': selected_years,
        'Effective Additional Outcomes': effective_additional_outcomes,
        'Temperature Increase': calculate_temperature_increase_per_year(rcp_scenarios[rcp_scenario], time_period[0], time_period[1]),
        'Costs': costs,
        'Worst Case Costs': worst_costs,
        'Zero Adaptation Costs': zero_adaptation_costs,
        'Mitigation Savings': mitigation_savings,
        'Adaptation Savings': adaptation_savings,
        'Mitigation Prevented': mitigation_prevented,
        'Adaptation Prevented': adaptation_prevented
    }
    df = pd.DataFrame(data)

    return dcc.send_data_frame(df.to_csv, "projected_outcomes.csv")

# Step 5: Run the App
if __name__ == '__main__':
    app.run_server(debug=True)


In [6]:
#version 16 with wbdata
import dash
from dash import dcc, html, Input, Output, State
import plotly.graph_objs as go
import pandas as pd
import numpy as np
import wbdata
from datetime import datetime

# Step 2: Prepare the Data
mnch_outcomes = {
    'Preterm Birth': {'odds_ratio': 1.04, 'current_rate': 0.1, 'cost_per_case': 1000, 'ci_lower': 1.03, 'ci_upper': 1.06},
    'Low Birth Weight': {'odds_ratio': 1.03, 'current_rate': 0.08, 'cost_per_case': 800, 'ci_lower': 1.02, 'ci_upper': 1.05},
    'Stillbirth': {'odds_ratio': 1.14, 'current_rate': 0.005, 'cost_per_case': 5000, 'ci_lower': 0.99, 'ci_upper': 1.32},
    'Congenital Anomalies': {'odds_ratio': 1.29, 'current_rate': 0.02, 'cost_per_case': 10000, 'ci_lower': 1.21, 'ci_upper': 1.38},
    'Gestational Diabetes Mellitus': {'odds_ratio': 1.28, 'current_rate': 0.05, 'cost_per_case': 2000, 'ci_lower': 1.05, 'ci_upper': 1.74},
    'Hypertensive Disorders': {'odds_ratio': 1.16, 'current_rate': 0.1, 'cost_per_case': 1500, 'ci_lower': 1.10, 'ci_upper': 1.22},
    'Prelabour Rupture of Membranes': {'odds_ratio': 1.63, 'current_rate': 0.02, 'cost_per_case': 1200, 'ci_lower': 1.23, 'ci_upper': 2.16},
    'Infections': {'odds_ratio': 1.15, 'current_rate': 0.02, 'cost_per_case': 800, 'ci_lower': 1.03, 'ci_upper': 1.40},
    'Placental Abruption': {'odds_ratio': 1.16, 'current_rate': 0.005, 'cost_per_case': 2500, 'ci_lower': 1.03, 'ci_upper': 1.40},
    'Neonatal Admissions': {'odds_ratio': 1.33, 'current_rate': 0.02, 'cost_per_case': 1500, 'ci_lower': 1.27, 'ci_upper': 1.38}
}

rcp_scenarios = {
    'RCP 2.6': 1.5,  # degrees Celsius increase by 2100
    'RCP 4.5': 2.4,
    'RCP 6.0': 3.0,
    'RCP 8.5': 4.3
}

population_growth_rates = {
    'Low Growth': 0.01,  # 1% annual growth
    'Medium Growth': 0.02,  # 2% annual growth
    'High Growth': 0.03  # 3% annual growth
}

pregnancy_growth_rates = {
    'High Reduction': -0.15,  # 15% reduction
    'Medium Reduction': -0.1,  # 10% reduction
    'Low Reduction': -0.05,  # 5% reduction
    'Low Growth': 0.01,  # 1% annual growth
    'Medium Growth': 0.02,  # 2% annual growth
    'High Growth': 0.03  # 3% annual growth
}

years = list(range(2024, 2051))

def calculate_population_growth(initial_population, growth_rate, num_years):
    return [initial_population * ((1 + growth_rate) ** year) for year in range(num_years)]

def calculate_pregnancies(population, pregnancy_rate):
    return [pop * pregnancy_rate for pop in population]

def calculate_temperature_increase_per_year(temp_increase_2100, start_year, end_year):
    total_years = 2100 - 2024  # Calculate total years from 2024 to 2100
    annual_increase = temp_increase_2100 / total_years
    return [annual_increase * (year - 2024) for year in range(start_year, end_year + 1)]

def calculate_additional_outcomes_and_costs(outcome, rcp_scenario, population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, adaptation_effectiveness):
    odds_ratio = mnch_outcomes[outcome]['odds_ratio']
    ci_lower = mnch_outcomes[outcome]['ci_lower']
    ci_upper = mnch_outcomes[outcome]['ci_upper']
    current_rate = mnch_outcomes[outcome]['current_rate']
    cost_per_case = mnch_outcomes[outcome]['cost_per_case']
    temp_increase_2100 = rcp_scenarios[rcp_scenario]
    population_growth_rate = population_growth_rates[population_growth_curve]
    pregnancy_growth_rate = pregnancy_growth_rates[pregnancy_growth_curve]
    
    start_year, end_year = time_period
    num_years = end_year - start_year + 1
    selected_years = list(range(start_year, end_year + 1))
    
    # Calculate population growth
    population_growth_selected = calculate_population_growth(current_population, population_growth_rate, num_years)
    
    # Calculate pregnancies without the effect of heat
    pregnancies = calculate_pregnancies(population_growth_selected, initial_pregnancy_rate * (1 + pregnancy_growth_rate))
    
    # Calculate baseline outcomes without the effect of heat
    baseline_outcomes = [preg * current_rate for preg in pregnancies]
    
    # Calculate the temperature increase per year
    temperature_increase_per_year = calculate_temperature_increase_per_year(temp_increase_2100, start_year, end_year)
    
    # Calculate projected outcomes with the effect of heat
    projected_rates = [current_rate * (odds_ratio ** temp_increase) for temp_increase in temperature_increase_per_year]
    projected_outcomes = [rate * preg for rate, preg in zip(projected_rates, pregnancies)]
    
    # Calculate additional outcomes due to the effect of heat
    additional_outcomes = [proj - base for proj, base in zip(projected_outcomes, baseline_outcomes)]
    
    # Apply adaptation effectiveness
    effective_additional_outcomes = [outcome * (1 - adaptation_effectiveness / 100) for outcome in additional_outcomes]
    
    # Calculate confidence intervals for effective additional outcomes
    effective_additional_outcomes_lower = [outcome * (1 - adaptation_effectiveness / 100) * ci_lower / odds_ratio for outcome in additional_outcomes]
    effective_additional_outcomes_upper = [outcome * (1 - adaptation_effectiveness / 100) * ci_upper / odds_ratio for outcome in additional_outcomes]
    
    # Calculate the total cost
    costs = [outcome * cost_per_case for outcome in effective_additional_outcomes]
    costs_lower = [outcome * cost_per_case for outcome in effective_additional_outcomes_lower]
    costs_upper = [outcome * cost_per_case for outcome in effective_additional_outcomes_upper]
    
    return selected_years, effective_additional_outcomes, costs, additional_outcomes, effective_additional_outcomes_lower, effective_additional_outcomes_upper, costs_lower, costs_upper

# Calculate Attributable Fraction
def calculate_attributable_fraction(odds_ratio):
    return (odds_ratio - 1) / odds_ratio

# Fetch population data from World Bank
def fetch_population_data(country_code):
    try:
        # Fetch the latest available population data
        print(f"Fetching population data for {country_code}...")
        population_data = wbdata.get_dataframe({'SP.POP.TOTL': 'population'}, country=country_code)
        if population_data.empty:
            print(f"No population data available for {country_code}")
            return None
        population_data = population_data.reset_index()
        population_data['date'] = pd.to_datetime(population_data['date'])
        latest_population = population_data.loc[population_data['date'].idxmax()]
        return latest_population['population']
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Step 3: Create the Dash Layout
app = dash.Dash(__name__)

app.layout = html.Div([
    html.Link(rel='stylesheet', href='/assets/style.css'),
    html.H1("MNCH Outcomes Due to Heat"),
    html.H2("Climate Change Projection", className='subtitle'),
    html.Div(className='container', children=[
        html.Div(className='input-container', children=[
            html.Div(className='input-box', children=[
                html.H3("Select Country or Global"),
                dcc.Dropdown(
                    id='country-dropdown',
                    options=[{'label': 'Global', 'value': 'global'}] + [{'label': country['name'], 'value': country['id']} for country in wbdata.get_countries()],
                    value='global',
                    className='dropdown'
                )
            ]),
            html.Div(className='input-box', children=[
                dcc.Checklist(
                    id='manual-input-checklist',
                    options=[{'label': 'Use Manual Input', 'value': 'manual'}],
                    value=[]
                )
            ]),
            html.Div(className='input-box', children=[
                html.H3("Select MNCH Outcome"),
                dcc.Dropdown(
                    id='mnch-outcome-dropdown',
                    options=[{'label': outcome, 'value': outcome} for outcome in mnch_outcomes.keys()],
                    value='Preterm Birth',
                    className='dropdown'
                )
            ]),
            html.Div(className='input-box', children=[
                html.H3("Select RCP Scenario"),
                dcc.Dropdown(
                    id='rcp-scenario-dropdown',
                    options=[{'label': scenario, 'value': scenario} for scenario in rcp_scenarios.keys()],
                    value='RCP 2.6',
                    className='dropdown'
                )
            ]),
            html.Div(className='input-box', children=[
                html.H3("Select Population Growth Curve"),
                dcc.Dropdown(
                    id='population-growth-dropdown',
                    options=[{'label': growth, 'value': growth} for growth in population_growth_rates.keys()],
                    value='Medium Growth',
                    className='dropdown'
                )
            ]),
            html.Div(className='input-box', children=[
                html.H3("Select Pregnancy Growth Rate"),
                dcc.Dropdown(
                    id='pregnancy-growth-dropdown',
                    options=[{'label': growth, 'value': growth} for growth in pregnancy_growth_rates.keys()],
                    value='Medium Growth',
                    className='dropdown'
                )
            ]),
            html.Div(className='input-box', children=[
                html.H3("Current Population"),
                dcc.Input(
                    id='current-population-input',
                    type='number',
                    value=7000000000,
                    className='input'
                )
            ]),
            html.Div(className='input-box', children=[
                html.H3("Initial Pregnancy Rate"),
                dcc.Input(
                    id='initial-pregnancy-rate-input',
                    type='number',
                    value=0.02,  # 2% initial pregnancy rate
                    step=0.001,
                    className='input'
                )
            ])
        ]),
        html.Div(className='slider-box', children=[
            html.H3("Adaptation Effectiveness (%)"),
            dcc.Slider(
                id='adaptation-effectiveness-slider',
                min=0,
                max=100,
                step=1,
                value=50,
                marks={i: f'{i}%' for i in range(0, 101, 10)},
                className='slider'
            )
        ]),
        html.Div(className='slider-box', children=[
            html.H3("Select Time Period"),
            dcc.RangeSlider(
                id='time-period-slider',
                min=2024,
                max=2050,
                step=1,
                marks={year: str(year) for year in years},
                value=[2024, 2030],
                className='slider'
            )
        ]),
        html.H2("Outputs", className='subtitle'),
        html.Div(className='key-indicators', children=[
            html.Div(className='key-indicator', children=[
                html.H2("Odds Ratio per 1°C increase"),
                html.P(id='odds-and-rate-display')
            ]),
            html.Div(className='key-indicator', children=[
                html.H2("Current Rate"),
                html.P(id='current-rate-display')
            ]),
            html.Div(className='key-indicator', children=[
                html.H2("Initial Pregnancy Rate"),
                html.P(id='initial-pregnancy-rate-display')
            ]),
            html.Div(className='key-indicator', children=[
                html.H2("Adaptation Effectiveness"),
                html.P(id='adaptation-effectiveness-display')
            ]),
            html.Div(className='key-indicator', children=[
                html.H2("Cost per Case"),
                html.P(id='cost-per-case-display')
            ]),
            html.Div(className='key-indicator', children=[
                html.H2("Attributable Fraction"),
                html.P(id='attributable-fraction-display')
            ]),
            html.Div(className='key-indicator', children=[
                html.H2("Total Additional Outcomes"),
                html.P(id='total-additional-outcomes-display')
            ]),
            html.Div(className='key-indicator', children=[
                html.H2("Total Cost"),
                html.P(id='total-cost-display')
            ]),
            html.Div(className='key-indicator', children=[
                html.H2("Total Mitigation Savings"),
                html.P(id='total-mitigation-savings-display')
            ]),
            html.Div(className='key-indicator', children=[
                html.H2("Total Adaptation Savings"),
                html.P(id='total-adaptation-savings-display')
            ]),
            html.Div(className='key-indicator', children=[
                html.H2("Total Mitigation Prevented"),
                html.P(id='total-mitigation-prevented-display')
            ]),
            html.Div(className='key-indicator', children=[
                html.H2("Total Adaptation Prevented"),
                html.P(id='total-adaptation-prevented-display')
            ])
        ]),
        html.Div(className='graph-container', children=[
            dcc.Graph(id='outcome-projection-graph'),
            dcc.Graph(id='additional-outcomes-prevented-graph'),
            dcc.Graph(id='cost-projection-graph'),
            dcc.Graph(id='cost-savings-graph')
        ]),
        html.Button("Export Data", id="export-button", className='button'),
        dcc.Download(id="download-dataframe-csv")
    ])
])

# Step 4: Add Interactivity
@app.callback(
    Output('odds-and-rate-display', 'children'),
    Output('current-rate-display', 'children'),
    Output('initial-pregnancy-rate-display', 'children'),
    Output('adaptation-effectiveness-display', 'children'),
    Output('cost-per-case-display', 'children'),
    Output('attributable-fraction-display', 'children'),
    Output('total-additional-outcomes-display', 'children'),
    Output('total-cost-display', 'children'),
    Output('total-mitigation-savings-display', 'children'),
    Output('total-adaptation-savings-display', 'children'),
    Output('total-mitigation-prevented-display', 'children'),
    Output('total-adaptation-prevented-display', 'children'),
    Output('outcome-projection-graph', 'figure'),
    Output('additional-outcomes-prevented-graph', 'figure'),
    Output('cost-projection-graph', 'figure'),
    Output('cost-savings-graph', 'figure'),
    Input('mnch-outcome-dropdown', 'value'),
    Input('rcp-scenario-dropdown', 'value'),
    Input('population-growth-dropdown', 'value'),
    Input('pregnancy-growth-dropdown', 'value'),
    Input('time-period-slider', 'value'),
    Input('current-population-input', 'value'),
    Input('initial-pregnancy-rate-input', 'value'),
    Input('adaptation-effectiveness-slider', 'value'),
    Input('country-dropdown', 'value'),
    Input('manual-input-checklist', 'value')
)
def update_dashboard(outcome, rcp_scenario, population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, adaptation_effectiveness, country, manual_input):
    odds_ratio = mnch_outcomes[outcome]['odds_ratio']
    
    # Fetch population data if manual input is not selected
    if 'manual' not in manual_input:
        if country == 'global':
            current_population = 7000000000
        else:
            current_population = fetch_population_data(country)
    
    # Calculate Attributable Fraction
    attributable_fraction = calculate_attributable_fraction(odds_ratio)
    
    # Calculate selected scenario outcomes and costs
    selected_years, effective_additional_outcomes, costs, additional_outcomes, effective_additional_outcomes_lower, effective_additional_outcomes_upper, costs_lower, costs_upper = calculate_additional_outcomes_and_costs(
        outcome, rcp_scenario, population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, adaptation_effectiveness
    )
    
    # Calculate worst scenario (RCP 8.5, 0% adaptation effectiveness) outcomes and costs
    _, worst_effective_additional_outcomes, worst_costs, worst_additional_outcomes, _, _, _, _ = calculate_additional_outcomes_and_costs(
        outcome, 'RCP 8.5', population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, 0
    )
    
    # Calculate 0% adaptation scenario outcomes and costs for comparison
    _, zero_adaptation_outcomes, zero_adaptation_costs, _, _, _, _, _ = calculate_additional_outcomes_and_costs(
        outcome, rcp_scenario, population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, 0
    )
    
    # Calculate cost savings
    mitigation_savings = [worst - selected for worst, selected in zip(worst_costs, costs)]
    adaptation_savings = [zero - selected for zero, selected in zip(zero_adaptation_costs, costs)]
    
    # Calculate additional outcomes prevented
    mitigation_prevented = [worst - selected for worst, selected in zip(worst_additional_outcomes, effective_additional_outcomes)]
    adaptation_prevented = [zero - selected for zero, selected in zip(zero_adaptation_outcomes, effective_additional_outcomes)]
    
    total_additional_outcomes = sum(effective_additional_outcomes)
    total_cost = sum(costs)
    total_mitigation_savings = sum(mitigation_savings)
    total_adaptation_savings = sum(adaptation_savings)
    total_mitigation_prevented = sum(mitigation_prevented)
    total_adaptation_prevented = sum(adaptation_prevented)
    
    display_text = f"{mnch_outcomes[outcome]['odds_ratio']} (95% CI: {mnch_outcomes[outcome]['ci_lower']} - {mnch_outcomes[outcome]['ci_upper']})"
    current_rate_text = f"{mnch_outcomes[outcome]['current_rate']}"
    initial_pregnancy_rate_text = f"{initial_pregnancy_rate}"
    adaptation_effectiveness_text = f"{adaptation_effectiveness}%"
    cost_per_case_text = f"${mnch_outcomes[outcome]['cost_per_case']}"
    attributable_fraction_text = f"{round(attributable_fraction * 100, 2)}%"
    total_additional_outcomes_text = f"{int(round(total_additional_outcomes))}"
    total_cost_text = f"${round(total_cost, 2):,}"
    total_mitigation_savings_text = f"${round(total_mitigation_savings, 2):,}"
    total_adaptation_savings_text = f"${round(total_adaptation_savings, 2):,}"
    total_mitigation_prevented_text = f"{int(round(total_mitigation_prevented))}"
    total_adaptation_prevented_text = f"{int(round(total_adaptation_prevented))}"
    
    fig_outcomes = go.Figure()
    
    # Add additional outcomes
    fig_outcomes.add_trace(go.Scatter(x=selected_years, y=effective_additional_outcomes, mode='lines', name=f'Selected Scenario ({rcp_scenario}, {adaptation_effectiveness}%)', line=dict(color='blue')))
    
    # Add confidence interval shading for effective additional outcomes
    fig_outcomes.add_trace(go.Scatter(
        x=selected_years + selected_years[::-1],
        y=effective_additional_outcomes_upper + effective_additional_outcomes_lower[::-1],
        fill='toself',
        fillcolor='rgba(0, 0, 255, 0.2)',
        line=dict(color='rgba(255, 255, 255, 0)'),
        showlegend=False,
        name='Confidence Interval'
    ))
    
    # Add temperature increase
    temperature_increase_per_year = calculate_temperature_increase_per_year(rcp_scenarios[rcp_scenario], time_period[0], time_period[1])
    fig_outcomes.add_trace(go.Scatter(x=selected_years, y=temperature_increase_per_year, mode='lines', name='Temperature Increase', yaxis='y2', line=dict(color='red')))
    
    # Update layout for multiple y-axes
    fig_outcomes.update_layout(
        title=f"Effective Additional {outcome} Outcomes Due to Heat and Temperature Increase from {time_period[0]} to {time_period[1]}",
        xaxis_title="Year",
        yaxis=dict(
            title="Effective Additional Outcomes",
            titlefont=dict(color="#1f77b4"),
            tickfont=dict(color="#1f77b4")
        ),
        yaxis2=dict(
            title="Temperature Increase (°C)",
            titlefont=dict(color="#ff7f0e"),
            tickfont=dict(color="#ff7f0e"),
            overlaying='y',
            side='right'
        ),
        legend=dict(yanchor="bottom", y=-0.6, xanchor="center", x=0.5)
    )
    
    fig_additional_outcomes_prevented = go.Figure()
    
    # Add worst-case scenario additional outcomes
    fig_additional_outcomes_prevented.add_trace(go.Scatter(x=selected_years, y=worst_additional_outcomes, mode='lines', name='Worst Case (RCP 8.5, 0% Adaptation)', line=dict(color='red')))
    
    # Add selected scenario additional outcomes
    fig_additional_outcomes_prevented.add_trace(go.Scatter(x=selected_years, y=effective_additional_outcomes, mode='lines', name=f'Selected Scenario ({rcp_scenario}, {adaptation_effectiveness}%)', line=dict(color='blue')))
    
    # Shade the area between the graphs to represent mitigation and adaptation
    fig_additional_outcomes_prevented.add_trace(go.Scatter(
        x=selected_years + selected_years[::-1],
        y=worst_additional_outcomes + effective_additional_outcomes[::-1],
        fill='toself',
        fillcolor='rgba(0, 255, 0, 0.2)',
        line=dict(color='rgba(255, 255, 255, 0)'),
        showlegend=True,
        name='Mitigation'
    ))
    fig_additional_outcomes_prevented.add_trace(go.Scatter(
        x=selected_years + selected_years[::-1],
        y=zero_adaptation_outcomes + effective_additional_outcomes[::-1],
        fill='toself',
        fillcolor='rgba(0, 0, 255, 0.2)',
        line=dict(color='rgba(255, 255, 255, 0)'),
        showlegend=True,
        name='Adaptation'
    ))
    
    fig_additional_outcomes_prevented.update_layout(
        title=f"Additional {outcome} Outcomes Prevented by Mitigation and Adaptation from {time_period[0]} to {time_period[1]}",
        xaxis_title="Year",
        yaxis=dict(
            title="Additional Outcomes",
            titlefont=dict(color="#1f77b4"),
            tickfont=dict(color="#1f77b4")
        ),
        legend=dict(yanchor="bottom", y=-0.5, xanchor="center", x=0.5)
    )
    
    fig_costs = go.Figure()
    
    # Add costs
    fig_costs.add_trace(go.Scatter(x=selected_years, y=costs, mode='lines', name=f'Selected Scenario ({rcp_scenario}, {adaptation_effectiveness}%)', line=dict(color='blue')))
    
    # Add confidence interval shading for costs
    fig_costs.add_trace(go.Scatter(
        x=selected_years + selected_years[::-1],
        y=costs_upper + costs_lower[::-1],
        fill='toself',
        fillcolor='rgba(0, 0, 255, 0.2)',
        line=dict(color='rgba(255, 255, 255, 0)'),
        showlegend=False,
        name='Confidence Interval'
    ))
    
    fig_costs.update_layout(
        title=f"Costs Associated with Effective Additional {outcome} Outcomes Due to Heat from {time_period[0]} to {time_period[1]}",
        xaxis_title="Year",
        yaxis=dict(
            title="Costs (USD)",
            titlefont=dict(color="#1f77b4"),
            tickfont=dict(color="#1f77b4")
        ),
        legend=dict(yanchor="bottom", y=-0.5, xanchor="center", x=0.5)
    )
    
    fig_cost_savings = go.Figure()
    
    # Add worst-case scenario costs
    fig_cost_savings.add_trace(go.Scatter(x=selected_years, y=worst_costs, mode='lines', name='Worst Case (RCP 8.5, 0% Adaptation)', line=dict(color='red')))
    
    # Add selected scenario costs
    fig_cost_savings.add_trace(go.Scatter(x=selected_years, y=costs, mode='lines', name=f'Selected Scenario ({rcp_scenario}, {adaptation_effectiveness}%)', line=dict(color='blue')))
    
    # Shade the area between the graphs to represent mitigation and adaptation
    fig_cost_savings.add_trace(go.Scatter(
        x=selected_years + selected_years[::-1],
        y=worst_costs + costs[::-1],
        fill='toself',
        fillcolor='rgba(0, 255, 0, 0.2)',
        line=dict(color='rgba(255, 255, 255, 0)'),
        showlegend=True,
        name='Mitigation'
    ))
    fig_cost_savings.add_trace(go.Scatter(
        x=selected_years + selected_years[::-1],
        y=zero_adaptation_costs + costs[::-1],
        fill='toself',
        fillcolor='rgba(0, 0, 255, 0.2)',
        line=dict(color='rgba(255, 255, 255, 0)'),
        showlegend=True,
        name='Adaptation'
    ))
    
    fig_cost_savings.update_layout(
        title=f"Cost Savings by Comparing Worst Scenario (RCP 8.5, 0% Adaptation) and Selected Scenario from {time_period[0]} to {time_period[1]}",
        xaxis_title="Year",
        yaxis=dict(
            title="Costs (USD)",
            titlefont=dict(color="#1f77b4"),
            tickfont=dict(color="#1f77b4")
        ),
        legend=dict(yanchor="bottom", y=-0.5, xanchor="center", x=0.5)
    )
    
    return display_text, current_rate_text, initial_pregnancy_rate_text, adaptation_effectiveness_text, cost_per_case_text, attributable_fraction_text, total_additional_outcomes_text, total_cost_text, total_mitigation_savings_text, total_adaptation_savings_text, total_mitigation_prevented_text, total_adaptation_prevented_text, fig_outcomes, fig_additional_outcomes_prevented, fig_costs, fig_cost_savings

# Export Data Callback
@app.callback(
    Output("download-dataframe-csv", "data"),
    Input("export-button", "n_clicks"),
    prevent_initial_call=True,
)
def export_data(n_clicks):
    if n_clicks is None:
        return dash.no_update

    # Retrieve values from the update_dashboard function (need to be moved to global scope)
    outcome = 'Preterm Birth'  # or get dynamically if possible
    rcp_scenario = 'RCP 2.6'  # or get dynamically if possible
    population_growth_curve = 'Medium Growth'  # or get dynamically if possible
    pregnancy_growth_curve = 'Medium Growth'  # or get dynamically if possible
    time_period = [2024, 2030]  # or get dynamically if possible
    current_population = 7000000000  # or get dynamically if possible
    initial_pregnancy_rate = 0.02  # or get dynamically if possible
    adaptation_effectiveness = 50  # or get dynamically if possible
    
    selected_years, effective_additional_outcomes, costs, additional_outcomes, _, _, _, _ = calculate_additional_outcomes_and_costs(
        outcome, rcp_scenario, population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, adaptation_effectiveness
    )
    
    _, worst_effective_additional_outcomes, worst_costs, worst_additional_outcomes, _, _, _, _ = calculate_additional_outcomes_and_costs(
        outcome, 'RCP 8.5', population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, 0
    )
    
    _, zero_adaptation_outcomes, zero_adaptation_costs, _, _, _, _ = calculate_additional_outcomes_and_costs(
        outcome, rcp_scenario, population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, 0
    )
    
    mitigation_savings = [worst - selected for worst, selected in zip(worst_costs, costs)]
    adaptation_savings = [zero - selected for zero, selected in zip(zero_adaptation_costs, costs)]
    mitigation_prevented = [worst - selected for worst, selected in zip(worst_additional_outcomes, effective_additional_outcomes)]
    adaptation_prevented = [zero - selected for zero, selected in zip(zero_adaptation_outcomes, effective_additional_outcomes)]

    # Create a DataFrame with the current projections
    data = {
        'Year': selected_years,
        'Effective Additional Outcomes': effective_additional_outcomes,
        'Temperature Increase': calculate_temperature_increase_per_year(rcp_scenarios[rcp_scenario], time_period[0], time_period[1]),
        'Costs': costs,
        'Worst Case Costs': worst_costs,
        'Zero Adaptation Costs': zero_adaptation_costs,
        'Mitigation Savings': mitigation_savings,
        'Adaptation Savings': adaptation_savings,
        'Mitigation Prevented': mitigation_prevented,
        'Adaptation Prevented': adaptation_prevented
    }
    df = pd.DataFrame(data)

    return dcc.send_data_frame(df.to_csv, "projected_outcomes.csv")

# Step 5: Run the App
if __name__ == '__main__':
    app.run_server(debug=True)


Fetching population data for ABW...
Fetching population data for AFR...
An error occurred: 'NoneType' object is not iterable
---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
Cell In[6], line 74, in calculate_additional_outcomes_and_costs(
    outcome='Preterm Birth',
    rcp_scenario='RCP 2.6',
    population_growth_curve='Medium Growth',
    pregnancy_growth_curve='Medium Growth',
    time_period=[2024, 2030],
    current_population=None,
    initial_pregnancy_rate=0.02,
    adaptation_effectiveness=50
)
     71 selected_years = list(range(start_year, end_year + 1))
     73 # Calculate population growth
---> 74 population_growth_selected = calculate_population_growth(current_population, population_growth_rate, num_years)
        population_growth_rate = 0.02
        num_years = 7
        current_population = None
     76 # Calculate pregnancies without the effect of heat
     77 preg

In [11]:
#version 17 with updated error handling
import dash
from dash import dcc, html, Input, Output, State
import plotly.graph_objs as go
import pandas as pd
import numpy as np
import wbdata
from datetime import datetime

# Step 2: Prepare the Data
mnch_outcomes = {
    'Preterm Birth': {'odds_ratio': 1.04, 'current_rate': 0.1, 'cost_per_case': 1000, 'ci_lower': 1.03, 'ci_upper': 1.06},
    'Low Birth Weight': {'odds_ratio': 1.03, 'current_rate': 0.08, 'cost_per_case': 800, 'ci_lower': 1.02, 'ci_upper': 1.05},
    'Stillbirth': {'odds_ratio': 1.14, 'current_rate': 0.005, 'cost_per_case': 5000, 'ci_lower': 0.99, 'ci_upper': 1.32},
    'Congenital Anomalies': {'odds_ratio': 1.29, 'current_rate': 0.02, 'cost_per_case': 10000, 'ci_lower': 1.21, 'ci_upper': 1.38},
    'Gestational Diabetes Mellitus': {'odds_ratio': 1.28, 'current_rate': 0.05, 'cost_per_case': 2000, 'ci_lower': 1.05, 'ci_upper': 1.74},
    'Hypertensive Disorders': {'odds_ratio': 1.16, 'current_rate': 0.1, 'cost_per_case': 1500, 'ci_lower': 1.10, 'ci_upper': 1.22},
    'Prelabour Rupture of Membranes': {'odds_ratio': 1.63, 'current_rate': 0.02, 'cost_per_case': 1200, 'ci_lower': 1.23, 'ci_upper': 2.16},
    'Infections': {'odds_ratio': 1.15, 'current_rate': 0.02, 'cost_per_case': 800, 'ci_lower': 1.03, 'ci_upper': 1.40},
    'Placental Abruption': {'odds_ratio': 1.16, 'current_rate': 0.005, 'cost_per_case': 2500, 'ci_lower': 1.03, 'ci_upper': 1.40},
    'Neonatal Admissions': {'odds_ratio': 1.33, 'current_rate': 0.02, 'cost_per_case': 1500, 'ci_lower': 1.27, 'ci_upper': 1.38}
}

rcp_scenarios = {
    'RCP 2.6': 1.5,  # degrees Celsius increase by 2100
    'RCP 4.5': 2.4,
    'RCP 6.0': 3.0,
    'RCP 8.5': 4.3
}

population_growth_rates = {
    'Low Growth': 0.01,  # 1% annual growth
    'Medium Growth': 0.02,  # 2% annual growth
    'High Growth': 0.03  # 3% annual growth
}

pregnancy_growth_rates = {
    'High Reduction': -0.15,  # 15% reduction
    'Medium Reduction': -0.1,  # 10% reduction
    'Low Reduction': -0.05,  # 5% reduction
    'Low Growth': 0.01,  # 1% annual growth
    'Medium Growth': 0.02,  # 2% annual growth
    'High Growth': 0.03  # 3% annual growth
}

years = list(range(2024, 2051))

def calculate_population_growth(initial_population, growth_rate, num_years):
    return [initial_population * ((1 + growth_rate) ** year) for year in range(num_years)]

def calculate_pregnancies(population, pregnancy_rate):
    return [pop * pregnancy_rate for pop in population]

def calculate_temperature_increase_per_year(temp_increase_2100, start_year, end_year):
    total_years = 2100 - 2024  # Calculate total years from 2024 to 2100
    annual_increase = temp_increase_2100 / total_years
    return [annual_increase * (year - 2024) for year in range(start_year, end_year + 1)]

def calculate_additional_outcomes_and_costs(outcome, rcp_scenario, population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, adaptation_effectiveness):
    odds_ratio = mnch_outcomes[outcome]['odds_ratio']
    ci_lower = mnch_outcomes[outcome]['ci_lower']
    ci_upper = mnch_outcomes[outcome]['ci_upper']
    current_rate = mnch_outcomes[outcome]['current_rate']
    cost_per_case = mnch_outcomes[outcome]['cost_per_case']
    temp_increase_2100 = rcp_scenarios[rcp_scenario]
    population_growth_rate = population_growth_rates[population_growth_curve]
    pregnancy_growth_rate = pregnancy_growth_rates[pregnancy_growth_curve]
    
    start_year, end_year = time_period
    num_years = end_year - start_year + 1
    selected_years = list(range(start_year, end_year + 1))
    
    # Calculate population growth
    population_growth_selected = calculate_population_growth(current_population, population_growth_rate, num_years)
    
    # Calculate pregnancies without the effect of heat
    pregnancies = calculate_pregnancies(population_growth_selected, initial_pregnancy_rate * (1 + pregnancy_growth_rate))
    
    # Calculate baseline outcomes without the effect of heat
    baseline_outcomes = [preg * current_rate for preg in pregnancies]
    
    # Calculate the temperature increase per year
    temperature_increase_per_year = calculate_temperature_increase_per_year(temp_increase_2100, start_year, end_year)
    
    # Calculate projected outcomes with the effect of heat
    projected_rates = [current_rate * (odds_ratio ** temp_increase) for temp_increase in temperature_increase_per_year]
    projected_outcomes = [rate * preg for rate, preg in zip(projected_rates, pregnancies)]
    
    # Calculate additional outcomes due to the effect of heat
    additional_outcomes = [proj - base for proj, base in zip(projected_outcomes, baseline_outcomes)]
    
    # Apply adaptation effectiveness
    effective_additional_outcomes = [outcome * (1 - adaptation_effectiveness / 100) for outcome in additional_outcomes]
    
    # Calculate confidence intervals for effective additional outcomes
    effective_additional_outcomes_lower = [outcome * (1 - adaptation_effectiveness / 100) * ci_lower / odds_ratio for outcome in additional_outcomes]
    effective_additional_outcomes_upper = [outcome * (1 - adaptation_effectiveness / 100) * ci_upper / odds_ratio for outcome in additional_outcomes]
    
    # Calculate the total cost
    costs = [outcome * cost_per_case for outcome in effective_additional_outcomes]
    costs_lower = [outcome * cost_per_case for outcome in effective_additional_outcomes_lower]
    costs_upper = [outcome * cost_per_case for outcome in effective_additional_outcomes_upper]
    
    return selected_years, effective_additional_outcomes, costs, additional_outcomes, effective_additional_outcomes_lower, effective_additional_outcomes_upper, costs_lower, costs_upper

# Calculate Attributable Fraction
def calculate_attributable_fraction(odds_ratio):
    return (odds_ratio - 1) / odds_ratio

# Fetch population data from World Bank
def fetch_population_data(country_code):
    try:
        # Fetch the latest available population data
        print(f"Fetching population data for {country_code}...")
        population_data = wbdata.get_dataframe({'SP.POP.TOTL': 'population'}, country=country_code, convert_date=True)
        if population_data.empty:
            print(f"No population data available for {country_code}")
            return None
        population_data = population_data.reset_index()
        population_data['date'] = pd.to_datetime(population_data['date'])
        latest_population = population_data.loc[population_data['date'].idxmax()]
        return latest_population['population']
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Step 3: Create the Dash Layout
app = dash.Dash(__name__)

app.layout = html.Div([
    html.Link(rel='stylesheet', href='/assets/style.css'),
    html.H1("MNCH Outcomes Due to Heat"),
    html.H2("Climate Change Projection", className='subtitle'),
    html.Div(className='container', children=[
        html.Div(className='input-container', children=[
            html.Div(className='input-box', children=[
                html.H3("Select Country or Global"),
                dcc.Dropdown(
                    id='country-dropdown',
                    options=[{'label': 'Global', 'value': 'global'}] + [{'label': country['name'], 'value': country['id']} for country in wbdata.get_countries()],
                    value='global',
                    className='dropdown'
                )
            ]),
            html.Div(className='input-box', children=[
                html.H3("Current Population"),
                dcc.Input(
                    id='current-population-input',
                    type='number',
                    value=7000000000,
                    className='input',
                    disabled=True
                ),
                dcc.Checklist(
                    id='manual-input-checklist',
                    options=[{'label': 'Use Manual Input', 'value': 'manual'}],
                    value=[],
                    style={'margin-left': '10px'}
                )
            ]),
            html.Div(className='input-box', children=[
                html.H3("Select MNCH Outcome"),
                dcc.Dropdown(
                    id='mnch-outcome-dropdown',
                    options=[{'label': outcome, 'value': outcome} for outcome in mnch_outcomes.keys()],
                    value='Preterm Birth',
                    className='dropdown'
                )
            ]),
            html.Div(className='input-box', children=[
                html.H3("Select RCP Scenario"),
                dcc.Dropdown(
                    id='rcp-scenario-dropdown',
                    options=[{'label': scenario, 'value': scenario} for scenario in rcp_scenarios.keys()],
                    value='RCP 2.6',
                    className='dropdown'
                )
            ]),
            html.Div(className='input-box', children=[
                html.H3("Select Population Growth Curve"),
                dcc.Dropdown(
                    id='population-growth-dropdown',
                    options=[{'label': growth, 'value': growth} for growth in population_growth_rates.keys()],
                    value='Medium Growth',
                    className='dropdown'
                )
            ]),
            html.Div(className='input-box', children=[
                html.H3("Select Pregnancy Growth Rate"),
                dcc.Dropdown(
                    id='pregnancy-growth-dropdown',
                    options=[{'label': growth, 'value': growth} for growth in pregnancy_growth_rates.keys()],
                    value='Medium Growth',
                    className='dropdown'
                )
            ]),
            html.Div(className='input-box', children=[
                html.H3("Initial Pregnancy Rate"),
                dcc.Input(
                    id='initial-pregnancy-rate-input',
                    type='number',
                    value=0.02,  # 2% initial pregnancy rate
                    step=0.001,
                    className='input'
                )
            ])
        ]),
        html.Div(className='slider-box', children=[
            html.H3("Adaptation Effectiveness (%)"),
            dcc.Slider(
                id='adaptation-effectiveness-slider',
                min=0,
                max=100,
                step=1,
                value=50,
                marks={i: f'{i}%' for i in range(0, 101, 10)},
                className='slider'
            )
        ]),
        html.Div(className='slider-box', children=[
            html.H3("Select Time Period"),
            dcc.RangeSlider(
                id='time-period-slider',
                min=2024,
                max=2050,
                step=1,
                marks={year: str(year) for year in years},
                value=[2024, 2030],
                className='slider'
            )
        ]),
        html.H2("Outputs", className='subtitle'),
        html.Div(className='key-indicators', children=[
            html.Div(className='key-indicator', children=[
                html.H2("Odds Ratio per 1°C increase"),
                html.P(id='odds-and-rate-display')
            ]),
            html.Div(className='key-indicator', children=[
                html.H2("Current Rate"),
                html.P(id='current-rate-display')
            ]),
            html.Div(className='key-indicator', children=[
                html.H2("Initial Pregnancy Rate"),
                html.P(id='initial-pregnancy-rate-display')
            ]),
            html.Div(className='key-indicator', children=[
                html.H2("Adaptation Effectiveness"),
                html.P(id='adaptation-effectiveness-display')
            ]),
            html.Div(className='key-indicator', children=[
                html.H2("Cost per Case"),
                html.P(id='cost-per-case-display')
            ]),
            html.Div(className='key-indicator', children=[
                html.H2("Attributable Fraction"),
                html.P(id='attributable-fraction-display')
            ]),
            html.Div(className='key-indicator', children=[
                html.H2("Total Additional Outcomes"),
                html.P(id='total-additional-outcomes-display')
            ]),
            html.Div(className='key-indicator', children=[
                html.H2("Total Cost"),
                html.P(id='total-cost-display')
            ]),
            html.Div(className='key-indicator', children=[
                html.H2("Total Mitigation Savings"),
                html.P(id='total-mitigation-savings-display')
            ]),
            html.Div(className='key-indicator', children=[
                html.H2("Total Adaptation Savings"),
                html.P(id='total-adaptation-savings-display')
            ]),
            html.Div(className='key-indicator', children=[
                html.H2("Total Mitigation Prevented"),
                html.P(id='total-mitigation-prevented-display')
            ]),
            html.Div(className='key-indicator', children=[
                html.H2("Total Adaptation Prevented"),
                html.P(id='total-adaptation-prevented-display')
            ])
        ]),
        html.Div(className='graph-container', children=[
            dcc.Graph(id='outcome-projection-graph'),
            dcc.Graph(id='additional-outcomes-prevented-graph'),
            dcc.Graph(id='cost-projection-graph'),
            dcc.Graph(id='cost-savings-graph')
        ]),
        html.Button("Export Data", id="export-button", className='button'),
        dcc.Download(id="download-dataframe-csv")
    ])
])

# Step 4: Add Interactivity
@app.callback(
    Output('odds-and-rate-display', 'children'),
    Output('current-rate-display', 'children'),
    Output('initial-pregnancy-rate-display', 'children'),
    Output('adaptation-effectiveness-display', 'children'),
    Output('cost-per-case-display', 'children'),
    Output('attributable-fraction-display', 'children'),
    Output('total-additional-outcomes-display', 'children'),
    Output('total-cost-display', 'children'),
    Output('total-mitigation-savings-display', 'children'),
    Output('total-adaptation-savings-display', 'children'),
    Output('total-mitigation-prevented-display', 'children'),
    Output('total-adaptation-prevented-display', 'children'),
    Output('outcome-projection-graph', 'figure'),
    Output('additional-outcomes-prevented-graph', 'figure'),
    Output('cost-projection-graph', 'figure'),
    Output('cost-savings-graph', 'figure'),
    Output('current-population-input', 'value'),
    Output('current-population-input', 'disabled'),
    Input('mnch-outcome-dropdown', 'value'),
    Input('rcp-scenario-dropdown', 'value'),
    Input('population-growth-dropdown', 'value'),
    Input('pregnancy-growth-dropdown', 'value'),
    Input('time-period-slider', 'value'),
    Input('current-population-input', 'value'),
    Input('initial-pregnancy-rate-input', 'value'),
    Input('adaptation-effectiveness-slider', 'value'),
    Input('country-dropdown', 'value'),
    Input('manual-input-checklist', 'value')
)
def update_dashboard(outcome, rcp_scenario, population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, adaptation_effectiveness, country, manual_input):
    odds_ratio = mnch_outcomes[outcome]['odds_ratio']
    
    # Fetch population data if manual input is not selected
    if 'manual' not in manual_input:
        if country == 'global':
            current_population = 7000000000
        else:
            current_population = fetch_population_data(country)
            if current_population is None:
                current_population = 7000000000

    # Calculate Attributable Fraction
    attributable_fraction = calculate_attributable_fraction(odds_ratio)
    
    # Calculate selected scenario outcomes and costs
    selected_years, effective_additional_outcomes, costs, additional_outcomes, effective_additional_outcomes_lower, effective_additional_outcomes_upper, costs_lower, costs_upper = calculate_additional_outcomes_and_costs(
        outcome, rcp_scenario, population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, adaptation_effectiveness
    )
    
    # Calculate worst scenario (RCP 8.5, 0% adaptation effectiveness) outcomes and costs
    _, worst_effective_additional_outcomes, worst_costs, worst_additional_outcomes, _, _, _, _ = calculate_additional_outcomes_and_costs(
        outcome, 'RCP 8.5', population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, 0
    )
    
    # Calculate 0% adaptation scenario outcomes and costs for comparison
    _, zero_adaptation_outcomes, zero_adaptation_costs, _, _, _, _, _ = calculate_additional_outcomes_and_costs(
        outcome, rcp_scenario, population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, 0
    )
    
    # Calculate cost savings
    mitigation_savings = [worst - selected for worst, selected in zip(worst_costs, costs)]
    adaptation_savings = [zero - selected for zero, selected in zip(zero_adaptation_costs, costs)]
    
    # Calculate additional outcomes prevented
    mitigation_prevented = [worst - selected for worst, selected in zip(worst_additional_outcomes, effective_additional_outcomes)]
    adaptation_prevented = [zero - selected for zero, selected in zip(zero_adaptation_outcomes, effective_additional_outcomes)]
    
    total_additional_outcomes = sum(effective_additional_outcomes)
    total_cost = sum(costs)
    total_mitigation_savings = sum(mitigation_savings)
    total_adaptation_savings = sum(adaptation_savings)
    total_mitigation_prevented = sum(mitigation_prevented)
    total_adaptation_prevented = sum(adaptation_prevented)
    
    display_text = f"{mnch_outcomes[outcome]['odds_ratio']} (95% CI: {mnch_outcomes[outcome]['ci_lower']} - {mnch_outcomes[outcome]['ci_upper']})"
    current_rate_text = f"{mnch_outcomes[outcome]['current_rate']}"
    initial_pregnancy_rate_text = f"{initial_pregnancy_rate}"
    adaptation_effectiveness_text = f"{adaptation_effectiveness}%"
    cost_per_case_text = f"${mnch_outcomes[outcome]['cost_per_case']}"
    attributable_fraction_text = f"{round(attributable_fraction * 100, 2)}%"
    total_additional_outcomes_text = f"{int(round(total_additional_outcomes))}"
    total_cost_text = f"${round(total_cost, 2):,}"
    total_mitigation_savings_text = f"${round(total_mitigation_savings, 2):,}"
    total_adaptation_savings_text = f"${round(total_adaptation_savings, 2):,}"
    total_mitigation_prevented_text = f"{int(round(total_mitigation_prevented))}"
    total_adaptation_prevented_text = f"{int(round(total_adaptation_prevented))}"
    
    fig_outcomes = go.Figure()
    
    # Add additional outcomes
    fig_outcomes.add_trace(go.Scatter(x=selected_years, y=effective_additional_outcomes, mode='lines', name=f'Selected Scenario ({rcp_scenario}, {adaptation_effectiveness}%)', line=dict(color='blue')))
    
    # Add confidence interval shading for effective additional outcomes
    fig_outcomes.add_trace(go.Scatter(
        x=selected_years + selected_years[::-1],
        y=effective_additional_outcomes_upper + effective_additional_outcomes_lower[::-1],
        fill='toself',
        fillcolor='rgba(0, 0, 255, 0.2)',
        line=dict(color='rgba(255, 255, 255, 0)'),
        showlegend=False,
        name='Confidence Interval'
    ))
    
    # Add temperature increase
    temperature_increase_per_year = calculate_temperature_increase_per_year(rcp_scenarios[rcp_scenario], time_period[0], time_period[1])
    fig_outcomes.add_trace(go.Scatter(x=selected_years, y=temperature_increase_per_year, mode='lines', name='Temperature Increase', yaxis='y2', line=dict(color='red')))
    
    # Update layout for multiple y-axes
    fig_outcomes.update_layout(
        title=f"Effective Additional {outcome} Outcomes Due to Heat and Temperature Increase from {time_period[0]} to {time_period[1]}",
        xaxis_title="Year",
        yaxis=dict(
            title="Effective Additional Outcomes",
            titlefont=dict(color="#1f77b4"),
            tickfont=dict(color="#1f77b4")
        ),
        yaxis2=dict(
            title="Temperature Increase (°C)",
            titlefont=dict(color="#ff7f0e"),
            tickfont=dict(color="#ff7f0e"),
            overlaying='y',
            side='right'
        ),
        legend=dict(yanchor="bottom", y=-0.6, xanchor="center", x=0.5)
    )
    
    fig_additional_outcomes_prevented = go.Figure()
    
    # Add worst-case scenario additional outcomes
    fig_additional_outcomes_prevented.add_trace(go.Scatter(x=selected_years, y=worst_additional_outcomes, mode='lines', name='Worst Case (RCP 8.5, 0% Adaptation)', line=dict(color='red')))
    
    # Add selected scenario additional outcomes
    fig_additional_outcomes_prevented.add_trace(go.Scatter(x=selected_years, y=effective_additional_outcomes, mode='lines', name=f'Selected Scenario ({rcp_scenario}, {adaptation_effectiveness}%)', line=dict(color='blue')))
    
    # Shade the area between the graphs to represent mitigation and adaptation
    fig_additional_outcomes_prevented.add_trace(go.Scatter(
        x=selected_years + selected_years[::-1],
        y=worst_additional_outcomes + effective_additional_outcomes[::-1],
        fill='toself',
        fillcolor='rgba(0, 255, 0, 0.2)',
        line=dict(color='rgba(255, 255, 255, 0)'),
        showlegend=True,
        name='Mitigation'
    ))
    fig_additional_outcomes_prevented.add_trace(go.Scatter(
        x=selected_years + selected_years[::-1],
        y=zero_adaptation_outcomes + effective_additional_outcomes[::-1],
        fill='toself',
        fillcolor='rgba(0, 0, 255, 0.2)',
        line=dict(color='rgba(255, 255, 255, 0)'),
        showlegend=True,
        name='Adaptation'
    ))
    
    fig_additional_outcomes_prevented.update_layout(
        title=f"Additional {outcome} Outcomes Prevented by Mitigation and Adaptation from {time_period[0]} to {time_period[1]}",
        xaxis_title="Year",
        yaxis=dict(
            title="Additional Outcomes",
            titlefont=dict(color="#1f77b4"),
            tickfont=dict(color="#1f77b4")
        ),
        legend=dict(yanchor="bottom", y=-0.5, xanchor="center", x=0.5)
    )
    
    fig_costs = go.Figure()
    
    # Add costs
    fig_costs.add_trace(go.Scatter(x=selected_years, y=costs, mode='lines', name=f'Selected Scenario ({rcp_scenario}, {adaptation_effectiveness}%)', line=dict(color='blue')))
    
    # Add confidence interval shading for costs
    fig_costs.add_trace(go.Scatter(
        x=selected_years + selected_years[::-1],
        y=costs_upper + costs_lower[::-1],
        fill='toself',
        fillcolor='rgba(0, 0, 255, 0.2)',
        line=dict(color='rgba(255, 255, 255, 0)'),
        showlegend=False,
        name='Confidence Interval'
    ))
    
    fig_costs.update_layout(
        title=f"Costs Associated with Effective Additional {outcome} Outcomes Due to Heat from {time_period[0]} to {time_period[1]}",
        xaxis_title="Year",
        yaxis=dict(
            title="Costs (USD)",
            titlefont=dict(color="#1f77b4"),
            tickfont=dict(color="#1f77b4")
        ),
        legend=dict(yanchor="bottom", y=-0.5, xanchor="center", x=0.5)
    )
    
    fig_cost_savings = go.Figure()
    
    # Add worst-case scenario costs
    fig_cost_savings.add_trace(go.Scatter(x=selected_years, y=worst_costs, mode='lines', name='Worst Case (RCP 8.5, 0% Adaptation)', line=dict(color='red')))
    
    # Add selected scenario costs
    fig_cost_savings.add_trace(go.Scatter(x=selected_years, y=costs, mode='lines', name=f'Selected Scenario ({rcp_scenario}, {adaptation_effectiveness}%)', line=dict(color='blue')))
    
    # Shade the area between the graphs to represent mitigation and adaptation
    fig_cost_savings.add_trace(go.Scatter(
        x=selected_years + selected_years[::-1],
        y=worst_costs + costs[::-1],
        fill='toself',
        fillcolor='rgba(0, 255, 0, 0.2)',
        line=dict(color='rgba(255, 255, 255, 0)'),
        showlegend=True,
        name='Mitigation'
    ))
    fig_cost_savings.add_trace(go.Scatter(
        x=selected_years + selected_years[::-1],
        y=zero_adaptation_costs + costs[::-1],
        fill='toself',
        fillcolor='rgba(0, 0, 255, 0.2)',
        line=dict(color='rgba(255, 255, 255, 0)'),
        showlegend=True,
        name='Adaptation'
    ))
    
    fig_cost_savings.update_layout(
        title=f"Cost Savings by Comparing Worst Scenario (RCP 8.5, 0% Adaptation) and Selected Scenario from {time_period[0]} to {time_period[1]}",
        xaxis_title="Year",
        yaxis=dict(
            title="Costs (USD)",
            titlefont=dict(color="#1f77b4"),
            tickfont=dict(color="#1f77b4")
        ),
        legend=dict(yanchor="bottom", y=-0.5, xanchor="center", x=0.5)
    )
    
    return (
        display_text, 
        current_rate_text, 
        initial_pregnancy_rate_text, 
        adaptation_effectiveness_text, 
        cost_per_case_text, 
        attributable_fraction_text, 
        total_additional_outcomes_text, 
        total_cost_text, 
        total_mitigation_savings_text, 
        total_adaptation_savings_text, 
        total_mitigation_prevented_text, 
        total_adaptation_prevented_text, 
        fig_outcomes, 
        fig_additional_outcomes_prevented, 
        fig_costs, 
        fig_cost_savings, 
        current_population, 
        'manual' not in manual_input
    )

# Export Data Callback
@app.callback(
    Output("download-dataframe-csv", "data"),
    Input("export-button", "n_clicks"),
    prevent_initial_call=True,
)
def export_data(n_clicks):
    if n_clicks is None:
        return dash.no_update

    # Retrieve values from the update_dashboard function (need to be moved to global scope)
    outcome = 'Preterm Birth'  # or get dynamically if possible
    rcp_scenario = 'RCP 2.6'  # or get dynamically if possible
    population_growth_curve = 'Medium Growth'  # or get dynamically if possible
    pregnancy_growth_curve = 'Medium Growth'  # or get dynamically if possible
    time_period = [2024, 2030]  # or get dynamically if possible
    current_population = 7000000000  # or get dynamically if possible
    initial_pregnancy_rate = 0.02  # or get dynamically if possible
    adaptation_effectiveness = 50  # or get dynamically if possible
    
    selected_years, effective_additional_outcomes, costs, additional_outcomes, _, _, _, _ = calculate_additional_outcomes_and_costs(
        outcome, rcp_scenario, population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, adaptation_effectiveness
    )
    
    _, worst_effective_additional_outcomes, worst_costs, worst_additional_outcomes, _, _, _, _ = calculate_additional_outcomes_and_costs(
        outcome, 'RCP 8.5', population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, 0
    )
    
    _, zero_adaptation_outcomes, zero_adaptation_costs, _, _, _, _ = calculate_additional_outcomes_and_costs(
        outcome, rcp_scenario, population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, 0
    )
    
    mitigation_savings = [worst - selected for worst, selected in zip(worst_costs, costs)]
    adaptation_savings = [zero - selected for zero, selected in zip(zero_adaptation_costs, costs)]
    mitigation_prevented = [worst - selected for worst, selected in zip(worst_additional_outcomes, effective_additional_outcomes)]
    adaptation_prevented = [zero - selected for zero, selected in zip(zero_adaptation_outcomes, effective_additional_outcomes)]

    # Create a DataFrame with the current projections
    data = {
        'Year': selected_years,
        'Effective Additional Outcomes': effective_additional_outcomes,
        'Temperature Increase': calculate_temperature_increase_per_year(rcp_scenarios[rcp_scenario], time_period[0], time_period[1]),
        'Costs': costs,
        'Worst Case Costs': worst_costs,
        'Zero Adaptation Costs': zero_adaptation_costs,
        'Mitigation Savings': mitigation_savings,
        'Adaptation Savings': adaptation_savings,
        'Mitigation Prevented': mitigation_prevented,
        'Adaptation Prevented': adaptation_prevented
    }
    df = pd.DataFrame(data)

    return dcc.send_data_frame(df.to_csv, "projected_outcomes.csv")

# Step 5: Run the App
if __name__ == '__main__':
    app.run_server(debug=True)


Fetching population data for ABW...
An error occurred: got an unexpected keyword argument 'convert_date'
Fetching population data for AFG...
An error occurred: got an unexpected keyword argument 'convert_date'
Fetching population data for CME...
An error occurred: got an unexpected keyword argument 'convert_date'


In [ ]:
import dash
from dash import dcc, html, Input, Output, State
import plotly.graph_objs as go
import pandas as pd
import numpy as np
import wbdata
from datetime import datetime

# Step 2: Prepare the Data
mnch_outcomes = {
    'Preterm Birth': {'odds_ratio': 1.04, 'current_rate': 0.1, 'cost_per_case': 1000, 'ci_lower': 1.03, 'ci_upper': 1.06},
    'Low Birth Weight': {'odds_ratio': 1.03, 'current_rate': 0.08, 'cost_per_case': 800, 'ci_lower': 1.02, 'ci_upper': 1.05},
    'Stillbirth': {'odds_ratio': 1.14, 'current_rate': 0.005, 'cost_per_case': 5000, 'ci_lower': 0.99, 'ci_upper': 1.32},
    'Congenital Anomalies': {'odds_ratio': 1.29, 'current_rate': 0.02, 'cost_per_case': 10000, 'ci_lower': 1.21, 'ci_upper': 1.38},
    'Gestational Diabetes Mellitus': {'odds_ratio': 1.28, 'current_rate': 0.05, 'cost_per_case': 2000, 'ci_lower': 1.05, 'ci_upper': 1.74},
    'Hypertensive Disorders': {'odds_ratio': 1.16, 'current_rate': 0.1, 'cost_per_case': 1500, 'ci_lower': 1.10, 'ci_upper': 1.22},
    'Prelabour Rupture of Membranes': {'odds_ratio': 1.63, 'current_rate': 0.02, 'cost_per_case': 1200, 'ci_lower': 1.23, 'ci_upper': 2.16},
    'Infections': {'odds_ratio': 1.15, 'current_rate': 0.02, 'cost_per_case': 800, 'ci_lower': 1.03, 'ci_upper': 1.40},
    'Placental Abruption': {'odds_ratio': 1.16, 'current_rate': 0.005, 'cost_per_case': 2500, 'ci_lower': 1.03, 'ci_upper': 1.40},
    'Neonatal Admissions': {'odds_ratio': 1.33, 'current_rate': 0.02, 'cost_per_case': 1500, 'ci_lower': 1.27, 'ci_upper': 1.38}
}

rcp_scenarios = {
    'RCP 2.6': 1.5,  # degrees Celsius increase by 2100
    'RCP 4.5': 2.4,
    'RCP 6.0': 3.0,
    'RCP 8.5': 4.3
}

population_growth_rates = {
    'Low Growth': 0.01,  # 1% annual growth
    'Medium Growth': 0.02,  # 2% annual growth
    'High Growth': 0.03  # 3% annual growth
}

pregnancy_growth_rates = {
    'High Reduction': -0.15,  # 15% reduction
    'Medium Reduction': -0.1,  # 10% reduction
    'Low Reduction': -0.05,  # 5% reduction
    'Low Growth': 0.01,  # 1% annual growth
    'Medium Growth': 0.02,  # 2% annual growth
    'High Growth': 0.03  # 3% annual growth
}

years = list(range(2024, 2051))

def calculate_population_growth(initial_population, growth_rate, num_years):
    return [initial_population * ((1 + growth_rate) ** year) for year in range(num_years)]

def calculate_pregnancies(population, pregnancy_rate):
    return [pop * pregnancy_rate for pop in population]

def calculate_temperature_increase_per_year(temp_increase_2100, start_year, end_year):
    total_years = 2100 - 2024  # Calculate total years from 2024 to 2100
    annual_increase = temp_increase_2100 / total_years
    return [annual_increase * (year - 2024) for year in range(start_year, end_year + 1)]

def calculate_additional_outcomes_and_costs(outcome, rcp_scenario, population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, adaptation_effectiveness):
    odds_ratio = mnch_outcomes[outcome]['odds_ratio']
    ci_lower = mnch_outcomes[outcome]['ci_lower']
    ci_upper = mnch_outcomes[outcome]['ci_upper']
    current_rate = mnch_outcomes[outcome]['current_rate']
    cost_per_case = mnch_outcomes[outcome]['cost_per_case']
    temp_increase_2100 = rcp_scenarios[rcp_scenario]
    population_growth_rate = population_growth_rates[population_growth_curve]
    pregnancy_growth_rate = pregnancy_growth_rates[pregnancy_growth_curve]
    
    start_year, end_year = time_period
    num_years = end_year - start_year + 1
    selected_years = list(range(start_year, end_year + 1))
    
    # Calculate population growth
    population_growth_selected = calculate_population_growth(current_population, population_growth_rate, num_years)
    
    # Calculate pregnancies without the effect of heat
    pregnancies = calculate_pregnancies(population_growth_selected, initial_pregnancy_rate * (1 + pregnancy_growth_rate))
    
    # Calculate baseline outcomes without the effect of heat
    baseline_outcomes = [preg * current_rate for preg in pregnancies]
    
    # Calculate the temperature increase per year
    temperature_increase_per_year = calculate_temperature_increase_per_year(temp_increase_2100, start_year, end_year)
    
    # Calculate projected outcomes with the effect of heat
    projected_rates = [current_rate * (odds_ratio ** temp_increase) for temp_increase in temperature_increase_per_year]
    projected_outcomes = [rate * preg for rate, preg in zip(projected_rates, pregnancies)]
    
    # Calculate additional outcomes due to the effect of heat
    additional_outcomes = [proj - base for proj, base in zip(projected_outcomes, baseline_outcomes)]
    
    # Apply adaptation effectiveness
    effective_additional_outcomes = [outcome * (1 - adaptation_effectiveness / 100) for outcome in additional_outcomes]
    
    # Calculate confidence intervals for effective additional outcomes
    effective_additional_outcomes_lower = [outcome * (1 - adaptation_effectiveness / 100) * ci_lower / odds_ratio for outcome in additional_outcomes]
    effective_additional_outcomes_upper = [outcome * (1 - adaptation_effectiveness / 100) * ci_upper / odds_ratio for outcome in additional_outcomes]
    
    # Calculate the total cost
    costs = [outcome * cost_per_case for outcome in effective_additional_outcomes]
    costs_lower = [outcome * cost_per_case for outcome in effective_additional_outcomes_lower]
    costs_upper = [outcome * cost_per_case for outcome in effective_additional_outcomes_upper]
    
    return selected_years, effective_additional_outcomes, costs, additional_outcomes, effective_additional_outcomes_lower, effective_additional_outcomes_upper, costs_lower, costs_upper

# Calculate Attributable Fraction
def calculate_attributable_fraction(odds_ratio):
    return (odds_ratio - 1) / odds_ratio

# Fetch population data from World Bank
def fetch_population_data(country_code):
    try:
        # Fetch the latest available population data
        print(f"Fetching population data for {country_code}...")
        population_data = wbdata.get_dataframe({'SP.POP.TOTL': 'population'}, country=country_code)
        if population_data.empty:
            print(f"No population data available for {country_code}")
            return None
        population_data = population_data.reset_index()
        population_data['date'] = pd.to_datetime(population_data['date'])
        latest_population = population_data.loc[population_data['date'].idxmax()]
        return latest_population['population']
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Step 3: Create the Dash Layout
app = dash.Dash(__name__)

app.layout = html.Div([
    html.Link(rel='stylesheet', href='/assets/style.css'),
    html.H1("MNCH Outcomes Due to Heat"),
    html.H2("Climate Change Projection", className='subtitle'),
    html.Div(className='container', children=[
        html.Div(className='input-container', children=[
            html.Div(className='input-box', children=[
                html.H3("Select Country or Global"),
                dcc.Dropdown(
                    id='country-dropdown',
                    options=[{'label': 'Global', 'value': 'global'}] + [{'label': country['name'], 'value': country['id']} for country in wbdata.get_countries()],
                    value='global',
                    className='dropdown'
                )
            ]),
            html.Div(className='input-box', children=[
                html.H3("Current Population"),
                dcc.Input(
                    id='current-population-input',
                    type='number',
                    value=7000000000,
                    className='input',
                    disabled=True
                ),
                dcc.Checklist(
                    id='manual-input-checklist',
                    options=[{'label': 'Use Manual Input', 'value': 'manual'}],
                    value=[],
                    style={'margin-left': '10px'}
                )
            ]),
            html.Div(className='input-box', children=[
                html.H3("Select MNCH Outcome"),
                dcc.Dropdown(
                    id='mnch-outcome-dropdown',
                    options=[{'label': outcome, 'value': outcome} for outcome in mnch_outcomes.keys()],
                    value='Preterm Birth',
                    className='dropdown'
                )
            ]),
            html.Div(className='input-box', children=[
                html.H3("Select RCP Scenario"),
                dcc.Dropdown(
                    id='rcp-scenario-dropdown',
                    options=[{'label': scenario, 'value': scenario} for scenario in rcp_scenarios.keys()],
                    value='RCP 2.6',
                    className='dropdown'
                )
            ]),
            html.Div(className='input-box', children=[
                html.H3("Select Population Growth Curve"),
                dcc.Dropdown(
                    id='population-growth-dropdown',
                    options=[{'label': growth, 'value': growth} for growth in population_growth_rates.keys()],
                    value='Medium Growth',
                    className='dropdown'
                )
            ]),
            html.Div(className='input-box', children=[
                html.H3("Select Pregnancy Growth Rate"),
                dcc.Dropdown(
                    id='pregnancy-growth-dropdown',
                    options=[{'label': growth, 'value': growth} for growth in pregnancy_growth_rates.keys()],
                    value='Medium Growth',
                    className='dropdown'
                )
            ]),
            html.Div(className='input-box', children=[
                html.H3("Initial Pregnancy Rate"),
                dcc.Input(
                    id='initial-pregnancy-rate-input',
                    type='number',
                    value=0.02,  # 2% initial pregnancy rate
                    step=0.001,
                    className='input'
                )
            ])
        ]),
        html.Div(className='slider-box', children=[
            html.H3("Adaptation Effectiveness (%)"),
            dcc.Slider(
                id='adaptation-effectiveness-slider',
                min=0,
                max=100,
                step=1,
                value=50,
                marks={i: f'{i}%' for i in range(0, 101, 10)},
                className='slider'
            )
        ]),
        html.Div(className='slider-box', children=[
            html.H3("Select Time Period"),
            dcc.RangeSlider(
                id='time-period-slider',
                min=2024,
                max=2050,
                step=1,
                marks={year: str(year) for year in years},
                value=[2024, 2030],
                className='slider'
            )
        ]),
        html.H2("Outputs", className='subtitle'),
        html.Div(className='key-indicators', children=[
            html.Div(className='key-indicator', children=[
                html.H2("Odds Ratio per 1°C increase"),
                html.P(id='odds-and-rate-display')
            ]),
            html.Div(className='key-indicator', children=[
                html.H2("Current Rate"),
                html.P(id='current-rate-display')
            ]),
            html.Div(className='key-indicator', children=[
                html.H2("Initial Pregnancy Rate"),
                html.P(id='initial-pregnancy-rate-display')
            ]),
            html.Div(className='key-indicator', children=[
                html.H2("Adaptation Effectiveness"),
                html.P(id='adaptation-effectiveness-display')
            ]),
            html.Div(className='key-indicator', children=[
                html.H2("Cost per Case"),
                html.P(id='cost-per-case-display')
            ]),
            html.Div(className='key-indicator', children=[
                html.H2("Attributable Fraction"),
                html.P(id='attributable-fraction-display')
            ]),
            html.Div(className='key-indicator', children=[
                html.H2("Total Additional Outcomes"),
                html.P(id='total-additional-outcomes-display')
            ]),
            html.Div(className='key-indicator', children=[
                html.H2("Total Cost"),
                html.P(id='total-cost-display')
            ]),
            html.Div(className='key-indicator', children=[
                html.H2("Total Mitigation Savings"),
                html.P(id='total-mitigation-savings-display')
            ]),
            html.Div(className='key-indicator', children=[
                html.H2("Total Adaptation Savings"),
                html.P(id='total-adaptation-savings-display')
            ]),
            html.Div(className='key-indicator', children=[
                html.H2("Total Mitigation Prevented"),
                html.P(id='total-mitigation-prevented-display')
            ]),
            html.Div(className='key-indicator', children=[
                html.H2("Total Adaptation Prevented"),
                html.P(id='total-adaptation-prevented-display')
            ])
        ]),
        html.Div(className='graph-container', children=[
            dcc.Graph(id='outcome-projection-graph'),
            dcc.Graph(id='additional-outcomes-prevented-graph'),
            dcc.Graph(id='cost-projection-graph'),
            dcc.Graph(id='cost-savings-graph')
        ]),
        html.Button("Export Data", id="export-button", className='button'),
        dcc.Download(id="download-dataframe-csv")
    ])
])

# Step 4: Add Interactivity
@app.callback(
    Output('odds-and-rate-display', 'children'),
    Output('current-rate-display', 'children'),
    Output('initial-pregnancy-rate-display', 'children'),
    Output('adaptation-effectiveness-display', 'children'),
    Output('cost-per-case-display', 'children'),
    Output('attributable-fraction-display', 'children'),
    Output('total-additional-outcomes-display', 'children'),
    Output('total-cost-display', 'children'),
    Output('total-mitigation-savings-display', 'children'),
    Output('total-adaptation-savings-display', 'children'),
    Output('total-mitigation-prevented-display', 'children'),
    Output('total-adaptation-prevented-display', 'children'),
    Output('outcome-projection-graph', 'figure'),
    Output('additional-outcomes-prevented-graph', 'figure'),
    Output('cost-projection-graph', 'figure'),
    Output('cost-savings-graph', 'figure'),
    Output('current-population-input', 'value'),
    Output('current-population-input', 'disabled'),
    Input('mnch-outcome-dropdown', 'value'),
    Input('rcp-scenario-dropdown', 'value'),
    Input('population-growth-dropdown', 'value'),
    Input('pregnancy-growth-dropdown', 'value'),
    Input('time-period-slider', 'value'),
    Input('current-population-input', 'value'),
    Input('initial-pregnancy-rate-input', 'value'),
    Input('adaptation-effectiveness-slider', 'value'),
    Input('country-dropdown', 'value'),
    Input('manual-input-checklist', 'value')
)
def update_dashboard(outcome, rcp_scenario, population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, adaptation_effectiveness, country, manual_input):
    odds_ratio = mnch_outcomes[outcome]['odds_ratio']
    
    # Fetch population data if manual input is not selected
    if 'manual' not in manual_input:
        if country == 'global':
            current_population = 7000000000
        else:
            current_population = fetch_population_data(country)
            if current_population is None:
                current_population = 7000000000

    # Calculate Attributable Fraction
    attributable_fraction = calculate_attributable_fraction(odds_ratio)
    
    # Calculate selected scenario outcomes and costs
    selected_years, effective_additional_outcomes, costs, additional_outcomes, effective_additional_outcomes_lower, effective_additional_outcomes_upper, costs_lower, costs_upper = calculate_additional_outcomes_and_costs(
        outcome, rcp_scenario, population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, adaptation_effectiveness
    )
    
    # Calculate worst scenario (RCP 8.5, 0% adaptation effectiveness) outcomes and costs
    _, worst_effective_additional_outcomes, worst_costs, worst_additional_outcomes, _, _, _, _ = calculate_additional_outcomes_and_costs(
        outcome, 'RCP 8.5', population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, 0
    )
    
    # Calculate 0% adaptation scenario outcomes and costs for comparison
    _, zero_adaptation_outcomes, zero_adaptation_costs, _, _, _, _, _ = calculate_additional_outcomes_and_costs(
        outcome, rcp_scenario, population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, 0
    )
    
    # Calculate cost savings
    mitigation_savings = [worst - selected for worst, selected in zip(worst_costs, costs)]
    adaptation_savings = [zero - selected for zero, selected in zip(zero_adaptation_costs, costs)]
    
    # Calculate additional outcomes prevented
    mitigation_prevented = [worst - selected for worst, selected in zip(worst_additional_outcomes, effective_additional_outcomes)]
    adaptation_prevented = [zero - selected for zero, selected in zip(zero_adaptation_outcomes, effective_additional_outcomes)]
    
    total_additional_outcomes = sum(effective_additional_outcomes)
    total_cost = sum(costs)
    total_mitigation_savings = sum(mitigation_savings)
    total_adaptation_savings = sum(adaptation_savings)
    total_mitigation_prevented = sum(mitigation_prevented)
    total_adaptation_prevented = sum(adaptation_prevented)
    
    display_text = f"{mnch_outcomes[outcome]['odds_ratio']} (95% CI: {mnch_outcomes[outcome]['ci_lower']} - {mnch_outcomes[outcome]['ci_upper']})"
    current_rate_text = f"{mnch_outcomes[outcome]['current_rate']}"
    initial_pregnancy_rate_text = f"{initial_pregnancy_rate}"
    adaptation_effectiveness_text = f"{adaptation_effectiveness}%"
    cost_per_case_text = f"${mnch_outcomes[outcome]['cost_per_case']}"
    attributable_fraction_text = f"{round(attributable_fraction * 100, 2)}%"
    total_additional_outcomes_text = f"{int(round(total_additional_outcomes))}"
    total_cost_text = f"${round(total_cost, 2):,}"
    total_mitigation_savings_text = f"${round(total_mitigation_savings, 2):,}"
    total_adaptation_savings_text = f"${round(total_adaptation_savings, 2):,}"
    total_mitigation_prevented_text = f"{int(round(total_mitigation_prevented))}"
    total_adaptation_prevented_text = f"{int(round(total_adaptation_prevented))}"
    
    fig_outcomes = go.Figure()
    
    # Add additional outcomes
    fig_outcomes.add_trace(go.Scatter(x=selected_years, y=effective_additional_outcomes, mode='lines', name=f'Selected Scenario ({rcp_scenario}, {adaptation_effectiveness}%)', line=dict(color='blue')))
    
    # Add confidence interval shading for effective additional outcomes
    fig_outcomes.add_trace(go.Scatter(
        x=selected_years + selected_years[::-1],
        y=effective_additional_outcomes_upper + effective_additional_outcomes_lower[::-1],
        fill='toself',
        fillcolor='rgba(0, 0, 255, 0.2)',
        line=dict(color='rgba(255, 255, 255, 0)'),
        showlegend=False,
        name='Confidence Interval'
    ))
    
    # Add temperature increase
    temperature_increase_per_year = calculate_temperature_increase_per_year(rcp_scenarios[rcp_scenario], time_period[0], time_period[1])
    fig_outcomes.add_trace(go.Scatter(x=selected_years, y=temperature_increase_per_year, mode='lines', name='Temperature Increase', yaxis='y2', line=dict(color='red')))
    
    # Update layout for multiple y-axes
    fig_outcomes.update_layout(
        title=f"Effective Additional {outcome} Outcomes Due to Heat and Temperature Increase from {time_period[0]} to {time_period[1]}",
        xaxis_title="Year",
        yaxis=dict(
            title="Effective Additional Outcomes",
            titlefont=dict(color="#1f77b4"),
            tickfont=dict(color="#1f77b4")
        ),
        yaxis2=dict(
            title="Temperature Increase (°C)",
            titlefont=dict(color="#ff7f0e"),
            tickfont=dict(color="#ff7f0e"),
            overlaying='y',
            side='right'
        ),
        legend=dict(yanchor="bottom", y=-0.6, xanchor="center", x=0.5)
    )
    
    fig_additional_outcomes_prevented = go.Figure()
    
    # Add worst-case scenario additional outcomes
    fig_additional_outcomes_prevented.add_trace(go.Scatter(x=selected_years, y=worst_additional_outcomes, mode='lines', name='Worst Case (RCP 8.5, 0% Adaptation)', line=dict(color='red')))
    
    # Add selected scenario additional outcomes
    fig_additional_outcomes_prevented.add_trace(go.Scatter(x=selected_years, y=effective_additional_outcomes, mode='lines', name=f'Selected Scenario ({rcp_scenario}, {adaptation_effectiveness}%)', line=dict(color='blue')))
    
    # Shade the area between the graphs to represent mitigation and adaptation
    fig_additional_outcomes_prevented.add_trace(go.Scatter(
        x=selected_years + selected_years[::-1],
        y=worst_additional_outcomes + effective_additional_outcomes[::-1],
        fill='toself',
        fillcolor='rgba(0, 255, 0, 0.2)',
        line=dict(color='rgba(255, 255, 255, 0)'),
        showlegend=True,
        name='Mitigation'
    ))
    fig_additional_outcomes_prevented.add_trace(go.Scatter(
        x=selected_years + selected_years[::-1],
        y=zero_adaptation_outcomes + effective_additional_outcomes[::-1],
        fill='toself',
        fillcolor='rgba(0, 0, 255, 0.2)',
        line=dict(color='rgba(255, 255, 255, 0)'),
        showlegend=True,
        name='Adaptation'
    ))
    
    fig_additional_outcomes_prevented.update_layout(
        title=f"Additional {outcome} Outcomes Prevented by Mitigation and Adaptation from {time_period[0]} to {time_period[1]}",
        xaxis_title="Year",
        yaxis=dict(
            title="Additional Outcomes",
            titlefont=dict(color="#1f77b4"),
            tickfont=dict(color="#1f77b4")
        ),
        legend=dict(yanchor="bottom", y=-0.5, xanchor="center", x=0.5)
    )
    
    fig_costs = go.Figure()
    
    # Add costs
    fig_costs.add_trace(go.Scatter(x=selected_years, y=costs, mode='lines', name=f'Selected Scenario ({rcp_scenario}, {adaptation_effectiveness}%)', line=dict(color='blue')))
    
    # Add confidence interval shading for costs
    fig_costs.add_trace(go.Scatter(
        x=selected_years + selected_years[::-1],
        y=costs_upper + costs_lower[::-1],
        fill='toself',
        fillcolor='rgba(0, 0, 255, 0.2)',
        line=dict(color='rgba(255, 255, 255, 0)'),
        showlegend=False,
        name='Confidence Interval'
    ))
    
    fig_costs.update_layout(
        title=f"Costs Associated with Effective Additional {outcome} Outcomes Due to Heat from {time_period[0]} to {time_period[1]}",
        xaxis_title="Year",
        yaxis=dict(
            title="Costs (USD)",
            titlefont=dict(color="#1f77b4"),
            tickfont=dict(color="#1f77b4")
        ),
        legend=dict(yanchor="bottom", y=-0.5, xanchor="center", x=0.5)
    )
    
    fig_cost_savings = go.Figure()
    
    # Add worst-case scenario costs
    fig_cost_savings.add_trace(go.Scatter(x=selected_years, y=worst_costs, mode='lines', name='Worst Case (RCP 8.5, 0% Adaptation)', line=dict(color='red')))
    
    # Add selected scenario costs
    fig_cost_savings.add_trace(go.Scatter(x=selected_years, y=costs, mode='lines', name=f'Selected Scenario ({rcp_scenario}, {adaptation_effectiveness}%)', line=dict(color='blue')))
    
    # Shade the area between the graphs to represent mitigation and adaptation
    fig_cost_savings.add_trace(go.Scatter(
        x=selected_years + selected_years[::-1],
        y=worst_costs + costs[::-1],
        fill='toself',
        fillcolor='rgba(0, 255, 0, 0.2)',
        line=dict(color='rgba(255, 255, 255, 0)'),
        showlegend=True,
        name='Mitigation'
    ))
    fig_cost_savings.add_trace(go.Scatter(
        x=selected_years + selected_years[::-1],
        y=zero_adaptation_costs + costs[::-1],
        fill='toself',
        fillcolor='rgba(0, 0, 255, 0.2)',
        line=dict(color='rgba(255, 255, 255, 0)'),
        showlegend=True,
        name='Adaptation'
    ))
    
    fig_cost_savings.update_layout(
        title=f"Cost Savings by Comparing Worst Scenario (RCP 8.5, 0% Adaptation) and Selected Scenario from {time_period[0]} to {time_period[1]}",
        xaxis_title="Year",
        yaxis=dict(
            title="Costs (USD)",
            titlefont=dict(color="#1f77b4"),
            tickfont=dict(color="#1f77b4")
        ),
        legend=dict(yanchor="bottom", y=-0.5, xanchor="center", x=0.5)
    )
    
    return (
        display_text, 
        current_rate_text, 
        initial_pregnancy_rate_text, 
        adaptation_effectiveness_text, 
        cost_per_case_text, 
        attributable_fraction_text, 
        total_additional_outcomes_text, 
        total_cost_text, 
        total_mitigation_savings_text, 
        total_adaptation_savings_text, 
        total_mitigation_prevented_text, 
        total_adaptation_prevented_text, 
        fig_outcomes, 
        fig_additional_outcomes_prevented, 
        fig_costs, 
        fig_cost_savings, 
        current_population, 
        'manual' not in manual_input
    )

# Export Data Callback
@app.callback(
    Output("download-dataframe-csv", "data"),
    Input("export-button", "n_clicks"),
    prevent_initial_call=True,
)
def export_data(n_clicks):
    if n_clicks is None:
        return dash.no_update

    # Retrieve values from the update_dashboard function (need to be moved to global scope)
    outcome = 'Preterm Birth'  # or get dynamically if possible
    rcp_scenario = 'RCP 2.6'  # or get dynamically if possible
    population_growth_curve = 'Medium Growth'  # or get dynamically if possible
    pregnancy_growth_curve = 'Medium Growth'  # or get dynamically if possible
    time_period = [2024, 2030]  # or get dynamically if possible
    current_population = 7000000000  # or get dynamically if possible
    initial_pregnancy_rate = 0.02  # or get dynamically if possible
    adaptation_effectiveness = 50  # or get dynamically if possible
    
    selected_years, effective_additional_outcomes, costs, additional_outcomes, _, _, _, _ = calculate_additional_outcomes_and_costs(
        outcome, rcp_scenario, population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, adaptation_effectiveness
    )
    
    _, worst_effective_additional_outcomes, worst_costs, worst_additional_outcomes, _, _, _, _ = calculate_additional_outcomes_and_costs(
        outcome, 'RCP 8.5', population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, 0
    )
    
    _, zero_adaptation_outcomes, zero_adaptation_costs, _, _, _, _ = calculate_additional_outcomes_and_costs(
        outcome, rcp_scenario, population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, 0
    )
    
    mitigation_savings = [worst - selected for worst, selected in zip(worst_costs, costs)]
    adaptation_savings = [zero - selected for zero, selected in zip(zero_adaptation_costs, costs)]
    mitigation_prevented = [worst - selected for worst, selected in zip(worst_additional_outcomes, effective_additional_outcomes)]
    adaptation_prevented = [zero - selected for zero, selected in zip(zero_adaptation_outcomes, effective_additional_outcomes)]

    # Create a DataFrame with the current projections
    data = {
        'Year': selected_years,
        'Effective Additional Outcomes': effective_additional_outcomes,
        'Temperature Increase': calculate_temperature_increase_per_year(rcp_scenarios[rcp_scenario], time_period[0], time_period[1]),
        'Costs': costs,
        'Worst Case Costs': worst_costs,
        'Zero Adaptation Costs': zero_adaptation_costs,
        'Mitigation Savings': mitigation_savings,
        'Adaptation Savings': adaptation_savings,
        'Mitigation Prevented': mitigation_prevented,
        'Adaptation Prevented': adaptation_prevented
    }
    df = pd.DataFrame(data)

    return dcc.send_data_frame(df.to_csv, "projected_outcomes.csv")

# Step 5: Run the App
if __name__ == '__main__':
    app.run(debug=True)


ERROR:__main__:Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/opt/anaconda3/envs/nics-env/lib/python3.12/site-packages/flask/app.py", line 917, in full_dispatch_request
    rv = self.dispatch_request()
         ^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/nics-env/lib/python3.12/site-packages/flask/app.py", line 902, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)  # type: ignore[no-any-return]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/nics-env/lib/python3.12/site-packages/asgiref/sync.py", line 254, in __call__
    return call_result.result()
           ^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/nics-env/lib/python3.12/concurrent/futures/_base.py", line 449, in result
    return self.__get_result()
           ^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/nics-env/lib/python3.12/concurrent/futures/_base.py", line 401, in

In [10]:
#version 18 with correction to population input
import dash
from dash import dcc, html, Input, Output, State
import plotly.graph_objs as go
import pandas as pd
import numpy as np
import wbdata
from datetime import datetime

# Step 2: Prepare the Data
mnch_outcomes = {
    'Gestational Diabetes': {'odds_ratio': 1.07, 'current_rate': 0.05, 'cost_per_case': 2000, 'ci_lower': 1.01, 'ci_upper': 1.54},
    'Hypertension during Pregnancy': {'odds_ratio': 1.12, 'current_rate': 0.1, 'cost_per_case': 1500, 'ci_lower': 1.03, 'ci_upper': 1.21},
    'Maternal Admission': {'odds_ratio': 1.01, 'current_rate': 0.02, 'cost_per_case': 1000, 'ci_lower': 1.00, 'ci_upper': 1.03},
    'Infections': {'odds_ratio': 1.29, 'current_rate': 0.02, 'cost_per_case': 800, 'ci_lower': 1.05, 'ci_upper': 1.58},
    'Prelabour Rupture of Membranes': {'odds_ratio': 1.08, 'current_rate': 0.02, 'cost_per_case': 1200, 'ci_lower': 1.07, 'ci_upper': 1.09},
    'Antenatal Bleeding': {'odds_ratio': 1.16, 'current_rate': 0.005, 'cost_per_case': 2500, 'ci_lower': 1.13, 'ci_upper': 1.40},
    'Cardiovascular Event': {'odds_ratio': 1.11, 'current_rate': 0.01, 'cost_per_case': 3000, 'ci_lower': 1.06, 'ci_upper': 1.15},
    'Stillbirth': {'odds_ratio': 1.14, 'current_rate': 0.005, 'cost_per_case': 5000, 'ci_lower': 0.99, 'ci_upper': 1.32},
    'Congenital Anomalies': {'odds_ratio': 1.13, 'current_rate': 0.02, 'cost_per_case': 10000, 'ci_lower': 0.99, 'ci_upper': 1.85},
    'Spontaneous Abortion': {'odds_ratio': 1.31, 'current_rate': 0.01, 'cost_per_case': 5000, 'ci_lower': 0.99, 'ci_upper': 3.30},
    'Non-reassuring Fetal Status': {'odds_ratio': 1.21, 'current_rate': 0.02, 'cost_per_case': 800, 'ci_lower': 1.12, 'ci_upper': 1.32},
    'Composite Outcome': {'odds_ratio': 1.42, 'current_rate': 0.1, 'cost_per_case': 2000, 'ci_lower': 1.00, 'ci_upper': 2.03},
    'Preterm Birth': {'odds_ratio': 1.04, 'current_rate': 0.1, 'cost_per_case': 32000, 'ci_lower': 1.03, 'ci_upper': 1.06},
    'Low Birth Weight': {'odds_ratio': 1.13, 'current_rate': 0.08, 'cost_per_case': 800, 'ci_lower': 1.02, 'ci_upper': 1.26},
    'Small for Gestational Age': {'odds_ratio': 1.10, 'current_rate': 0.02, 'cost_per_case': 800, 'ci_lower': 1.02, 'ci_upper': 1.18},
    'Neonatal Admission': {'odds_ratio': 1.22, 'current_rate': 0.02, 'cost_per_case': 1500, 'ci_lower': 1.02, 'ci_upper': 1.43},
    'Neonatal Morbidity': {'odds_ratio': 1.04, 'current_rate': 0.02, 'cost_per_case': 800, 'ci_lower': 1.03, 'ci_upper': 1.06},
    'Obstetric Complication': {'odds_ratio': 1.05, 'current_rate': 0.1, 'cost_per_case': 1500, 'ci_lower': 1.03, 'ci_upper': 1.06}
}

rcp_scenarios = {
    'RCP 2.6': 1.5,  # degrees Celsius increase by 2100
    'RCP 4.5': 2.4,
    'RCP 6.0': 3.0,
    'RCP 8.5': 4.3
}

population_growth_rates = {
    'Low Growth': 0.01,  # 1% annual growth
    'Medium Growth': 0.02,  # 2% annual growth
    'High Growth': 0.03  # 3% annual growth
}

pregnancy_growth_rates = {
    'High Reduction': -0.15,  # 15% reduction
    'Medium Reduction': -0.1,  # 10% reduction
    'Low Reduction': -0.05,  # 5% reduction
    'Low Growth': 0.01,  # 1% annual growth
    'Medium Growth': 0.02,  # 2% annual growth
    'High Growth': 0.03  # 3% annual growth
}

years = list(range(2024, 2051))

def calculate_population_growth(initial_population, growth_rate, num_years):
    return [initial_population * ((1 + growth_rate) ** year) for year in range(num_years)]

def calculate_pregnancies(population, pregnancy_rate):
    return [pop * pregnancy_rate for pop in population]

def calculate_temperature_increase_per_year(temp_increase_2100, start_year, end_year):
    total_years = 2100 - 2024  # Calculate total years from 2024 to 2100
    annual_increase = temp_increase_2100 / total_years
    return [annual_increase * (year - 2024) for year in range(start_year, end_year + 1)]

def calculate_additional_outcomes_and_costs(outcome, rcp_scenario, population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, adaptation_effectiveness):
    odds_ratio = mnch_outcomes[outcome]['odds_ratio']
    ci_lower = mnch_outcomes[outcome]['ci_lower']
    ci_upper = mnch_outcomes[outcome]['ci_upper']
    current_rate = mnch_outcomes[outcome]['current_rate']
    cost_per_case = mnch_outcomes[outcome]['cost_per_case']
    temp_increase_2100 = rcp_scenarios[rcp_scenario]
    population_growth_rate = population_growth_rates[population_growth_curve]
    pregnancy_growth_rate = pregnancy_growth_rates[pregnancy_growth_curve]
    
    start_year, end_year = time_period
    num_years = end_year - start_year + 1
    selected_years = list(range(start_year, end_year + 1))
    
    # Calculate population growth
    population_growth_selected = calculate_population_growth(current_population, population_growth_rate, num_years)
    
    # Calculate pregnancies without the effect of heat
    pregnancies = calculate_pregnancies(population_growth_selected, initial_pregnancy_rate * (1 + pregnancy_growth_rate))
    
    # Calculate baseline outcomes without the effect of heat
    baseline_outcomes = [preg * current_rate for preg in pregnancies]
    
    # Calculate the temperature increase per year
    temperature_increase_per_year = calculate_temperature_increase_per_year(temp_increase_2100, start_year, end_year)
    
    # Calculate projected outcomes with the effect of heat
    projected_rates = [current_rate * (odds_ratio ** temp_increase) for temp_increase in temperature_increase_per_year]
    projected_outcomes = [rate * preg for rate, preg in zip(projected_rates, pregnancies)]
    
    # Calculate additional outcomes due to the effect of heat
    additional_outcomes = [proj - base for proj, base in zip(projected_outcomes, baseline_outcomes)]
    
    # Apply adaptation effectiveness
    effective_additional_outcomes = [outcome * (1 - adaptation_effectiveness / 100) for outcome in additional_outcomes]
    
    # Calculate confidence intervals for effective additional outcomes
    effective_additional_outcomes_lower = [outcome * (1 - adaptation_effectiveness / 100) * ci_lower / odds_ratio for outcome in additional_outcomes]
    effective_additional_outcomes_upper = [outcome * (1 - adaptation_effectiveness / 100) * ci_upper / odds_ratio for outcome in additional_outcomes]
    
    # Calculate the total cost
    costs = [outcome * cost_per_case for outcome in effective_additional_outcomes]
    costs_lower = [outcome * cost_per_case for outcome in effective_additional_outcomes_lower]
    costs_upper = [outcome * cost_per_case for outcome in effective_additional_outcomes_upper]
    
    return selected_years, effective_additional_outcomes, costs, additional_outcomes, effective_additional_outcomes_lower, effective_additional_outcomes_upper, costs_lower, costs_upper

# Calculate Attributable Fraction
def calculate_attributable_fraction(odds_ratio):
    return (odds_ratio - 1) / odds_ratio

# Fetch population data from World Bank
def fetch_population_data(country_code):
    try:
        # Fetch the latest available population data
        print(f"Fetching population data for {country_code}...")
        population_data = wbdata.get_dataframe({'SP.POP.TOTL': 'population'}, country=country_code)
        if population_data.empty:
            print(f"No population data available for {country_code}")
            return None
        population_data = population_data.reset_index()
        population_data['date'] = pd.to_datetime(population_data['date'])
        latest_population = population_data.loc[population_data['date'].idxmax()]
        return latest_population['population']
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Step 3: Create the Dash Layout
app = dash.Dash(__name__)

app.layout = html.Div([
    html.Link(rel='stylesheet', href='/assets/style.css'),
    html.H1("MNCH Outcomes Due to Heat"),
    html.H2("Climate Change Projection", className='subtitle'),
    html.Div(className='container', children=[
        html.Div(className='input-container', children=[
            html.Div(className='input-box', children=[
                html.H3("Select Country or Global"),
                dcc.Dropdown(
                    id='country-dropdown',
                    options=[{'label': 'Global', 'value': 'global'}] + [{'label': country['name'], 'value': country['id']} for country in wbdata.get_countries()],
                    value='global',
                    className='dropdown'
                )
            ]),
            html.Div(className='input-box', children=[
                html.H3("Current Population"),
                dcc.Input(
                    id='current-population-input',
                    type='number',
                    value=7000000000,
                    className='input'
                ),
                dcc.Checklist(
                    id='manual-input-checklist',
                    options=[{'label': 'Use Manual Input', 'value': 'manual'}],
                    value=[],
                    style={'margin-left': '10px'}
                )
            ]),
            html.Div(className='input-box', children=[
                html.H3("Select MNCH Outcome"),
                dcc.Dropdown(
                    id='mnch-outcome-dropdown',
                    options=[{'label': outcome, 'value': outcome} for outcome in mnch_outcomes.keys()],
                    value='Preterm Birth',
                    className='dropdown'
                )
            ]),
            html.Div(className='input-box', children=[
                html.H3("Select RCP Scenario"),
                dcc.Dropdown(
                    id='rcp-scenario-dropdown',
                    options=[{'label': scenario, 'value': scenario} for scenario in rcp_scenarios.keys()],
                    value='RCP 2.6',
                    className='dropdown'
                )
            ]),
            html.Div(className='input-box', children=[
                html.H3("Select Population Growth Curve"),
                dcc.Dropdown(
                    id='population-growth-dropdown',
                    options=[{'label': growth, 'value': growth} for growth in population_growth_rates.keys()],
                    value='Medium Growth',
                    className='dropdown'
                )
            ]),
            html.Div(className='input-box', children=[
                html.H3("Select Pregnancy Growth Rate"),
                dcc.Dropdown(
                    id='pregnancy-growth-dropdown',
                    options=[{'label': growth, 'value': growth} for growth in pregnancy_growth_rates.keys()],
                    value='Medium Growth',
                    className='dropdown'
                )
            ]),
            html.Div(className='input-box', children=[
                html.H3("Initial Pregnancy Rate"),
                dcc.Input(
                    id='initial-pregnancy-rate-input',
                    type='number',
                    value=0.02,  # 2% initial pregnancy rate
                    step=0.001,
                    className='input'
                )
            ])
        ]),
        html.Div(className='slider-box', children=[
            html.H3("Adaptation Effectiveness (%)"),
            dcc.Slider(
                id='adaptation-effectiveness-slider',
                min=0,
                max=100,
                step=1,
                value=50,
                marks={i: f'{i}%' for i in range(0, 101, 10)},
                className='slider'
            )
        ]),
        html.Div(className='slider-box', children=[
            html.H3("Select Time Period"),
            dcc.RangeSlider(
                id='time-period-slider',
                min=2024,
                max=2050,
                step=1,
                marks={year: str(year) for year in years},
                value=[2024, 2030],
                className='slider'
            )
        ]),
        html.H2("Outputs", className='subtitle'),
        html.Div(className='key-indicators', children=[
            html.Div(className='key-indicator', children=[
                html.H2("Odds Ratio per 1°C increase"),
                html.P(id='odds-and-rate-display')
            ]),
            html.Div(className='key-indicator', children=[
                html.H2("Current Rate"),
                html.P(id='current-rate-display')
            ]),
            html.Div(className='key-indicator', children=[
                html.H2("Initial Pregnancy Rate"),
                html.P(id='initial-pregnancy-rate-display')
            ]),
            html.Div(className='key-indicator', children=[
                html.H2("Adaptation Effectiveness"),
                html.P(id='adaptation-effectiveness-display')
            ]),
            html.Div(className='key-indicator', children=[
                html.H2("Cost per Case"),
                html.P(id='cost-per-case-display')
            ]),
            html.Div(className='key-indicator', children=[
                html.H2("Attributable Fraction"),
                html.P(id='attributable-fraction-display')
            ]),
            html.Div(className='key-indicator', children=[
                html.H2("Total Additional Outcomes"),
                html.P(id='total-additional-outcomes-display')
            ]),
            html.Div(className='key-indicator', children=[
                html.H2("Total Cost"),
                html.P(id='total-cost-display')
            ]),
            html.Div(className='key-indicator', children=[
                html.H2("Total Mitigation Savings"),
                html.P(id='total-mitigation-savings-display')
            ]),
            html.Div(className='key-indicator', children=[
                html.H2("Total Adaptation Savings"),
                html.P(id='total-adaptation-savings-display')
            ]),
            html.Div(className='key-indicator', children=[
                html.H2("Total Mitigation Prevented"),
                html.P(id='total-mitigation-prevented-display')
            ]),
            html.Div(className='key-indicator', children=[
                html.H2("Total Adaptation Prevented"),
                html.P(id='total-adaptation-prevented-display')
            ])
        ]),
        html.Div(className='graph-container', children=[
            dcc.Graph(id='outcome-projection-graph'),
            dcc.Graph(id='additional-outcomes-prevented-graph'),
            dcc.Graph(id='cost-projection-graph'),
            dcc.Graph(id='cost-savings-graph')
        ]),
        html.Button("Export Data", id="export-button", className='button'),
        dcc.Download(id="download-dataframe-csv")
    ])
])

# Step 4: Add Interactivity

@app.callback(
    Output('current-population-input', 'value'),
    Output('current-population-input', 'disabled'),
    Input('country-dropdown', 'value'),
    Input('manual-input-checklist', 'value')
)
def update_population_input(country, manual_input):
    if 'manual' in manual_input:
        return dash.no_update, False
    if country == 'global':
        return 7000000000, True
    population = fetch_population_data(country)
    if population is None:
        population = 7000000000
    return population, True

@app.callback(
    Output('odds-and-rate-display', 'children'),
    Output('current-rate-display', 'children'),
    Output('initial-pregnancy-rate-display', 'children'),
    Output('adaptation-effectiveness-display', 'children'),
    Output('cost-per-case-display', 'children'),
    Output('attributable-fraction-display', 'children'),
    Output('total-additional-outcomes-display', 'children'),
    Output('total-cost-display', 'children'),
    Output('total-mitigation-savings-display', 'children'),
    Output('total-adaptation-savings-display', 'children'),
    Output('total-mitigation-prevented-display', 'children'),
    Output('total-adaptation-prevented-display', 'children'),
    Output('outcome-projection-graph', 'figure'),
    Output('additional-outcomes-prevented-graph', 'figure'),
    Output('cost-projection-graph', 'figure'),
    Output('cost-savings-graph', 'figure'),
    Input('mnch-outcome-dropdown', 'value'),
    Input('rcp-scenario-dropdown', 'value'),
    Input('population-growth-dropdown', 'value'),
    Input('pregnancy-growth-dropdown', 'value'),
    Input('time-period-slider', 'value'),
    Input('current-population-input', 'value'),
    Input('initial-pregnancy-rate-input', 'value'),
    Input('adaptation-effectiveness-slider', 'value'),
    Input('country-dropdown', 'value'),
    Input('manual-input-checklist', 'value')
)
def update_dashboard(outcome, rcp_scenario, population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, adaptation_effectiveness, country, manual_input):
    odds_ratio = mnch_outcomes[outcome]['odds_ratio']
    
    # Fetch population data if manual input is not selected
    if 'manual' not in manual_input:
        if country == 'global':
            current_population = 7000000000
        else:
            current_population = fetch_population_data(country)
            if current_population is None:
                current_population = 7000000000

    # Calculate Attributable Fraction
    attributable_fraction = calculate_attributable_fraction(odds_ratio)
    
    # Calculate selected scenario outcomes and costs
    selected_years, effective_additional_outcomes, costs, additional_outcomes, effective_additional_outcomes_lower, effective_additional_outcomes_upper, costs_lower, costs_upper = calculate_additional_outcomes_and_costs(
        outcome, rcp_scenario, population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, adaptation_effectiveness
    )
    
    # Calculate worst scenario (RCP 8.5, 0% adaptation effectiveness) outcomes and costs
    _, worst_effective_additional_outcomes, worst_costs, worst_additional_outcomes, _, _, _, _ = calculate_additional_outcomes_and_costs(
        outcome, 'RCP 8.5', population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, 0
    )
    
    # Calculate 0% adaptation scenario outcomes and costs for comparison
    _, zero_adaptation_outcomes, zero_adaptation_costs, _, _, _, _, _ = calculate_additional_outcomes_and_costs(
        outcome, rcp_scenario, population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, 0
    )
    
    # Calculate cost savings
    mitigation_savings = [worst - selected for worst, selected in zip(worst_costs, costs)]
    adaptation_savings = [zero - selected for zero, selected in zip(zero_adaptation_costs, costs)]
    
    # Calculate additional outcomes prevented
    mitigation_prevented = [worst - selected for worst, selected in zip(worst_additional_outcomes, effective_additional_outcomes)]
    adaptation_prevented = [zero - selected for zero, selected in zip(zero_adaptation_outcomes, effective_additional_outcomes)]
    
    total_additional_outcomes = sum(effective_additional_outcomes)
    total_cost = sum(costs)
    total_mitigation_savings = sum(mitigation_savings)
    total_adaptation_savings = sum(adaptation_savings)
    total_mitigation_prevented = sum(mitigation_prevented)
    total_adaptation_prevented = sum(adaptation_prevented)
    
    display_text = f"{mnch_outcomes[outcome]['odds_ratio']} (95% CI: {mnch_outcomes[outcome]['ci_lower']} - {mnch_outcomes[outcome]['ci_upper']})"
    current_rate_text = f"{mnch_outcomes[outcome]['current_rate']*100}%"
    initial_pregnancy_rate_text = f"{initial_pregnancy_rate*100}%" #convert to percentage
    adaptation_effectiveness_text = f"{adaptation_effectiveness}%"
    cost_per_case_text = f"${mnch_outcomes[outcome]['cost_per_case']}"
    attributable_fraction_text = f"{round(attributable_fraction * 100, 2)}%"
    total_additional_outcomes_text = f"{int(round(total_additional_outcomes))}"
    total_cost_text = f"${round(total_cost, 2):,}"
    total_mitigation_savings_text = f"${round(total_mitigation_savings, 2):,}"
    total_adaptation_savings_text = f"${round(total_adaptation_savings, 2):,}"
    total_mitigation_prevented_text = f"{int(round(total_mitigation_prevented))}"
    total_adaptation_prevented_text = f"{int(round(total_adaptation_prevented))}"
    
    fig_outcomes = go.Figure()
    
    # Add additional outcomes
    fig_outcomes.add_trace(go.Scatter(x=selected_years, y=effective_additional_outcomes, mode='lines', name=f'Selected Scenario ({rcp_scenario}, {adaptation_effectiveness}%)', line=dict(color='blue')))
    
    # Add confidence interval shading for effective additional outcomes
    fig_outcomes.add_trace(go.Scatter(
        x=selected_years + selected_years[::-1],
        y=effective_additional_outcomes_upper + effective_additional_outcomes_lower[::-1],
        fill='toself',
        fillcolor='rgba(0, 0, 255, 0.2)',
        line=dict(color='rgba(255, 255, 255, 0)'),
        showlegend=False,
        name='Confidence Interval'
    ))
    
    # Add temperature increase
    temperature_increase_per_year = calculate_temperature_increase_per_year(rcp_scenarios[rcp_scenario], time_period[0], time_period[1])
    fig_outcomes.add_trace(go.Scatter(x=selected_years, y=temperature_increase_per_year, mode='lines', name='Temperature Increase', yaxis='y2', line=dict(color='red')))
    
    # Update layout for multiple y-axes
    fig_outcomes.update_layout(
        title=f"Effective Additional {outcome} Outcomes Due to Heat and Temperature Increase from {time_period[0]} to {time_period[1]}",
        xaxis_title="Year",
        yaxis=dict(
            title="Effective Additional Outcomes",
            tickfont=dict(color="#1f77b4")
        ),
        yaxis2=dict(
            title="Temperature Increase (°C)",
            tickfont=dict(color="#ff7f0e"),
            overlaying='y',
            side='right'
        ),
        legend=dict(yanchor="bottom", y=-0.7, xanchor="center", x=0.5)
    )
    
    fig_additional_outcomes_prevented = go.Figure()
    
    # Add worst-case scenario additional outcomes
    fig_additional_outcomes_prevented.add_trace(go.Scatter(x=selected_years, y=worst_additional_outcomes, mode='lines', name='Worst Case (RCP 8.5, 0% Adaptation)', line=dict(color='red')))
    
    # Add selected scenario additional outcomes
    fig_additional_outcomes_prevented.add_trace(go.Scatter(x=selected_years, y=effective_additional_outcomes, mode='lines', name=f'Selected Scenario ({rcp_scenario}, {adaptation_effectiveness}%)', line=dict(color='blue')))
    
    # Shade the area between the graphs to represent mitigation and adaptation
    fig_additional_outcomes_prevented.add_trace(go.Scatter(
        x=selected_years + selected_years[::-1],
        y=worst_additional_outcomes + effective_additional_outcomes[::-1],
        fill='toself',
        fillcolor='rgba(0, 255, 0, 0.2)',
        line=dict(color='rgba(255, 255, 255, 0)'),
        showlegend=True,
        name='Mitigation'
    ))
    fig_additional_outcomes_prevented.add_trace(go.Scatter(
        x=selected_years + selected_years[::-1],
        y=zero_adaptation_outcomes + effective_additional_outcomes[::-1],
        fill='toself',
        fillcolor='rgba(0, 0, 255, 0.2)',
        line=dict(color='rgba(255, 255, 255, 0)'),
        showlegend=True,
        name='Adaptation'
    ))
    
    fig_additional_outcomes_prevented.update_layout(
        title=f"Additional {outcome} Outcomes Prevented by Mitigation and Adaptation from {time_period[0]} to {time_period[1]}",
        xaxis_title="Year",
        yaxis=dict(
            title="Additional Outcomes",
            tickfont=dict(color="#1f77b4")
        ),
        legend=dict(yanchor="bottom", y=-0.7, xanchor="center", x=0.5)
    )
    
    fig_costs = go.Figure()
    
    # Add costs
    fig_costs.add_trace(go.Scatter(x=selected_years, y=costs, mode='lines', name=f'Selected Scenario ({rcp_scenario}, {adaptation_effectiveness}%)', line=dict(color='blue')))
    
    # Add confidence interval shading for costs
    fig_costs.add_trace(go.Scatter(
        x=selected_years + selected_years[::-1],
        y=costs_upper + costs_lower[::-1],
        fill='toself',
        fillcolor='rgba(0, 0, 255, 0.2)',
        line=dict(color='rgba(255, 255, 255, 0)'),
        showlegend=False,
        name='Confidence Interval'
    ))
    
    fig_costs.update_layout(
        title=f"Costs Associated with Effective Additional {outcome} Outcomes Due to Heat from {time_period[0]} to {time_period[1]}",
        xaxis_title="Year",
        yaxis=dict(
            title="Costs (USD)",
            tickfont=dict(color="#1f77b4")
        ),
        legend=dict(yanchor="bottom", y=-0.7, xanchor="center", x=0.5)
    )
    
    fig_cost_savings = go.Figure()
    
    # Add worst-case scenario costs
    fig_cost_savings.add_trace(go.Scatter(x=selected_years, y=worst_costs, mode='lines', name='Worst Case (RCP 8.5, 0% Adaptation)', line=dict(color='red')))
    
    # Add selected scenario costs
    fig_cost_savings.add_trace(go.Scatter(x=selected_years, y=costs, mode='lines', name=f'Selected Scenario ({rcp_scenario}, {adaptation_effectiveness}%)', line=dict(color='blue')))
    
    # Shade the area between the graphs to represent mitigation and adaptation
    fig_cost_savings.add_trace(go.Scatter(
        x=selected_years + selected_years[::-1],
        y=worst_costs + costs[::-1],
        fill='toself',
        fillcolor='rgba(0, 255, 0, 0.2)',
        line=dict(color='rgba(255, 255, 255, 0)'),
        showlegend=True,
        name='Mitigation'
    ))
    fig_cost_savings.add_trace(go.Scatter(
        x=selected_years + selected_years[::-1],
        y=zero_adaptation_costs + costs[::-1],
        fill='toself',
        fillcolor='rgba(0, 0, 255, 0.2)',
        line=dict(color='rgba(255, 255, 255, 0)'),
        showlegend=True,
        name='Adaptation'
    ))
    
    fig_cost_savings.update_layout(
        title=f"Cost Savings by Comparing Worst Scenario (RCP 8.5, 0% Adaptation) and Selected Scenario from {time_period[0]} to {time_period[1]}",
        xaxis_title="Year",
        yaxis=dict(
            title="Costs (USD)",
            tickfont=dict(color="#1f77b4")
        ),
        legend=dict(yanchor="bottom", y=-0.7, xanchor="center", x=0.5)
    )
    
    return (
        display_text, 
        current_rate_text, 
        initial_pregnancy_rate_text, 
        adaptation_effectiveness_text, 
        cost_per_case_text, 
        attributable_fraction_text, 
        total_additional_outcomes_text, 
        total_cost_text, 
        total_mitigation_savings_text, 
        total_adaptation_savings_text, 
        total_mitigation_prevented_text, 
        total_adaptation_prevented_text, 
        fig_outcomes, 
        fig_additional_outcomes_prevented, 
        fig_costs, 
        fig_cost_savings
    )

# Export Data Callback
@app.callback(
    Output("download-dataframe-csv", "data"),
    Input("export-button", "n_clicks"),
    prevent_initial_call=True,
)
def export_data(n_clicks):
    if n_clicks is None:
        return dash.no_update

    # Retrieve values from the update_dashboard function (need to be moved to global scope)
    outcome = 'Preterm Birth'  # or get dynamically if possible
    rcp_scenario = 'RCP 2.6'  # or get dynamically if possible
    population_growth_curve = 'Medium Growth'  # or get dynamically if possible
    pregnancy_growth_curve = 'Medium Growth'  # or get dynamically if possible
    time_period = [2024, 2030]  # or get dynamically if possible
    current_population = 7000000000  # or get dynamically if possible
    initial_pregnancy_rate = 0.02  # or get dynamically if possible
    adaptation_effectiveness = 50  # or get dynamically if possible
    
    selected_years, effective_additional_outcomes, costs, additional_outcomes, _, _, _, _ = calculate_additional_outcomes_and_costs(
        outcome, rcp_scenario, population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, adaptation_effectiveness
    )
    
    _, worst_effective_additional_outcomes, worst_costs, worst_additional_outcomes, _, _, _, _ = calculate_additional_outcomes_and_costs(
        outcome, 'RCP 8.5', population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, 0
    )
    
    _, zero_adaptation_outcomes, zero_adaptation_costs, _, _, _, _ = calculate_additional_outcomes_and_costs(
        outcome, rcp_scenario, population_growth_curve, pregnancy_growth_curve, time_period, current_population, initial_pregnancy_rate, 0
    )
    
    mitigation_savings = [worst - selected for worst, selected in zip(worst_costs, costs)]
    adaptation_savings = [zero - selected for zero, selected in zip(zero_adaptation_costs, costs)]
    mitigation_prevented = [worst - selected for worst, selected in zip(worst_additional_outcomes, effective_additional_outcomes)]
    adaptation_prevented = [zero - selected for zero, selected in zip(zero_adaptation_outcomes, effective_additional_outcomes)]

    # Create a DataFrame with the current projections
    data = {
        'Year': selected_years,
        'Effective Additional Outcomes': effective_additional_outcomes,
        'Temperature Increase': calculate_temperature_increase_per_year(rcp_scenarios[rcp_scenario], time_period[0], time_period[1]),
        'Costs': costs,
        'Worst Case Costs': worst_costs,
        'Zero Adaptation Costs': zero_adaptation_costs,
        'Mitigation Savings': mitigation_savings,
        'Adaptation Savings': adaptation_savings,
        'Mitigation Prevented': mitigation_prevented,
        'Adaptation Prevented': adaptation_prevented
    }
    df = pd.DataFrame(data)

    return dcc.send_data_frame(df.to_csv, "projected_outcomes.csv")

# Step 5: Run the App
if __name__ == '__main__':
    app.run(debug=True)
